## Start

### Import libs

In [ ]:
#Import libs 
import pandas as pd
import numpy as np
import statistics as st
from scipy.interpolate import interp1d
from scipy.stats import gmean
from scipy import stats
import math
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from tqdm import tqdm
import textwrap
import seaborn as sns
from statistics import mean
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score as r2, mean_absolute_error as mae, mean_squared_error as mse, accuracy_score
from sklearn.metrics import make_scorer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.inspection import permutation_importance
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.python import keras
from xgboost import XGBRegressor
from datetime import datetime
import random
pd.set_option("display.precision", 3)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('display.max_columns', 15)
import warnings
warnings.filterwarnings('ignore')

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Flatten, Dense, Dropout, Conv1DTranspose, Conv2DTranspose
from tensorflow.keras.initializers import RandomUniform
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

### Upload main data

In [ ]:
# # Loading the ACG_wells_JOINT_BEST_v10.csv file
# path = 'C:\\jupyter\\SPP\\input\\'
# data_init = pd.read_csv(path + 'ACG_wells_JOINT_BEST_v10.csv', sep=',')
# df = data_init.copy()
# df = df[1:]

In [ ]:
# #Select only neccessary data
# df_cln = df[[   'wellName', 'MD', 'BADPORLOG', 'Casings', 'FORMATION', 'DEVI', 'HAZI',
#                 'FLANK', 'FLANK1', 'FLANK2', 'NET', 'NET_VSH','FLUIDS',
#                 'LPERM', 'PHIT', 
#                 'GR_N', 'GRMATRIX', 'GRSHALE','VSH', 'NPSS', 'RHOB', 'RHOF', 'RHOMA', 
#                 'RDEEP',  'SON', 'SONSH', 
#                 'TVD_SCS','TST', 'X', 'Y']]
# #Fill up nan and -9999 values with 0
# df_cln = df_cln.fillna(0)
# df_cln = df_cln.replace(-9999, 0)
# df_cln = df_cln.replace('-9999', '0')
# #Assing proper datatypes for df
# dicttypes = {   'wellName':'string', 'MD':'float', 'BADPORLOG':'int', 'Casings':'float', 'FORMATION':'string','DEVI':'float', 'HAZI':'float',
#                 'FLANK':'int', 'FLANK1':'int', 'FLANK2':'int', 'NET':'int', 'NET_VSH':'int', 'FLUIDS':'int',
#                 'LPERM':'float','PHIT':'float',
#                 'GR_N':'float', 'GRMATRIX':'float', 'GRSHALE':'float', 'VSH':'float', 'NPSS':'float','RHOB':'float','RHOF':'float', 'RHOMA':'float',
#                 'RDEEP':'float', 'SON':'float', 'SONSH':'float',
#                 'TVD_SCS':'float', 'TST':'float', 'X':'float', 'Y':'float'}
# df_cln = df_cln.astype(dicttypes, errors='ignore')
# df_cln.loc[df_cln.FORMATION=='0', 'FORMATION']='None'
# # Save data to parquet
# df_cln.to_parquet('ACG_wells_JOINT_BEST_v10.parquet.gzip', compression='gzip')

In [3]:
#Loading metadata, distribution wells per Platforms and all the that.
path = 'C:\\jupyter\\SPP\\input\\'
metadata_init = pd.read_csv(path + 'ACG_wells_metadata.csv', sep=',')
metadata = metadata_init.copy()
metadata = metadata.rename(columns={'X':'X_wellhead', 'Y':'Y_wellhead'})
metadata.Status = metadata.Status.str.strip()
metadata.Status = metadata.Status.str.lower()
metadata.loc[metadata.Status == 'oil', 'Status' ] = 'production oil'
metadata.loc[metadata.Status == 'oil producer', 'Status' ] = 'production oil'
metadata.loc[metadata.Status == 'production', 'Status' ] = 'production oil'
metadata.loc[metadata.Status == 'produiction oil', 'Status' ] = 'production oil'
metadata.loc[metadata.Status == 'production_oil', 'Status' ] = 'production oil'
metadata.loc[metadata.Status == 'abandoned production oil', 'Status' ] = 'abandoned oil'
metadata.loc[metadata.Status == 'abandoned  oil', 'Status' ] = 'abandoned oil'
metadata.loc[metadata.Status == 'abandoned oi', 'Status' ] = 'abandoned oil'
metadata.loc[metadata.Status == 'injector  - water', 'Status' ] = 'injector - water'
metadata.loc[metadata.Status == 'injector water', 'Status' ] = 'injector - water'
metadata.loc[metadata.Status == 'injetor  - water', 'Status' ] = 'injector - water'
metadata.loc[metadata.Status == 'abandoned injector - water per b', 'Status' ] = 'abandoned injector - water'
metadata.loc[metadata.Status == 'plugged and abandoned', 'Status' ] = 'p&a'
metadata.loc[metadata.X_wellhead==118.270, 'X_wellhead'] = 526258.84
metadata.loc[metadata.Y_wellhead==526261.510, 'Y_wellhead'] = 4435802.01
metadata.loc[metadata.well=='C39', 'X_wellhead'] = 526258.840
metadata.loc[metadata.well=='C39', 'Y_wellhead'] = 4435802.010
metadata.loc[metadata.field=='West Azeri', 'field'] = 'WEST AZERI'
metadata.loc[metadata.field=='COP', 'field'] = 'WEST CHIRAG'
metadata.loc[metadata.well=='AZERI2', 'field'] = 'WEST AZERI'
metadata.loc[metadata.well=='AZERI3', 'field'] = 'WEST AZERI'
metadata.loc[metadata.well=='B31', 'field'] = 'CENTRAL AZERI'
metadata.loc[metadata.well=='J28_bpQIP', 'field'] = 'WEST CHIRAG'

#Read data from parquet
path = 'C:\\jupyter\\SPP\\input\\'
df_prq = pd.read_parquet(path + 'ACG_wells_JOINT_BEST_v10.parquet.gzip')
df_prq.rename(columns={'wellName':'well'}, inplace=True)
df_prq = df_prq.set_index('well').join(metadata.set_index('well')).reset_index()
# print('wells in df totally:', len(df_prq.well.unique()))
# Filter data with bad_well_list 
bad_well_list = ['E10Z','Predrill_J01Z', 'Predrill_J08', 'J28_bpQIP', 'A01W_2']
df_prq = df_prq[~df_prq.well.isin(bad_well_list)]
#Assign any Fluidcode_mod number by variable gross_pay=1 and gross_pay=0 if Fluidcode_mod as NaN
df_prq.loc[df_prq.FLUIDS>0, 'FLUIDS_int'] = 1
df_prq.loc[df_prq.FLUIDS<=0, 'FLUIDS_int'] = 0
df_prq.FLUIDS_int = df_prq.FLUIDS_int.astype('int')
# Unite of FU for each formation

df_bal = df_prq[df_prq.FORMATION.str.contains('Balakhany')]
df_bal.loc[df_bal.FORMATION.str.contains('Balakhany VIII'), 'FORMATION_up'] = 'Balakhany VIII'
df_bal.loc[df_bal.FORMATION.str.contains('Balakhany X'), 'FORMATION_up'] = 'Balakhany X'
df_bal = df_bal[df_bal.FORMATION_up.notna()]
#Getting XY mean coords of Balakhany formation
xy_coord_mean = df_bal[['well', 'FORMATION_up', 'X', 'Y']]
xy_coord_mean = xy_coord_mean.groupby(['well', 'FORMATION_up']).agg({'X': 'mean', 'Y':'mean'}).reset_index()
xy_coord_mean = xy_coord_mean[xy_coord_mean.FORMATION_up.str.contains('Balakhany') & (xy_coord_mean.X>0) & (xy_coord_mean.Y>0)]
df_bal.rename(columns={'X':'X_traj', 'Y':'Y_traj'}, inplace=True)
df_bal = df_bal.set_index(['well', 'FORMATION_up']).join(xy_coord_mean.set_index(['well', 'FORMATION_up'])).reset_index()

## List of functions

In [ ]:
# Display 3 offsets wells
def display_3offset_wells(well, formation, dataset_dist=df_dist_kh_bal_fin, dataset_logs=df_bal_net2_kh):
    """
    Pay attention dataset_dist=df_dist_kh_bal_fin, dataset_logs=df_bal_net2_kh
    well:       just well name
    formation:  just formation
    """
    def well_offset_selection(dataset_dist, fmname, well_target):
        try:
            well_df = dataset_dist[(dataset_dist.well == well_target) & (dataset_dist.FORMATION_up == fmname)][['well', 'well1', 'well2', 'well3',
                                                                                                                        'dist1', 'dist2', 'dist3',
                                                                                                                'KHtst','KHtst_1', 'KHtst_2', 'KHtst_3']]
            well1 = well_df['well1'].iloc[0]
            well2 = well_df['well2'].iloc[0]
            well3 = well_df['well3'].iloc[0]
            dist1 = well_df['dist1'].astype('int').iloc[0]
            dist2 = well_df['dist2'].astype('int').iloc[0]
            dist3 = well_df['dist3'].astype('int').iloc[0]
            kh = well_df['KHtst'].astype('int').iloc[0]
            kh1 = well_df['KHtst_1'].astype('int').iloc[0]
            kh2 = well_df['KHtst_2'].astype('int').iloc[0]
            kh3 = well_df['KHtst_3'].astype('int').iloc[0]
        except Exception as e:
            print(f'It looks like the desired formation is absent. The error is "{e}"')
        return {'target': well_target, 'w1':well1, 'w2':well2, 'w3':well3, 
                'dist': 0,'d1':dist1, 'd2':dist2,'d3':dist3,
                'kh':kh,'kh1':kh1, 'kh2':kh2, 'kh3':kh3}
    def display_tracks(dataset, wellname, fmname, ref_depth, depth_step, r, c, kh_value, dist):
        try:
            data = dataset[(dataset.well==wellname) & (dataset.FORMATION_up == fmname)]
            depth = data[ref_depth]
            grn = data['GR_N']
            vsh = data['VSH']
            rhob = data['RHOB'] 
            npss = data['NPSS']
            rdeep = data['RDEEP']
            phit = data['PHIT'] 
            net = data['NET_clp2']
            perm = data['LPERM']
            kh = data['KHtst']
            well_bal_tops = df_bal[(df_bal.well == wellname)].groupby('FORMATION')[ref_depth].apply(lambda x: x.iloc[0]).reset_index()
            ax[r,c].plot(grn, depth, color='lightgreen', lw=2, zorder=10)
            ax[r,c].set_xlim(0, 150) 
            ax[r,c].grid(axis='y')
            ax[r,c].invert_yaxis()
            ax[r,c].yaxis.set_ticks(np.arange(min(depth), max(depth), depth_step))
            ax[r,c].set_xticks([])
            ax[r,c].tick_params(axis='y', labelsize=8)
            ax[r,c].set_title(wellname + ' ' + fmname + ' kh:' + str(kh_value) + ' dist:' + str(dist), fontsize=12) 
            for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
                ax[r,c].hlines(    well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
                                    xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
                # ax[r,c].text(10, well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0]+0.5*depth_step, i, fontsize = 7, color ="black")
            ax[r,c+1].plot(rhob, depth, color='red')
            ax[r,c+1].xaxis.set_ticks(np.arange(1.65, 2.65, 0.3))
            ax[r,c+1].set_xlim(1.65, 2.65)
            ax[r,c+1].grid(axis='y')
            ax[r,c+1].grid(axis='x')
            ax[r,c+1].invert_yaxis()
            ax[r,c+1].yaxis.set_ticks(np.arange(min(depth), max(depth), depth_step))
            ax[r,c+1].set_xticks([])
            ax[r,c+1].set_yticks([])
            for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
                ax[r,c+1].hlines( well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
                                xmin=0, xmax=150, linewidth=2, color='black', lw=2, alpha=0.33)
                ax[r,c+1].text(1.67, well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0]+0.5*depth_step, i, fontsize = 7, color ="black")
            twin1 = ax[r,c+1].twiny()
            twin1.plot(npss, depth, color='blue')
            twin1.set_xlim(0.6, 0)
            twin1.set_xticks([])
            ax[r,c+2].plot(phit, depth, color='green', linestyle='dashed')
            ax[r,c+2].set_xlim(0.3, 0)
            ax[r,c+2].grid(axis='x')
            ax[r,c+2].grid(axis='y')
            ax[r,c+2].invert_yaxis()
            ax[r,c+2].yaxis.set_ticks(np.arange(min(depth), max(depth), depth_step))
            ax[r,c+2].set_xticks([])
            ax[r,c+2].set_yticks([])
            ax[r,c+2].vlines(0.13, ymin=min(depth), ymax=max(depth), color='black', linestyle='dashed')
            for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
                ax[r,c+2].hlines(    well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
                                    xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
            twin2 = ax[r,c+2].twiny()
            twin2.plot(net, depth, color='orange', linewidth=0.5)
            twin2.fill_betweenx(depth,net, color='orange', alpha=0.33)
            twin2.set_xlim(0, 1)
            twin2.set_xticks([])
            ax[r,c+3].plot(perm, depth, color='purple', alpha=0.66)
            ax[r,c+3].set_xscale('log')
            ax[r,c+3].set_xlim(0.1, 1000)
            ax[r,c+3].grid(axis='y')
            ax[r,c+3].grid(axis='x')
            ax[r,c+3].invert_yaxis()
            ax[r,c+3].yaxis.set_ticks(np.arange(min(depth), max(depth), depth_step))
            ax[r,c+3].set_xticks([])
            ax[r,c+3].set_yticks([])
            for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
                ax[r,c+3].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.5)
            twin4 = ax[r,c+3].twiny()
            twin4.plot(kh, depth, color='black', alpha=1)
            twin4.set_xticks([])
        except Exception as e:
            print(f'It looks like the desired formation is absent. The error is "{e}"')
        return fig.show()

In [5]:
# Function to display in TST well plots with logging curves
def well_display_khtst( dataset, wellname, fmname, net_var, comments, 
                        ref_depth, fm_flag, depth_step, kh_include, print):
    """
    dataset = df_bal or something else
    net_var = NET or FLUIDS_int
    comments = put what you want
    ref_depth = MD or TST
    fm_flag = 1 if you need a FORMATION_up, 0 if just a simple FORMATION
    depth_step = step for ticks on the diagramm
    kh_include = 1 if we have KHtst in dataset, 0 if there is not KHtst
    print = 1 if we want to print the plot
    """
    if fm_flag == 0:
        data = dataset[(dataset.well==wellname) & (dataset.FORMATION == fmname)]
    if fm_flag == 1:
        data = dataset[(dataset.well==wellname) & (dataset.FORMATION_up == fmname)]
    depth = data[ref_depth]
    grn = data['GR_N']
    vsh = data['VSH']
    rhob = data['RHOB'] 
    npss = data['NPSS']
    rdeep = data['RDEEP']
    phit = data['PHIT'] 
    net = data[net_var]
    perm = data['LPERM']
    if kh_include == 1:
        kh = data['KHtst']
    else:
        data['KHtst'] = 0
        kh = data['KHtst']
    fig, ax = plt.subplots(1,4, figsize=(7,7), sharey=True)
    well_bal_tops = df_bal[(df_bal.well == wellname)].groupby('FORMATION')[ref_depth].apply(lambda x: x.iloc[0]).reset_index()
    ax[0].yaxis.set_ticks(np.arange(min(depth), max(depth), depth_step))
    ax[0].plot(grn, depth, color='lightgreen', lw=3, zorder=10)
    ax[0].invert_yaxis() 
    ax[0].set_xlim(0, 150) 
    ax[0].grid(axis='y')
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[0].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
                    xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.33)
    twin0 = ax[0].twiny()
    twin0.plot(vsh, depth, color='black', alpha=0.5, zorder=5)
    twin0.set_xlim(0, 1.5)
    ax[1].plot(rhob, depth, color='red') 
    ax[1].invert_yaxis() 
    ax[1].xaxis.set_ticks(np.arange(1.65, 2.65, 0.3))
    ax[1].set_xlim(1.65, 2.65)
    ax[1].grid(axis='y'), ax[1].grid(axis='x')
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[1].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], 
        xmin=0, xmax=150, linewidth=2, color='black', lw=2, alpha=0.33)
        ax[1].text(1.67, well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0]+0.5*depth_step, i, fontsize = 7, color ="black")
    twin1 = ax[1].twiny()
    twin1.plot(npss, depth, color='blue')
    twin1.set_xlim(0.6, 0)
    # ax[2].plot(rdeep, depth, color='black'), ax[2].set_xscale('log'), ax[2].set_xlim(0.1, 50), ax[2].invert_yaxis(), ax[2].grid(axis='x', which='both')
    ax[2].plot(phit, depth, color='green', linestyle='dashed'), ax[2].set_xlim(0.3, 0), ax[2].grid(axis='x') 
    ax[2].invert_yaxis()
    ax[2].grid(axis='y')
    ax[2].vlines(0.13, ymin=min(depth), ymax=max(depth), color='black', linestyle='dashed')
    twin2 = ax[2].twiny()
    twin2.plot(net, depth, color='orange', linewidth=0.5)
    twin2.fill_betweenx(depth,net, color='orange', alpha=0.33)
    twin2.set_xlim(0, 1)
    twin2.set_ylim(min(depth), max(depth))
    ax[3].plot(perm, depth, color='purple', alpha=0.66), ax[3].set_xscale('log'), ax[3].set_xlim(0.1, 1000)
    ax[3].invert_yaxis()
    ax[3].grid(axis='y')
    for i in well_bal_tops[well_bal_tops.FORMATION.str.contains(fmname)].FORMATION:
        ax[3].hlines(well_bal_tops[well_bal_tops.FORMATION==i][ref_depth].iloc[0], xmin=0, xmax=1000, linewidth=2, color='black', lw=2, alpha=0.66)
    twin4 = ax[3].twiny()
    twin4.plot(kh, depth, color='black', alpha=1)
    fig.suptitle(wellname + ' ' + fmname + ' ' + ref_depth + ' ' + str(round(max(kh.dropna()),0)) + ' ' + str(comments), fontsize=14)
    fig.tight_layout()
    if print == 1:
        path = 'C:\\jupyter\\SPP\\inputoutput\\wellplots\\'
        fig.savefig(path + fmname.replace(' ','') + '_' + wellname + '.png')
    else:
        pass

    def display_subplots():
        try:
            well_dist_dict = well_offset_selection(dataset_dist, fmname, well_target)
            display_tracks(dataset_logs, well_dist_dict['target'], fmname,'TST', 10, 0,0,well_dist_dict['kh'], well_dist_dict['dist'])
            display_tracks(dataset_logs, well_dist_dict['w1'], fmname,'TST', 10 ,0,4, well_dist_dict['kh1'], well_dist_dict['d1'])  
            display_tracks(dataset_logs, well_dist_dict['w2'], fmname,'TST', 10,1,0, well_dist_dict['kh2'], well_dist_dict['d2'])      
            display_tracks(dataset_logs, well_dist_dict['w3'], fmname,'TST', 10,1,4, well_dist_dict['kh3'], well_dist_dict['d3'])
        except Exception as e:
            print(f'It looks like the desired formation is absent. The error is "{e}"')
    well_target = well
    fmname = formation
    fig, ax = plt.subplots(2,8, figsize=(9,8), constrained_layout=True)
    return display_subplots()
# Draw a map
def map_value_2plots(metadata, dataset, formation, value, color, multi_chr = 0.001, multi_azr = 0.001):
    """
    metadata, 
    dataset = dataset with X & Y, 
    formation = 'Balakhany VIII',  
    value = for example 'KHtst' or 'tst_interv'
    multi_chr = 0.001, multi_azr = 0.001
    """
    fig = make_subplots(rows=2, cols=1, subplot_titles=('crg: ' + str(multi_chr), 'azr: ' + str(multi_azr)), 
                        vertical_spacing = 0.025)
    azr_lst = ['CENTRAL AZERI', 'WEST AZERI', 'EAST AZERI']
    chg_lst = ['CHIRAG', 'DWG', 'DDGG', 'WEST CHIRAG']
    field_avg_coord = metadata.groupby('field')[['X_wellhead','Y_wellhead']].mean().reset_index()
    field_avg_coord_chg = field_avg_coord[field_avg_coord.field.isin(chg_lst)]
    field_avg_coord_azr = field_avg_coord[field_avg_coord.field.isin(azr_lst)] 
    df_chg = dataset[(dataset.FORMATION_up == formation) & (dataset.field.isin(chg_lst))]
    df_azr = dataset[(dataset.FORMATION_up == formation) & (dataset.field.isin(azr_lst))]
    fig.add_trace(go.Scatter(x=df_chg.X, y=df_chg.Y, customdata = df_chg[['well', value, color]],
                            marker=dict(color=df_chg[color], size=df_chg[value]*multi_chr, colorscale='Viridis_r',  showscale=True,
                            line=dict(color='rgb(47, 57, 61)', width=0.5)),
                            mode='markers', hovertemplate="".join(["well:%{customdata[0]}, value:%{customdata[1]}, color:%{customdata[2]}<extra></extra>"])),
                            row=1, col=1)
    fig.add_trace(go.Scatter(x=field_avg_coord_chg.X_wellhead, y=field_avg_coord_chg.Y_wellhead, customdata = field_avg_coord_chg[['field']],
                            text=field_avg_coord_chg['field'], textposition="middle right",
                            marker=dict(color='rgb(0, 0,0)', size=12),
                            mode='markers+text', 
                            marker_symbol='square', hovertemplate="".join(["%{customdata[0]}<extra></extra>"])),
                            row=1, col=1)
    fig.add_trace(go.Scatter(x=df_azr.X, y=df_azr.Y, customdata = df_azr[['well', value, color]],
                            marker=dict(color=df_azr[color], size=df_azr[value]*multi_azr, colorscale='Viridis_r',  showscale=False,
                            line=dict(color='rgb(47, 57, 61)', width=0.5)),
                            mode='markers', hovertemplate="".join(["well:%{customdata[0]}, value:%{customdata[1]}, color:%{customdata[2]}<extra></extra>"])),
                            row=2, col=1)
    fig.add_trace(go.Scatter(x=field_avg_coord_azr.X_wellhead, y=field_avg_coord_azr.Y_wellhead, customdata = field_avg_coord_azr[['field']],
                            text=field_avg_coord_azr['field'], textposition="middle right",
                            marker=dict(color='rgb(0, 0,0)', size=12),
                            mode='markers+text', 
                            marker_symbol='square', hovertemplate="".join(["%{customdata[0]}<extra></extra>"])),
                            row=2, col=1)
    fig.update_layout(  title_text= ('formation: ' + str(formation) + ' value: ' + str(value) + ' color: ' + str(color)),
                        autosize=True, width=1300, height=1400, margin=dict(l=10,r=10,b=10,t=50), showlegend=False)
    return fig.show()
# Calculation NTD
def ntd_calculation_big(dataset, desired_fm, net_var='NET'):
    df_lst = []
    for well_in_loop in tqdm(dataset.well.unique()[:]):
        well_lst = []
        data = dataset[(dataset.well==well_in_loop)]
        data.iloc[0, 3] = 0
        data.iloc[-1, 3] = 0
        tst_top = [data.iloc[i]['TST'] for i in range(len(data)-1)
                    if (data.iloc[i][net_var] == 1 and data.iloc[i-1][net_var]==0)]
        tst_bot = [data.iloc[i]['TST'] for i in range(len(data)-1)
                    if (data.iloc[i][net_var] == 1 and data.iloc[i+1][net_var]==0)]
        for k in range(len(tst_top)):
            if (round(tst_top[k],1) == round(tst_bot[k],1)):
                h_tst = 0 
            elif (round(tst_bot[k],1) == round(tst_top[k]+0.1,1)):
                h_tst = 0
            else:
                h_tst = (round((tst_bot[k] - tst_top[k]),1))
                md_perm = []
                md_phit = []
                md_vsh = []
                for i in range(len(data)):
                    if round(data.iloc[i]['TST'],1) >= round(tst_top[k],1) and round(data.iloc[i]['TST'],1) <= round(tst_bot[k],1):
                        md_perm.append(data.iloc[i]['LPERM'])
                        md_phit.append(data.iloc[i]['PHIT'])
                        md_vsh.append(data.iloc[i]['VSH'])
                if len(md_perm) == 0:
                    md_perm.append(0)
                if len(md_phit) == 0:
                    md_phit.append(0)
                if len(md_vsh) == 0:
                    md_vsh.append(0)
                well_lst.append([data.iloc[0]['well'], h_tst, tst_top[k], tst_bot[k], round(mean(md_perm),0), round(mean(md_phit),2), round(mean(md_vsh),2)])
            df_tst = pd.DataFrame(well_lst, columns = ['well', 'h_tst', 'top_tst', 'bot_tst', 'md_perm_avg', 'md_phit_avg', 'md_vsh_avg'])
        df_lst.append(df_tst)
    ntd_bal = pd.concat(df_lst)
    ntd_bal['FORMATION_up'] = desired_fm
    return ntd_bal
def ntd_calculation_brief(dataset,well,desired_fm, net_var='NET'):
    data = dataset[(dataset.well==well) & (dataset.FORMATION_up==desired_fm)]
    data.iloc[0, 3] = 0
    data.iloc[-1, 3] = 0
    tst_top = [data.iloc[i]['TST'] for i in range(len(data)-1)
                if (data.iloc[i][net_var] == 1 and data.iloc[i-1][net_var]==0)]
    tst_bot = [data.iloc[i]['TST'] for i in range(len(data)-1)
                if (data.iloc[i][net_var] == 1 and data.iloc[i+1][net_var]==0)]
    tops = zip(tst_top, tst_bot)
    df_htst = pd.DataFrame(tops, columns=['tst_top', 'tst_bot'])
    df_htst['FORMATION_up'] = desired_fm
    df_htst['well'] = well
    df_htst['h_tst'] = df_htst.tst_bot - df_htst.tst_top
    df_htst = df_htst[['well','FORMATION_up','tst_top','tst_bot','h_tst']]
    return df_htst
# Calculation NTD zero
def ntd_calculation_zero(dataset,well,formation, net_var='NET'):
    data = dataset[(dataset.well==well) & (dataset.FORMATION_up==formation)]
    data.iloc[0, 3] = 1
    data.iloc[-1, 3] = 1
    tst_zero_top = [data.iloc[i]['TST'].round(3) for i in range(len(data)-1)
                if (data.iloc[i][net_var] == 0 and data.iloc[i-1][net_var] == 1)]
    tst_zero_bot = [data.iloc[i]['TST'].round(3) for i in range(len(data)-1) 
                if (data.iloc[i][net_var] == 0 and data.iloc[i+1][net_var] == 1)]
    tops_zero = zip(tst_zero_top, tst_zero_bot)
    df_zero_htst = pd.DataFrame(tops_zero, columns=['tst_zero_top', 'tst_zero_bot'])
    df_zero_htst['FORMATION_up'] = formation
    df_zero_htst['well'] = well
    df_zero_htst['h_tst_zero'] = df_zero_htst.tst_zero_bot - df_zero_htst.tst_zero_top
    df_zero_htst = df_zero_htst[['well','FORMATION_up','tst_zero_top','tst_zero_bot','h_tst_zero']]
    return df_zero_htst
# Print numerical table with layers
def ntd_numerical(dataset, wellname, fmname):
    """
    dataset = ntd_final
    """
    df = dataset[(dataset.well==wellname) & (dataset.FORMATION_up == fmname) ][['well','h_tst','top_tst', 'bot_tst','FORMATION_up']]
    q50 = df['h_tst'].quantile(q=0.5, interpolation='nearest')
    df['q50'] = q50
    return df
#Cleaning NET variable and making up NET_clp with clipped data
def ntd_htst_cleaning(dataset, cutoff):
    """
    dataset - any updated dataset like df_bal...
    cutoff - value in TST to remove layers with thickness below cutoff
    """
    df_list_ntd = []
    for well in tqdm(dataset.well.unique()):
        ntd_well = dataset[(dataset.well ==well)]
        ntd_well_cutoff = ntd_well[ntd_well.h_tst >= cutoff]
        well_short = df_bal[['well', 'FORMATION_up', 'MD', 'TST', 'GR_N', 'NET', 'FORMATION']]
        net_well = well_short[(well_short.well==well)]
        net_well['NET_clp'] = 0
        for j in range(len(ntd_well_cutoff.well)):
            ntd_top = ntd_well_cutoff.iloc[j, 2].round(3)
            ntd_bot = ntd_well_cutoff.iloc[j, 3].round(3)
            for i in range(len(net_well.TST)):
                well_tst = net_well['TST'].iloc[i].round(3)
                if well_tst >= ntd_top and well_tst <= ntd_bot:
                    net_well['NET_clp'].iloc[i] = 1
        df_list_ntd.append(net_well)
    net_clp = pd.concat(df_list_ntd)
    return net_clp
# Cleaning NET_clp variable from zero values with zero_samples <=cutoff
def ntd_htst_zero_cleaning(dataset_zero, dataset, cutoff, net_var1, net_var2):
    df_list_ntd_zero = []
    for well in tqdm(dataset_zero.well.unique()):
        ntd_well_zero = dataset_zero[(dataset_zero.well ==well)]
        ntd_well_zero_sel = ntd_well_zero[ntd_well_zero.h_tst_zero <= cutoff]
        well_zero_short = dataset[['well','FORMATION_up','MD','TST', net_var1, 'GR_N', 'NET', 'FORMATION']]
        well_zero_short[net_var2] = well_zero_short[net_var1]
        well_zero_sel = well_zero_short[(well_zero_short.well==well)]
        for j in range(len(ntd_well_zero_sel.well)):
            ntd_zero_top = ntd_well_zero_sel.iloc[j, 2].round(3)
            ntd_zero_bot = ntd_well_zero_sel.iloc[j, 3].round(3)
            for i in range(len(well_zero_sel.TST)):
                well_zero_tst = well_zero_sel['TST'].iloc[i].round(3)
                if well_zero_tst >= ntd_zero_top and well_zero_tst <= ntd_zero_bot:
                    well_zero_sel[net_var2].iloc[i] = 1
        df_list_ntd_zero.append(well_zero_sel)
    result = pd.concat(df_list_ntd_zero)
    return result
# View desired TST-interval
def net_view1(dataset, well, top, bot):
    dataset = dataset[dataset.well==well][['well','TST','GR_N', 'RHOB', 'NET','NET_clp']]
    return dataset[(dataset.TST >= top) & (dataset.TST <= bot)].head(50)
#TST sampling & TST KH curve calculation per formation/well
def proph_calculation(dataset, net_var):
    df_smpl_lst = []
    print('TST sampling calculation')
    for well_smpl in tqdm(dataset.well.unique()[:]):
        tst_sampl = dataset[dataset.well==well_smpl]['TST'].diff()
        df_new = dataset[dataset.well==well_smpl].join(tst_sampl, rsuffix='_smpl')    
        df_smpl_lst.append(df_new)
    df_bal_tst_smpl = pd.concat(df_smpl_lst)
    df_kh_lst_fm = []
    print('KHtst calculation')
    for fm_kh in ['Balakhany VIII', 'Balakhany X']:
        df_kh_lst = []
        for well_kh in tqdm(dataset.well.unique()[:]):
            well_tst_perm = df_bal_tst_smpl[(df_bal_tst_smpl.well==well_kh) & 
                                            (df_bal_tst_smpl.FORMATION_up==fm_kh)].sort_values(by='MD', ascending=False)
            well_tst_perm.loc[well_tst_perm[net_var] == 0, 'LPERM'] = 0
            well_tst_perm.loc[well_tst_perm[net_var] == 0, 'PHIT'] = 0
            well_tst_perm.loc[well_tst_perm[net_var] == 0, 'VSH'] = 0
            well_tst_perm['khtst'] = well_tst_perm.LPERM*well_tst_perm.TST_smpl
            well_tst_perm['phithtst'] = well_tst_perm.PHIT*well_tst_perm.TST_smpl
            well_tst_perm['vshhtst'] = well_tst_perm.VSH*well_tst_perm.TST_smpl
            well_tst_perm['KHtst'] = well_tst_perm.khtst.cumsum()
            well_tst_perm['PHITHtst'] = well_tst_perm.phithtst.cumsum()
            well_tst_perm['VSHHtst'] = well_tst_perm.vshhtst.cumsum()
            well_tst_perm = well_tst_perm.sort_values(by='MD')
            df_kh_lst.append(well_tst_perm)
        df_khlst = pd.concat(df_kh_lst)
        df_kh_lst_fm.append(df_khlst)
    df_khlst_fm = pd.concat(df_kh_lst_fm)
    # df_khlst_fm = df_khlst_fm.dropna()
    return df_khlst_fm[['well', 'FORMATION_up', 'MD', 'TST', 'TST_smpl','KHtst','PHITHtst','VSHHtst']]
# Comparison NET_clp and NET_clp2
def well_display_net(dataset, well, formation, net1='NET_clp', net2_flag=0, net2='NET_clp_v2'):
    well_sel = dataset[(dataset.well == well) & (dataset.FORMATION_up == formation)]
    depth = well_sel['TST']
    grn = well_sel['GR_N']
    net = well_sel['NET']
    net_clp = well_sel[net1]
    if net2_flag == 0:
        fig, ax = plt.subplots(1,3, figsize=(4.5,8), sharey=True)
        ax[0].yaxis.set_ticks(np.arange(min(depth), max(depth), 5))
        ax[0].plot(grn, depth, color='green'), ax[0].invert_yaxis(), ax[0].set_xlim(0, 150), ax[0].grid(axis='y')
        well_bal_tops = well_sel.groupby('FORMATION')['TST'].apply(lambda x: x.iloc[0]).reset_index()
        for i in well_bal_tops[well_bal_tops.FORMATION.str.contains('Balakhany VIII')].FORMATION:
            ax[0].hlines(well_bal_tops[well_bal_tops.FORMATION==i]['TST'].iloc[0], xmin=0, xmax=150, color='black', lw=2, alpha=0.66)
            ax[0].text(10, well_bal_tops[well_bal_tops.FORMATION==i]['TST'].iloc[0]+3, i, fontsize = 7, color ="black")
        ax[1].plot(net, depth, color='orange'), ax[1].set_xlim(0, 1), ax[1].grid(axis='y')
        ax[1].fill_betweenx(depth,net, color='orange', alpha=0.33)
        ax[2].plot(net_clp, depth, color='orange'), ax[2].set_xlim(0, 1), ax[2].grid(axis='y')
        ax[2].fill_betweenx(depth,net_clp, color='orange', alpha=0.33)
        fig.suptitle(well_sel.well.unique()[0], fontsize=14)
        fig.tight_layout()
    if net2_flag == 1:
        net_clp2 = well_sel[net2]
        fig, ax = plt.subplots(1,4, figsize=(6,8), sharey=True)
        ax[0].yaxis.set_ticks(np.arange(min(depth), max(depth), 5))
        ax[0].plot(grn, depth, color='green'), ax[0].invert_yaxis(), ax[0].set_xlim(0, 150), ax[0].grid(axis='y')
        well_bal_tops = well_sel.groupby('FORMATION')['TST'].apply(lambda x: x.iloc[0]).reset_index()
        for i in well_bal_tops[well_bal_tops.FORMATION.str.contains('Balakhany VIII')].FORMATION:
            ax[0].hlines(well_bal_tops[well_bal_tops.FORMATION==i]['TST'].iloc[0], xmin=0, xmax=150, color='black', lw=2, alpha=0.66)
            ax[0].text(10, well_bal_tops[well_bal_tops.FORMATION==i]['TST'].iloc[0]+3, i, fontsize = 7, color ="black")
        ax[1].plot(net, depth, color='orange', lw=0.25), ax[1].set_xlim(0, 1), ax[1].grid(axis='y')
        ax[1].fill_betweenx(depth,net, color='orange', alpha=0.33)
        ax[2].plot(net_clp, depth, color='orange', lw=0.25), ax[2].set_xlim(0, 1), ax[2].grid(axis='y')
        ax[2].fill_betweenx(depth,net_clp, color='orange', alpha=0.33)
        ax[3].plot(net_clp2, depth, color='orange', lw=0.25), ax[3].set_xlim(0, 1), ax[3].grid(axis='y')
        ax[3].fill_betweenx(depth,net_clp2, color='orange', alpha=0.33)
        fig.suptitle(well_sel.well.unique()[0], fontsize=14)
        fig.tight_layout()
    return fig.show()
# Run RFR model with train/test split
def rfr_train_test_split(train_dataset, gs_set, scorer, target='KHtst', rng=0.25, margin=0.005):
    """
    'train_ds', 
    'metrics: r2_train, r2_test, mae_train, mae_test, test_in', 
    'grid_search', 
    'result_df', 
    'train_df', 
    'test_df'
    --------
    scorer = make_scorer(mse, greater_is_better=False) <- format scorer like this
    """
    train_dataset_list = []
    grids_setting_list = []
    metrics_dict = []
    # X_train/x_test data splitting
    y = np.array(train_dataset[['well','FORMATION_up',target]])
    x = np.array(train_dataset.drop(target, axis=1))
    num = random.randint(0,100)
    # num=42
    train_dataset_list.append(train_dataset.drop(['FORMATION_up', target], axis=1).columns[1:].values.tolist())
    x_train_init, x_test_init, y_train_init, y_test_init = train_test_split(x, y, test_size=0.3, random_state=num)
    # Taking well names from train/test datasets
    # x_train_wells = x_train_init[:,2]
    # x_test_wells = x_test_init[:,2]
    y_train_wells = y_train_init[:,0:2]
    y_test_wells = y_test_init[:,0:2]
    x_train = x_train_init[:,2:]
    x_test = x_test_init[:,2:]
    y_train = y_train_init[:,2]
    y_test = y_test_init[:,2]
    # GridSearch for ML-model
    grid_rfr = RandomForestRegressor(n_jobs=-1, random_state=42)
    grid_calc_rfr = GridSearchCV(estimator = grid_rfr, param_grid = gs_set, scoring=scorer, cv = 5)
    grid_calc_rfr.fit(x_train, y_train)
    gd_sr_setting = grid_calc_rfr.best_params_
    grids_setting_list.append(gd_sr_setting)
    print('Grid_search: ', grid_rfr)
    # Applying Pipeline for ML-model
    rfr = Pipeline([("scaler",StandardScaler()),("rfr",RandomForestRegressor(**gd_sr_setting, n_jobs=-1, random_state=42))])
    rfr.fit(x_train, y_train)
    y_pred_train = rfr.predict(x_train)
    y_pred_test = rfr.predict(x_test)
    # Reporting
    print('Pipeline: ', rfr.steps[1][1])
    up_range = rng + 1
    dwn_range = 1 - rng
    well_fm_train = pd.DataFrame(y_train_wells, columns=['well', 'FORMATION_up'])
    rfr_train = pd.DataFrame(zip(y_train, y_pred_train), columns=['actual','predict'])
    df_rfr_train = well_fm_train.join(rfr_train)
    df_rfr_train['l_limit'] = df_rfr_train.actual*dwn_range - margin
    df_rfr_train['h_limit'] = df_rfr_train.actual*up_range + margin
    df_rfr_train['qc'] = 'out'
    df_rfr_train['dataset'] = 'train'
    df_rfr_train.loc[(df_rfr_train.predict >= df_rfr_train.l_limit) & (df_rfr_train.predict <= df_rfr_train.h_limit), 'qc'] = 'in'
    well_fm_test = pd.DataFrame(y_test_wells, columns=['well', 'FORMATION_up'])
    rfr_test = pd.DataFrame(zip(y_test, y_pred_test), columns=['actual','predict'])
    df_rfr_test = well_fm_test.join(rfr_test)
    df_rfr_test['l_limit'] = df_rfr_test.actual*dwn_range - margin
    df_rfr_test['h_limit'] = df_rfr_test.actual*up_range + margin
    df_rfr_test['qc'] = 'out'
    df_rfr_test['dataset'] = 'test'
    df_rfr_test.loc[(df_rfr_test.predict >= df_rfr_test.l_limit) & (df_rfr_test.predict <= df_rfr_test.h_limit), 'qc'] = 'in'
    df_rfr_result = pd.concat([df_rfr_train,df_rfr_test])
    df_rfr_result['diff'] = (df_rfr_result.actual - df_rfr_result.predict).round(3)
    metrics_dict = {    'r2_train':     r2(y_train, y_pred_train).round(2), 
                        'r2_test':      r2(y_test, y_pred_test).round(2),
                        'mae_train':    mae(y_train, y_pred_train).round(2), 
                        'mae_test':     mae(y_test, y_pred_test).round(2),
                        'train_in':     df_rfr_train['qc'].value_counts(normalize=True)['in'].round(2),
                        'test_in':      df_rfr_test['qc'].value_counts(normalize=True)['in'].round(2)}
    feature_imp = pd.Series(rfr.steps[1][1].feature_importances_, index=train_dataset_list[0]).sort_values(ascending=True)
    return {'train_ds':train_dataset_list[0], 
            'metrics':metrics_dict, 
            'grid_search' : grids_setting_list, 
            'result_df' : df_rfr_result,
            'train_df' : df_rfr_train,
            'test_df' : df_rfr_test,
            'feature_imp' : feature_imp}
# Run RFR model with loop
def rfr_loop(dataset, fmname, target, hyperdict, rng, margin):
    """
    'train_ds', 'train_ftrs', 'result_df', 'grid_search', 'metrics'
    """
    y_test_lst = []
    y_pred_test_lst = []
    well_exclude_lst = []
    fm_exclude_lst = []
    gs_settings_lst = []
    metrics_r2_lst = []
    metrics_mae_lst = []
    ftr_imp_lst = []
    for i in tqdm(range(len(dataset))[:]):
        #Making up the feature and target datasets
        df_wo_well = dataset.drop([i])
        well_exclude = dataset.iloc[i]['well']
        well_exclude_lst.append(well_exclude)
        fm_exclude = dataset.iloc[i][fmname]
        fm_exclude_lst.append(fm_exclude)
        y_train = np.array(df_wo_well[target])
        x_train = np.array(df_wo_well.drop(['well',fmname, target], axis=1))
        well_train = np.array(df_wo_well['well'])
        y_test = np.array(dataset.iloc[i][target])
        y_test_lst.append(y_test)
        x_test = np.array(dataset.drop(['well', fmname, target], axis=1).iloc[i])
        # Statement of ML-model
        rfr = Pipeline([("scaler",StandardScaler()),("rfr",RandomForestRegressor(**hyperdict, n_jobs=-1, random_state=42))])                                                                                  
        # Fitting the ML-model
        rfr.fit(x_train, y_train)
        y_pred_train = rfr.predict(x_train)
        y_pred_test = rfr.predict([x_test])
        y_pred_test_lst.append(y_pred_test[0])
        # Metrics computation for the ML-model
        r2_train = r2(y_train, y_pred_train).round(5)
        mae_train = mae(y_train, y_pred_train)
        metrics_r2_lst.append(r2_train)
        metrics_mae_lst.append(mae_train.round(5))
        feature_imp = pd.Series(rfr.steps[1][1].feature_importances_, index=df_wo_well.drop(['well', fmname,target], axis=1).columns.tolist()).sort_values(ascending=True)
        ftr_imp_lst.append(feature_imp)
    # Building up of dataframe
    print(rfr.steps[1][1])
    res_rfr_sha = pd.DataFrame( zip(y_test_lst, y_pred_test_lst, well_exclude_lst, fm_exclude_lst, metrics_r2_lst, metrics_mae_lst, ftr_imp_lst), 
                            columns = ['actual','predict','well', 'FORMATION_up','metrics_r2', 'metrics_mae','features_imp'])
    res_rfr_sha['l_range'] = res_rfr_sha.actual*(1-rng) - margin 
    res_rfr_sha['h_range'] = res_rfr_sha.actual*(1+rng) + margin
    res_rfr_sha['qc'] = 'out'
    res_rfr_sha.loc[(res_rfr_sha.predict >= res_rfr_sha.l_range) & (res_rfr_sha.predict <= res_rfr_sha.h_range), 'qc'] = 'in'
    wells_tot = res_rfr_sha.shape[0]
    wells_unpred = res_rfr_sha['qc'].value_counts()['out']
    wells_unpred_vv = (res_rfr_sha['qc'].value_counts()['out']/res_rfr_sha.shape[0]).round(3)
    try:
        wells_pred = res_rfr_sha['qc'].value_counts()['in']
        wells_pred_vv =  (res_rfr_sha['qc'].value_counts()['in']/res_rfr_sha.shape[0]).round(3)
    except:
        wells_pred = 0
        wells_pred_vv = 0
    res_rfr_sha['diff'] = res_rfr_sha.actual - res_rfr_sha.predict
    res_rfr_sha = res_rfr_sha[['well','FORMATION_up','actual','predict', 'diff', 'l_range', 'h_range', 'qc', 'metrics_r2', 'metrics_mae', 'features_imp']]
    types_dict = {'actual': 'float64', 'predict': 'float64', 'diff': 'float64', 'l_range': 'float64', 'h_range': 'float64'}
    res_rfr_sha = res_rfr_sha.astype(types_dict)
    res_rfr_sha = res_rfr_sha.round({'actual': 3, 'predict': 3, 'diff': 3})
    metrics_dict = {    'wells_total':          wells_tot, 
                        'wells_unpred':         wells_unpred,
                        'wells_unpred_v/v':     wells_unpred_vv,
                        'wells_pred':           wells_pred,
                        'wells_pred_v/v':       wells_pred_vv
                    }
    return {    'train_ds': dataset.columns.tolist(),
                'train_ftrs': df_wo_well.drop(['well', fmname,target], axis=1).columns.tolist(),
                'result_df': res_rfr_sha,
                'grid_search' : hyperdict,
                'metrics':metrics_dict,
                'feature_imp' : feature_imp
            }
# Run XGBR model with loop 
def xgbr_loop(dataset, fmname, target, hyperdict, rng, margin):
    """
    'train_ds', 'train_ftrs', 'result_df', 'grid_search', 'metrics'
    """
    y_test_lst = []
    y_pred_test_lst = []
    well_exclude_lst = []
    fm_exclude_lst = []
    gs_settings_lst = []
    metrics_r2_lst = []
    metrics_mae_lst = []
    ftr_imp_lst = []
    for i in tqdm(range(len(dataset))[:]):
        #Making up the feature and target datasets
        df_wo_well = dataset.drop([i])
        well_exclude = dataset.iloc[i]['well']
        well_exclude_lst.append(well_exclude)
        fm_exclude = dataset.iloc[i][fmname]
        fm_exclude_lst.append(fm_exclude)
        y_train = np.array(df_wo_well[target])
        x_train = np.array(df_wo_well.drop(['well',fmname, target], axis=1))
        well_train = np.array(df_wo_well['well'])
        y_test = np.array(dataset.iloc[i][target])
        y_test_lst.append(y_test)
        x_test = np.array(dataset.drop(['well', fmname, target], axis=1).iloc[i])
        xgbr = Pipeline([("scaler",StandardScaler()),("xgbr",XGBRegressor(**hyperdict, n_jobs=-1, random_state=42))])
        # Fitting the ML-model
        xgbr.fit(x_train, y_train)
        y_pred_train = xgbr.predict(x_train)
        y_pred_test = xgbr.predict([x_test])
        y_pred_test_lst.append(y_pred_test[0])
        # Metrics computation for the ML-model
        r2_train = r2(y_train, y_pred_train).round(5)
        mae_train = mae(y_train, y_pred_train)
        metrics_r2_lst.append(r2_train)
        metrics_mae_lst.append(mae_train.round(5))
        feature_imp = pd.Series(xgbr.steps[1][1].feature_importances_, index=df_wo_well.drop(['well', fmname,target], axis=1).columns.tolist()).sort_values(ascending=True)
        ftr_imp_lst.append(feature_imp)

    # Building up of dataframe
    print(xgbr.steps[1][1])
    res_rfr_sha = pd.DataFrame( zip(y_test_lst, y_pred_test_lst, well_exclude_lst, fm_exclude_lst, metrics_r2_lst, metrics_mae_lst, ftr_imp_lst), 
                            columns = ['actual','predict','well', 'FORMATION_up','metrics_r2', 'metrics_mae','features_imp'])
    res_rfr_sha['l_range'] = res_rfr_sha.actual*(1-rng) - margin 
    res_rfr_sha['h_range'] = res_rfr_sha.actual*(1+rng) + margin 
    res_rfr_sha['qc'] = 'out'
    res_rfr_sha.loc[(res_rfr_sha.predict >= res_rfr_sha.l_range) & (res_rfr_sha.predict <= res_rfr_sha.h_range), 'qc'] = 'in'
    wells_tot = res_rfr_sha.shape[0]
    wells_unpred = res_rfr_sha['qc'].value_counts()['out']
    wells_unpred_vv = (res_rfr_sha['qc'].value_counts()['out']/res_rfr_sha.shape[0]).round(3)
    try:
        wells_pred = res_rfr_sha['qc'].value_counts()['in']
        wells_pred_vv =  (res_rfr_sha['qc'].value_counts()['in']/res_rfr_sha.shape[0]).round(3)
    except:
        wells_pred = 0
        wells_pred_vv = 0
    res_rfr_sha['diff'] = res_rfr_sha.actual - res_rfr_sha.predict
    res_rfr_sha = res_rfr_sha[['well','FORMATION_up','actual','predict', 'diff','l_range', 'h_range', 'qc', 'metrics_r2', 'metrics_mae', 'features_imp']]
    types_dict = {'actual': 'float64', 'predict': 'float64', 'diff': 'float64', 'l_range': 'float64', 'h_range': 'float64'}
    res_rfr_sha = res_rfr_sha.astype(types_dict)
    res_rfr_sha = res_rfr_sha.round({'actual': 0, 'predict': 0, 'diff': 0})
    metrics_dict = {    'wells_total':          wells_tot, 
                        'wells_unpred':         wells_unpred,
                        'wells_unpred_v/v':     wells_unpred_vv,
                        'wells_pred':           wells_pred,
                        'wells_pred_v/v':       wells_pred_vv
                    }
    return {    'train_ds': dataset.columns.tolist(),
                'train_ftrs': df_wo_well.drop(['well', fmname,target], axis=1).columns.tolist(),
                'result_df': res_rfr_sha,
                'grid_search' : hyperdict,
                'metrics':metrics_dict,
                'feature_imp' : feature_imp
            }
# Display results of ML-modeling
def xplot_qc(dataset, dataframe, max_val, rng=0.25):
    fig1_ml = px.scatter(dataset[dataframe], x='actual', y='predict', 
                        color='qc', 
                        hover_data=['well'], 
                        width=400, height=400,
                        #  color_discrete_sequence=["red", "green"]
                        )
    up_range = rng+1
    dwn_range = 1- rng
    fig1_ml.update_traces(marker=dict(size=10,opacity=0.75,line=dict(color='rgb(47, 57, 61)', width=1)))
    fig2_ml=px.line(x=[0,max_val], y=[0,max_val])
    fig2_1_ml=px.line(x=[0,max_val], y=[0,max_val*up_range])
    fig2_2_ml=px.line(x=[0,max_val], y=[0,max_val*dwn_range])
    fig2_ml.update_traces(line=dict(color = 'blue'))
    fig2_1_ml.update_traces(line=dict(color = 'blue', dash='dash'))
    fig2_2_ml.update_traces(line=dict(color = 'blue', dash='dash'))
    fig3_ml = go.Figure(data = fig1_ml.data + fig2_ml.data + fig2_1_ml.data + fig2_2_ml.data)
    fig3_ml.update_layout(  title = 'Comparison Actual vs Pred' + 
                                    ' QC_train: ' + str(dataset['metrics']['train_in']) +
                                    ' QC_test: ' + str(dataset['metrics']['test_in']),
                            width=600,height=400, xaxis_title='actual', yaxis_title='predict',
                            margin=dict(l=10,r=10,b=10,t=40))
    return fig3_ml.show()
# Calculate weighted avg properties 
def avg_prop_calculation(dataset_ntd, dataset, formation):
    well_data = []
    well_formation = formation
    for well in tqdm(dataset_ntd.well.unique()):
        # print(well)
        ntd_well_avgprop = dataset_ntd[(dataset_ntd.well ==well)]
        well_avgprop_sel = dataset[(dataset.well==well)]
        well_phit = []
        well_phit10 = []
        well_phit50 = []
        well_phit90 = []
        well_vsh = []
        well_vsh10 = []
        well_vsh50 = []
        well_vsh90 = []
        well_gperm = []
        well_h = []
        for layers in range(len(ntd_well_avgprop.well)):
            ntd_top = ntd_well_avgprop.iloc[layers, 2].round(3)
            ntd_bot = ntd_well_avgprop.iloc[layers, 3].round(3)
            ntd_h = ntd_well_avgprop.iloc[layers, 4].round(3)
            phit_lst = []
            vsh_lst = []
            perm_lst = []
            for depth in range(len(well_avgprop_sel.TST)):
                well_avgprop_tst = well_avgprop_sel['TST'].iloc[depth].round(3)
                if well_avgprop_tst >= ntd_top and well_avgprop_tst <= ntd_bot:
                    phit_lst.append(well_avgprop_sel['PHIT'].iloc[depth])
                    vsh_lst.append(well_avgprop_sel['VSH'].iloc[depth])
                    perm_lst.append(well_avgprop_sel['LPERM'].iloc[depth])
            well_phit.append(mean(phit_lst)*ntd_h)
            well_phit10.append(np.quantile(phit_lst, 0.1)*ntd_h)
            well_phit50.append(np.quantile(phit_lst, 0.5)*ntd_h)
            well_phit90.append(np.quantile(phit_lst, 0.9)*ntd_h)
            well_vsh.append(mean(vsh_lst)*ntd_h)
            well_vsh10.append(np.quantile(vsh_lst, 0.1)*ntd_h)
            well_vsh50.append(np.quantile(vsh_lst, 0.5)*ntd_h)
            well_vsh90.append(np.quantile(vsh_lst, 0.9)*ntd_h)
            well_gperm.append(gmean(perm_lst)*ntd_h)
            well_h.append(ntd_h)
        well_phit_wavg = sum(well_phit)/sum(well_h)
        well_phit10_wavg = sum(well_phit10)/sum(well_h)
        well_phit50_wavg = sum(well_phit50)/sum(well_h)
        well_phit90_wavg = sum(well_phit90)/sum(well_h)
        well_vsh_wavg = sum(well_vsh)/sum(well_h)
        well_vsh10_wavg = sum(well_vsh10)/sum(well_h)
        well_vsh50_wavg = sum(well_vsh50)/sum(well_h)
        well_vsh90_wavg = sum(well_vsh90)/sum(well_h)
        well_perm_wavg = sum(well_gperm)/sum(well_h)
        well_hmax = max(well_h)
        well_h_p50 = np.quantile(well_h, 0.5)
        well_layers_count =len(well_h)
        well_hsum = sum(well_h)
        well_data.append([  well, well_formation, 
                            well_hmax, well_h_p50, well_layers_count, well_hsum,
                            well_phit_wavg, well_phit10_wavg, well_phit50_wavg, well_phit90_wavg,
                            well_vsh_wavg, well_vsh10_wavg, well_vsh50_wavg, well_vsh90_wavg,
                            well_perm_wavg])
    result = pd.DataFrame(well_data, columns=[  'well','FORMATION_up',
                                                'htst_max', 'htst_p50','htst_count', 'htst_sum',            
                                                'phit_wavg', 'phit10_wavg','phit50_wavg','phit90_wavg',
                                                'vsh_wavg', 'vsh10_wavg', 'vsh50_wavg', 'vsh90_wavg',
                                                'perm_wavg'])
    return result
# Euclidian dist calculation with prop
def dist_prop_calc(dataset, dist_formation, dist_cutoff, value):
    """
    dataset have to contain 'X', 'Y', 'TVD_SCS' and 'KHtst', if you assing value as KHtst
    """
    data = dataset[(dataset.FORMATION_up == dist_formation)]
    row_name = data.well.reset_index().drop(['index'], axis=1)
    distance_fm = pd.DataFrame(euclidean_distances(data[['X', 'Y', 'TVD_SCS']]), columns=list(data.well))
    distance_fm_well = distance_fm.join(row_name).set_index('well')
    distance_fm_well.reset_index(inplace=True)
    def well_kh_accum(wells, dataset, kh_formation):
        well_kh_accum = []
        well_x_accum = []
        well_y_accum = []
        for i in wells:
            well_kh_accum.append(dataset[(dataset.well==i)&(dataset.FORMATION_up == kh_formation)][value].reset_index())    
            well_x_accum.append(dataset[(dataset.well==i)&(dataset.FORMATION_up == kh_formation)]['X'].reset_index())
            well_y_accum.append(dataset[(dataset.well==i)&(dataset.FORMATION_up == kh_formation)]['Y'].reset_index())
        well_kh3 = pd.concat(well_kh_accum).T[1:]
        well_kh3.columns = [value + '_1',value + '_2', value + '_3']
        well_x3 = pd.concat(well_x_accum).T[1:]
        well_x3.columns = ['x1','x2','x3']
        well_y3 = pd.concat(well_y_accum).T[1:]
        well_y3.columns = ['y1','y2','y3']
        final = pd.concat([ well_kh3.reset_index().drop('index',axis=1), 
                            well_x3.reset_index().drop('index',axis=1), 
                            well_y3.reset_index().drop('index',axis=1)], axis=1)
        return final
    df_collect = []
    for num, well_name in enumerate(distance_fm_well.well[:]):
        well_dist3 = distance_fm_well[distance_fm_well.well == well_name].T[1:].sort_values(by=num)
        well_dist3_s2 = well_dist3[well_dist3[num] > dist_cutoff][:3].reset_index()
        well_dist3_tuple = tuple(well_dist3_s2['index'])
        well_dist3_res = well_dist3_s2.T[1:].reset_index().drop('index', axis=1)   
        well_name3_res = well_dist3_s2.T[:1].reset_index().drop('index', axis=1)
        well_kh3_res = well_kh_accum(well_dist3_tuple,dataset, dist_formation)
        well_dist3_res.columns =['dist1', 'dist2', 'dist3']
        well_name3_res.columns =['well1', 'well2', 'well3']
        concat_df = pd.concat([well_dist3_res, well_kh3_res, well_name3_res], axis=1)
        result = concat_df.join(pd.DataFrame([well_name], columns=['well']))
        df_collect.append(result)     
    df_well_kh_dist = pd.concat(df_collect).reset_index().drop('index', axis=1)
    df_well_kh_dist['FORMATION_up'] = dist_formation
    return df_well_kh_dist
# Feature importance bar chart for 1-to-all algorithm
def feature_imp_loop(dataset, wellname, fmname, xsize, ysize):
    # dataset = test['result_df']
    data = dataset[(dataset.well==wellname) & (dataset.FORMATION_up == fmname)]
    ftr_imp = data['features_imp'].values[0]
    f, ax = plt.subplots(figsize=(xsize, ysize))
    ftr_imp.plot.barh()
    ax.set_title('RFR feature imp  ' + wellname + ' ' + fmname)
    ax.tick_params(axis='y', labelsize=8, rotation=0)
    return f.show()
# Save datafram to csv
def save_tocsv(dataframe, filename, flag):
    if flag == 1:
        # Saving avg_prop dataframe to .csv
        path = 'C:\\jupyter\\SPP\\inputoutput\\'
        dataframe.to_csv(path + filename)
    else:
        pass
# Feature importance bar chart for split dataframe
def feature_imp_split(dataset, xsize, ysize):
    fig, ax = plt.subplots(figsize=(xsize, ysize))
    ax = dataset.plot.barh()
    ax.set_title("RFR Feature Importances")
    ax.tick_params(axis='y', labelsize=9, rotation=0)
    ax.figure.tight_layout()
    return fig.show()
# Logging results of ml
def write_res_file(finename, comments, target, trainds, metrics, gridsearch):
    with open(finename, 'a') as file:
        # Get the current date and time
        current_datetime = datetime.now()
        # Write the result to the file
        file.write(f'\n{current_datetime} \n {comments} target: {target}')
        file.write(f'\n training_ds_{trainds} \n metrics_{[metrics]} \n grid_search_{gridsearch}')
    file.close()
# Remover categorical values from datasets
def cat_finder(dataset):
    """
    cat_list: categorical columns to drop out
    get_dum_list: categorical columns to run via pd.get_dummies
    """
    cat_list = []
    gm_list = []
    for col in dataset.columns:
        # print(i)
        if dataset[col].dtype == 'string':
            cat_list.append(col)
            if col != 'well':
                gm_list.append(col)
    # return {'cat_list':cat_list,
    #         'get_dum_list': gm_list}
    return cat_list, gm_list
# Display results of ML-modeling ver2
def xplot_qc2(data, max_val, rng, margin, round):
    data = data.round({'actual': round, 'predict': round, 'diff': round})
    ds_train = data[data.dataset == 'train']
    ds_test = data[data.dataset == 'test']
    up_range = rng + 1
    dwn_range = 1 - rng
    colors = {'in': 'green', 'out': 'red'}
    qc_colors_tr = [colors[qc] for qc in ds_train.qc]
    qc_colors_ts = [colors[qc] for qc in ds_test.qc]
    scatter_train = go.Scatter( x=ds_train.actual, y=ds_train.predict,
                                mode='markers',
                                marker=dict(color=qc_colors_tr, size=7, opacity=0.75, line=dict(color='rgb(47, 57, 61)', width=0.5)),
                                customdata = ds_train[['well','actual','predict','diff', 'FORMATION_up']],
                                hovertemplate="".join(
                                ["w:%{customdata[0]},a:%{customdata[1]}, p:%{customdata[2]},d:%{customdata[3]}, f:%{customdata[4]}<extra></extra>"])
                                )
    scatter_test = go.Scatter(  x=ds_test.actual, y=ds_test.predict, 
                                mode='markers',
                                marker=dict(color=qc_colors_ts, size=7, opacity=0.75, line=dict(color='rgb(47, 57, 61)', width=0.5)),
                                customdata = ds_test[['well','actual','predict','diff', 'FORMATION_up']],
                                hovertemplate="".join(
                                ["w:%{customdata[0]},a:%{customdata[1]}, p:%{customdata[2]},d:%{customdata[3]}, f:%{customdata[4]}<extra></extra>"])
                                )
    line_trace_up = go.Scatter(x=[0, max_val], y=[0 + margin, max_val*up_range + margin], mode='lines+markers', line=dict(color='blue'))
    line_trace_dw = go.Scatter(x=[0, max_val], y=[0 - margin, max_val*dwn_range - margin], mode='lines+markers', marker=dict(color='blue'))
    fig = make_subplots(rows=1, cols=2, subplot_titles=('train ds', 'test ds'))
    fig.add_trace(scatter_train,  row=1, col=1)
    fig.add_trace(line_trace_up,  row=1, col=1)
    fig.add_trace(line_trace_dw,  row=1, col=1)
    fig.update_xaxes(title_text='actual', row=1, col=1)
    fig.update_yaxes(title_text='predict', row=1, col=1)
    fig.add_trace(scatter_test,  row=1, col=2)
    fig.add_trace(line_trace_up,  row=1, col=2)
    fig.add_trace(line_trace_dw,  row=1, col=2)
    fig.update_xaxes(title_text='actual', row=1, col=2)
    fig.update_yaxes(title_text='predict', row=1, col=2)
    fig.update_layout(  title_text= ('rfr_train_test_split'), width=900, height=450, 
                        margin=dict(l=10,r=10,b=10,t=50), showlegend=False)
    return fig.show()
# Display results of ML-modeling ver2 via loop    
def xplot_qc2_loop(data, max_val, rng, margin=0.005):
    data = data.round({'actual': 3, 'predict': 3, 'diff ': 3})
    up_range = rng + 1
    dwn_range = 1 - rng
    colors = {'in': 'green', 'out': 'red'}
    qc_colors = [colors[qc] for qc in data.qc]
    scatter = go.Scatter( x=data.actual, y=data.predict,
                            mode='markers',
                            marker=dict(color=qc_colors, size=7, opacity=0.75, line=dict(color='rgb(47, 57, 61)', width=0.5)),
                            customdata = data[['well','actual','predict', 'diff', 'FORMATION_up']],
                            hovertemplate="".join(
                            ["w:%{customdata[0]},a:%{customdata[1]}, p:%{customdata[2]}, d:%{customdata[3]}, f:%{customdata[4]}<extra></extra>"])
                            )
    fig = go.Figure()
    line_up = go.Scatter(x=[0, max_val], y=[0 + margin, max_val*up_range + margin], mode='lines+markers', line=dict(color='blue'))
    line_dw = go.Scatter(x=[0, max_val], y=[0 - margin, max_val*dwn_range - margin], mode='lines+markers', marker=dict(color='blue'))
    fig.add_trace(scatter)
    fig.add_trace(line_up)
    fig.add_trace(line_dw)
    fig.update_xaxes(title_text='actual')
    fig.update_yaxes(title_text='predict')
    fig.update_layout(  title_text= ('rfr_loop'), width=450, height=450, 
                        margin=dict(l=10,r=10,b=10,t=50), showlegend=False)
    return fig.show()
# Display results of ML-modeling on map
def map_qc(metadata, data, fmname, scale):
    data['diff'] = abs(data['diff'])
    data = data[data.FORMATION_up == fmname]
    data_in = data[data.qc=='in']
    data_out = data[data.qc=='out']
    field_avg_coord = metadata.groupby('field')[['X_wellhead','Y_wellhead']].mean().reset_index()
    platform  = go.Scatter(         x=field_avg_coord.X_wellhead, y=field_avg_coord.Y_wellhead, customdata = field_avg_coord[['field']],
                                    text=field_avg_coord['field'], textposition="middle right",
                                    marker=dict(color='rgb(0, 0,0)', size=12),
                                    mode='markers+text', 
                                    marker_symbol='square', hovertemplate="".join(["%{customdata[0]}<extra></extra>"])
                                    )
    scatter_data_in = go.Scatter(   x=data_in.X, y=data_in.Y,
                                    mode='markers',
                                    marker=dict(symbol='circle', color='green', size=data_in['actual']*scale,
                                    opacity=0.75, line=dict(color='rgb(47, 57, 61)', width=0.5)
                                    ),
                                    customdata = data_in[['well', 'diff']],
                                    hovertemplate="".join(["well:%{customdata[0]}, diff:%{customdata[1]}<extra></extra>"])
                                    )
    scatter_data_out = go.Scatter(  x=data_out.X, y=data_out.Y, 
                                    mode='markers',
                                    marker=dict(symbol='diamond', color='red', size=data_out['diff']*scale,
                                    opacity=0.75, line=dict(color='rgb(47, 57, 61)', width=0.5)),
                                    customdata = data_out[['well', 'diff']],
                                    hovertemplate="".join(["well:%{customdata[0]}, diff:%{customdata[1]}<extra></extra>"])
                                    )
    fig = go.Figure()
    fig.add_trace(platform)
    fig.add_trace(scatter_data_in)
    fig.add_trace(scatter_data_out)
    fig.update_layout(title_text= ('rfr_train_test_split'),autosize=True, width=1000, height=600, margin=dict(l=10,r=10,b=10,t=50), showlegend=False)
    return fig.show()
# Pairplot new version
def pairplot_special(dataset, xsize, ysize, flag=1):
    if flag == 1:
        def corrfunc(x, y, **kws):
            r, _ = stats.pearsonr(x, y)
            ax = plt.gca()
            ax.annotate("r = {:.2f}".format(r),
                        xy=(.1, .9), xycoords=ax.transAxes)
        sns.set_context(rc={'axes.labelsize':10, 'lines.linewidth': 0.75})
        g = sns.PairGrid(dataset)
        g.fig.set_size_inches(xsize,ysize)
        g.set(xticklabels=[], yticklabels=[]) 
        g.map_upper(plt.scatter, s=10, alpha=0.5)
        g.map_diag(sns.distplot, kde=False)
        g.map_lower(sns.kdeplot, cmap="Blues_d")
        g.map_lower(corrfunc)
    else:
        pass
# Columns reorder for better display of variables
def columns_reorder(dataset, selected_column):
    new_order = [col for col in dataset.columns if col != selected_column] + [selected_column]
    dataset = dataset[new_order]
    return dataset
# Just simple x-plot for 1 dataframe
def log_map_plot(dataframe, x_var, y_var, min_val, max_val):
    fig = go.Figure()
    scatter = go.Scatter(   x=dataframe[x_var], y=dataframe[y_var], 
                            mode='markers',
                            marker=dict(color='orange', size=10, opacity=0.75, line=dict(color='rgb(47, 57, 61)', width=0.5)),
                            customdata = dataframe[['well',x_var,y_var]],
                            hovertemplate="".join(
                            ["w:%{customdata[0]},x:%{customdata[1]}, y:%{customdata[2]}<extra></extra>"])
                            )
    line = go.Scatter(x=[min_val, max_val], y=[min_val, max_val], mode='lines', line=dict(color='blue'))
    fig.add_trace(scatter)
    fig.add_trace(line)
    fig.update_layout(  title_text= ('scatter plot'), width=600, height=600, 
                        margin=dict(l=10,r=10,b=10,t=50), showlegend=False)
    return fig.show()
# Joining main and additional dataframes for predictions
def join_add_df_prediction(base_dataframe, add_dataframe, target_var):
    """
    Both dataframes have contain 'well' & 'FORMATION_up' for joining
    """
    join_dataframe = base_dataframe.set_index(['well','FORMATION_up']).join(add_dataframe.set_index(['well','FORMATION_up'])).reset_index()
    col_names, gm_list = cat_finder(join_dataframe)
    df_corr = join_dataframe.drop(col_names, axis=1)
    df_corr = columns_reorder(df_corr, target_var)
    mem_cell = pd.get_dummies(join_dataframe[gm_list], columns=gm_list, drop_first=True)
    mem_cell.rename(columns={'FORMATION_up_Balakhany X':'FORMATION_up_gm'},inplace=True)
    join_dataframe_gm = pd.concat([join_dataframe, mem_cell], axis=1)
    return df_corr, join_dataframe_gm
# Preparation dataframes for pairplot and for predictions
def join_df_prediction(base_dataframe, target_var):
    col_names, gm_list = cat_finder(base_dataframe)
    df_corr = base_dataframe.drop(col_names, axis=1)
    df_corr = columns_reorder(df_corr, target_var)
    mem_cell = pd.get_dummies(base_dataframe[gm_list], columns=gm_list, drop_first=True)
    mem_cell.rename(columns={'FORMATION_up_Balakhany X':'FORMATION_up_gm'},inplace=True)
    dataframe = pd.concat([base_dataframe, mem_cell], axis=1)
    return df_corr, dataframe
# Function to calculate grid_search via train_split
def run_rfr_train_test_split(dataset, gs_set, scorer, target, rng, margin, logtxt_name, comment, xplot_flag, ftr_imp_flag):
    model_res = rfr_train_test_split(dataset, gs_set, scorer, target, rng, margin)
    write_res_file(logtxt_name, comment, target, 
                    model_res['train_ds'], model_res['metrics'], model_res['grid_search'])
    print('train_ds: ', model_res['train_ds'])
    print('metrics: ', model_res['metrics'])
    print('grid_search: ', model_res['grid_search'])
    model_res_hyper_par = model_res['grid_search'][0]
    if xplot_flag == 1:
        xplot_qc2(dataset['result_df'], 0.3, 0.05, margin)
    else:
        pass
    if ftr_imp_flag == 1:
        feature_imp_split(dataset['feature_imp'], 6, 4)
    else:
        pass
    return model_res_hyper_par
# Function to calculate target via 1-to-all
def run_rfr_1_to_all(dataset, hyperdict, target, rng, margin, logtxt_name, comment, xplot_flag, max_val, ftr_imp_flag):
    loop_res = rfr_loop(dataset, 'FORMATION_up', target, hyperdict, rng, margin)
    write_res_file(logtxt_name, comment, target, loop_res['train_ds'], loop_res['metrics'], loop_res['grid_search'])
    loop_res_pred = loop_res['result_df']
    print('train_ftrs: ',loop_res['train_ftrs'])
    print('metrics: ',loop_res['metrics'])
    if xplot_flag == 1:
        xplot_qc2_loop(loop_res['result_df'], max_val, rng, margin)
    else:
        pass
    if ftr_imp_flag == 1:
        feature_imp_split(loop_res['feature_imp'], 6, 4)
    else:
        pass
    return loop_res_pred

## Narmin

In [ ]:
# # # wells_narmin = sorted(['C15Z', 'C11Z', 'C33Z', 'J16', 'C31', 'C28', 'E33', 'E35A', 'F03'])
# df_prq_narmin = df_prq
# df_prq_narmin_bal8 = df_prq_narmin[df_prq_narmin.FORMATION.str.contains('Balakhany VIII')]
# df_prq_narmin_bal8['FORMATION_up'] = 'Balakhany VIII'
# df_prq_narmin_bal10 = df_prq_narmin[df_prq_narmin.FORMATION.str.contains('Balakhany X')]
# df_prq_narmin_bal10['FORMATION_up'] = 'Balakhany X'
# df_prq_narmin_perA = df_prq_narmin[df_prq_narmin.FORMATION.str.contains('Pereriv A')]
# df_prq_narmin_perA['FORMATION_up'] = 'Pereriv A'
# df_prq_narmin_perB = df_prq_narmin[df_prq_narmin.FORMATION.str.contains('Pereriv B')]
# df_prq_narmin_perB['FORMATION_up'] = 'Pereriv B'
# df_prq_narmin_perC = df_prq_narmin[df_prq_narmin.FORMATION.str.contains('Pereriv C')]
# df_prq_narmin_perC['FORMATION_up'] = 'Pereriv C'
# df_prq_narmin_perD = df_prq_narmin[df_prq_narmin.FORMATION.str.contains('Pereriv D')]
# df_prq_narmin_perD['FORMATION_up'] = 'Pereriv D'
# def tst_calc_narmin(data, fmname):
#     short = data[['well','FORMATION','TST']]
#     top = short.groupby('well')['TST'].apply(lambda x: x.iloc[0]).reset_index()
#     top.rename(columns={'TST':'TST_top'}, inplace=True)
#     bot = short.groupby('well')['TST'].apply(lambda x: x.iloc[-1]).reset_index()
#     bot.rename(columns={'TST':'TST_bot'}, inplace=True)
#     int_tst = top.set_index('well').join(bot.set_index('well')).reset_index()
#     int_tst['TST_thickness'] = int_tst.TST_bot - int_tst.TST_top
#     int_tst['FORMATION_up'] = fmname
#     return int_tst
# bal8 = tst_calc_narmin(df_prq_narmin_bal8,'Balakhany VIII')
# bal10 = tst_calc_narmin(df_prq_narmin_bal10,'Balakhany X')
# perA = tst_calc_narmin(df_prq_narmin_perA,'Pereriv A')
# perB = tst_calc_narmin(df_prq_narmin_perB,'Pereriv B')
# perC = tst_calc_narmin(df_prq_narmin_perC,'Pereriv C')
# perD = tst_calc_narmin(df_prq_narmin_perD,'Pereriv D')
# tst_fin = pd.concat([bal8, bal10, perA, perB, perC, perD]).sort_values(by='well')
# def ntd_calculation_brief(dataset,well,desired_fm, net_var='NET'):
#     data = dataset[(dataset.well==well) & (dataset.FORMATION_up==desired_fm)]
#     data.iloc[0, 3] = 0
#     data.iloc[-1, 3] = 0
#     tst_top = [data.iloc[i]['TST'] for i in range(len(data)-1)
#                 if (data.iloc[i][net_var] == 1 and data.iloc[i-1][net_var]==0)]
#     tst_bot = [data.iloc[i]['TST'] for i in range(len(data)-1)
#                 if (data.iloc[i][net_var] == 1 and data.iloc[i+1][net_var]==0)]
#     tops = zip(tst_top, tst_bot)
#     df_htst = pd.DataFrame(tops, columns=['tst_top', 'tst_bot'])
#     df_htst['FORMATION_up'] = desired_fm
#     df_htst['well'] = well
#     df_htst['h_tst'] = df_htst.tst_bot - df_htst.tst_top
#     df_htst = df_htst[['well','FORMATION_up','tst_top','tst_bot','h_tst']]
#     return df_htst
# def net_sum_narmin(data, fmname):
#     df_list = []
#     print('Computation NTD is running: ' + fmname)
#     for well in tqdm(data.well.unique()):
#         df = ntd_calculation_brief(data, well, fmname, 'NET')
#         df_list.append(df)
#     ntd_net = pd.concat(df_list)
#     result = ntd_net.groupby('well')['h_tst'].sum().reset_index()
#     result.rename(columns={'h_tst':'sum_net'}, inplace=True)
#     result['FORMATION_up'] = fmname
#     return result
# net_bal8 = net_sum_narmin(df_prq_narmin_bal8, 'Balakhany VIII')
# net_bal10 = net_sum_narmin(df_prq_narmin_bal10, 'Balakhany X')
# net_perA = net_sum_narmin(df_prq_narmin_perA, 'Pereriv A')
# net_perB = net_sum_narmin(df_prq_narmin_perB, 'Pereriv B')
# net_perC = net_sum_narmin(df_prq_narmin_perC, 'Pereriv C')
# net_perD = net_sum_narmin(df_prq_narmin_perD, 'Pereriv D')
# net_fin = pd.concat([net_bal8, net_bal10, net_perA, net_perB, net_perC, net_perD]).sort_values(by='well')
# tst_net_fin = tst_fin.set_index(['well','FORMATION_up']).join(net_fin.set_index(['well','FORMATION_up'])).reset_index().sort_values(by=['well','FORMATION_up'])
# tst_net_fin.fillna(0, inplace=True)
# tst_net_fin.rename(columns = {'TST_thickness':'tst_gross', 'sum_net':'tst_net'}, inplace=True)
# tst_net_fin.to_csv('narmin_tst_net.csv', index=False)

## Data preparation

### Well quality calculation & printing of diagrams 

In [ ]:
#Counting of bad quality logs
bal8_list = [   'Balakhany VIII sand', 'Balakhany VIII 20',   'Balakhany VIII 10',   'Balakhany VIII 25',
                'Balakhany VIII 15',    'Balakhany VIII 5']
well_tot8 = []
well_zero8 = []
well_clean8 = []
for j in df_bal[(df_bal.FORMATION.isin(bal8_list) & (df_bal.PHIT>0))].well.unique():
    phit_zero = (len(df_bal[(df_bal.well==j) & (df_bal.FORMATION_up == 'Balakhany VIII')]))
    phit_nonzero = (len(df_bal[(df_bal.well==j) & (df_bal.FORMATION_up == 'Balakhany VIII') & (df_bal.PHIT > 0)]))
    well_tot8.append(j)
    if round((phit_nonzero/phit_zero),2)<=0.90:
        well_zero8.append(j)
    else:
        well_clean8.append(j)
        # well_display_ntd(df_bal, j, 'Balakhany VIII', 'NET', round((phit_nonzero/phit_zero),2), 1) #printing well plots with high quality logs

bal10_list = ['Balakhany X', 'Balakhany X sand', 'Balakhany X 40', 'Balakhany X 20', 'Balakhany X 50']
well_tot10 = []
well_zero10 = []
well_clean10 = []
for j in df_bal[(df_bal.FORMATION.isin(bal10_list) & (df_bal.PHIT>0))].well.unique():
    phit_zero = (len(df_bal[(df_bal.well==j) & (df_bal.FORMATION_up == 'Balakhany X')]))
    phit_nonzero = (len(df_bal[(df_bal.well==j) & (df_bal.FORMATION_up == 'Balakhany X') & (df_bal.PHIT > 0)]))
    well_tot10.append(j)
    if round((phit_nonzero/phit_zero),2)<=0.90:
        # well_display_ntd(df_bal, j, 'Balakhany X', 'NET', round((phit_nonzero/phit_zero),2), 1)
        well_zero10.append(j)
    else:
        well_clean10.append(j)
print('well_tot8', len(well_tot8))
print('well_zero8', len(well_zero8))
print('well_clean8', len(well_clean8))
print('----------------------')
print('well_tot10', len(well_tot10))
print('well_zero10', len(well_zero10))
print('well_clean10', len(well_clean10))

# broken wells Bal8
# A08, A19, H01Z, J05 
# broken wells Bal10
# C31, D25
# high tst_interv Bal8
# E30Z
# small tst_interv Bal8
# G01Z, E05, E01, E01Y, E11Z, E07, H01Y, H01Z, A14
# Add wells after review Bal8 
# D04Z, 
# Remove wells from clean_list by any reasons
remove_tst8 = ['A08','A19','J05','E30Z','G01Z', 'E05', 'E01', 'E01Y', 'E11Z', 'E07', 'H01Y', 'H01Z', 'A14']
well_clean8_v2 = [i for i in well_clean8 if i not in remove_tst8]
remove_tst10 = ['C31','D25', 'E21A']
well_clean10_v2 = [i for i in well_clean10 if i not in remove_tst10]
print('----------------------')
print('well_clean8_v2: ', len(well_clean8_v2))
print('well_clean10_v2: ', len(well_clean10_v2))

In [2407]:
# well_clean8_v2_df = pd.DataFrame(well_clean8_v2, columns=['well'])
# well_clean8_v2_df['FORMATION_up'] = 'Balakhany VIII'

# well_clean10_v2_df = pd.DataFrame(well_clean10_v2, columns=['well'])
# well_clean10_v2_df['FORMATION_up'] = 'Balakhany X'
# well_clean_v2_df = pd.concat([well_clean8_v2_df, well_clean10_v2_df])
# well_clean_v2_df.to_csv('well_clean_v2_df.csv', index=False)

### NetThicknessDistribution

In [ ]:
# Limitation dataframe to cleaned wells for Bal8 & Bal10
df_net_bal8 = df_bal[['well', 'MD', 'TST', 'NET', 'FORMATION_up', 'LPERM', 'PHIT', 'VSH']]
df_net_bal8 = df_net_bal8[df_net_bal8.well.isin(well_clean8_v2) & (df_net_bal8.FORMATION_up=='Balakhany VIII')]
df_net_bal10 = df_bal[['well', 'MD', 'TST', 'NET', 'FORMATION_up', 'LPERM', 'PHIT', 'VSH']]
df_net_bal10 = df_net_bal10[df_net_bal10.well.isin(well_clean10_v2) & (df_net_bal10.FORMATION_up=='Balakhany X')]
# Calculation dataframe with h_tst from MD to TST for Bal8
df_list8 = []
print('Calculation dataframe with h_tst from MD to TST for Bal8')
for well in tqdm(df_net_bal8.well.unique()):
    df = ntd_calculation_brief(df_net_bal8, well, 'Balakhany VIII', 'NET')
    df_list8.append(df)
ntd_net_8 = pd.concat(df_list8)
df_list10 = []
print('Calculation dataframe with h_tst from MD to TST for Bal10')
for well in tqdm(df_net_bal10.well.unique()):
    df = ntd_calculation_brief(df_net_bal10, well, 'Balakhany X', 'NET')
    df_list10.append(df)
ntd_net_10 = pd.concat(df_list10)
ntd_net_final = pd.concat([ntd_net_8, ntd_net_10])
# Cleaning NET variable and making up NET_clp with clipped data, join NET_clp to main dataframe
print('Cleaning NET variable and making up NET_clp with clipped data')
net_clp =  ntd_htst_cleaning(ntd_net_final, 1)
df_bal_net = df_bal.set_index(['well','MD']).join(net_clp.drop(
    ['FORMATION_up','NET','TST', 'FORMATION', 'GR_N'], axis=1).set_index(['well','MD'])).reset_index()
df_bal_net = df_bal_net[df_bal_net.NET_clp.notna()]
# Cleaning NET_clp from 1-point zero
print('Cleaning NET_clp from 1-point zero')
for i in tqdm(range(len(df_bal_net.NET_clp))):
    if (df_bal_net.NET_clp.iloc[i] == 0 and  
        df_bal_net.NET_clp.iloc[i-1] == 1 and 
        df_bal_net.NET_clp.iloc[i+1] == 1):
        df_bal_net.NET_clp.iloc[i] = 1
# NET-zero layers removing
df_zero_bal = df_bal_net[['well', 'MD', 'TST', 'NET_clp', 'FORMATION_up']]
df_zero_bal8 = df_zero_bal[df_zero_bal.well.isin(well_clean8_v2) & (df_zero_bal.FORMATION_up=='Balakhany VIII')]
df_zero_list8 = []
print('NET-zero layers removing Bal8')
for well in tqdm(df_zero_bal8.well.unique()):
    df = ntd_calculation_zero(df_zero_bal8, well, 'Balakhany VIII', 'NET_clp')
    df_zero_list8.append(df)
ntd_zero_8 = pd.concat(df_zero_list8)
df_zero_bal10 = df_zero_bal[df_zero_bal.well.isin(well_clean10_v2) & (df_zero_bal.FORMATION_up=='Balakhany X')]
df_zero_list10 = []
print('NET-zero layers removing Bal10')
for well in tqdm(df_zero_bal10.well.unique()):
    df = ntd_calculation_zero(df_zero_bal10, well, 'Balakhany X', 'NET_clp')
    df_zero_list10.append(df)
ntd_zero_10 = pd.concat(df_zero_list10)
ntd_zero = pd.concat([ntd_zero_8, ntd_zero_10])
print('Run ntd_htst_zero_cleaning')
net_clp2 = ntd_htst_zero_cleaning(ntd_zero, df_bal_net, 1, 'NET_clp', 'NET_clp2')
#Joining NET_clp2 to main dataframe df_bal_net
df_bal_net2 = df_bal_net.set_index(['well','MD']).join(net_clp2.drop(
    ['FORMATION_up','GR_N', 'NET','NET_clp', 'FORMATION','TST'], axis=1).set_index(['well','MD'])).reset_index()
df_bal_net2 = df_bal_net2[df_bal_net2.NET_clp2.notna()]
# Displaying of ramdom well for example
well_display_net(df_bal_net2, 'J28', 'Balakhany VIII', 'NET_clp', 1, 'NET_clp2')

## Full set of FU for Bal VIII / X FORMATION

In [8]:
def prep_fullset_bal_fu_calc_prop(dataset):
    """
    Calculation XY coordinates of Bal VIII/X 
    Calculation KHtst and tst_interv for Bal8 and Bal10 based on NET_clp2
    Calculation average properties for Bal8 and Bal10 based on NET_clp2
    Joing KHtst/tst_interv and avg prop into one dataframe
    """
    # Calculation XY coordinates of Bal VIII/X
    df_bal_net2_kh_xy8 = dataset[   (dataset.well.isin(well_clean8_v2)) & 
                                    (dataset.FORMATION_up=='Balakhany VIII')][['well','FORMATION_up','X','Y','TVD_SCS','field']].groupby(
                                    ['well','FORMATION_up']).apply(lambda x: x.iloc[0]).drop(['well','FORMATION_up'], axis=1).reset_index() 
    df_bal_net2_kh_xy10 = dataset[  (dataset.well.isin(well_clean10_v2)) & 
                                    (dataset.FORMATION_up=='Balakhany X')][['well','FORMATION_up','X','Y','TVD_SCS','field']].groupby(
                                    ['well','FORMATION_up']).apply(lambda x: x.iloc[0]).drop(['well','FORMATION_up'], axis=1).reset_index()
    df_bal_net2_kh_xy = pd.concat([df_bal_net2_kh_xy8, df_bal_net2_kh_xy10]).sort_values(by='well')  
    # Removing Bal VIII and Bal X from FORMATION to calc avg prop
    df_net2_bal8 = dataset[[ 'well', 'MD', 'TST', 'TVD_SCS','NET_clp2', 'FORMATION_up', 'FORMATION', 
                                        'LPERM', 'PHIT', 'VSH', 'KHtst','PHITHtst', 'VSHHtst', 'X','Y','field']]
    df_net2_bal8 = df_net2_bal8[    (df_net2_bal8.well.isin(well_clean8_v2)) & 
                                    (df_net2_bal8.FORMATION_up=='Balakhany VIII')]
    df_net2_bal10 = dataset[['well', 'MD', 'TST', 'TVD_SCS','NET_clp2', 'FORMATION_up', 'FORMATION', 
                                        'LPERM', 'PHIT', 'VSH', 'KHtst','PHITHtst','VSHHtst', 'X','Y','field']]
    df_net2_bal10 = df_net2_bal10[  (df_net2_bal10.well.isin(well_clean10_v2)) & 
                                    (df_net2_bal10.FORMATION_up=='Balakhany X')]
    # Calculation NTD for Bal8 and Bal10 based on NET_clp2
    print('Calculation NTD for Bal8 and Bal10 based on NET_clp2')
    df_recalc_list8 = []
    for well in tqdm(df_net2_bal8.well.unique()):
        df = ntd_calculation_brief(df_net2_bal8, well, 'Balakhany VIII', 'NET_clp2')
        df_recalc_list8.append(df)
    ntd_net2_8 = pd.concat(df_recalc_list8)
    ntd_net2_8.drop_duplicates(inplace=True)
    df_recalc_list10 = []
    for well in tqdm(df_net2_bal10.well.unique()):
        df = ntd_calculation_brief(df_net2_bal10, well, 'Balakhany X', 'NET_clp2')
        df_recalc_list10.append(df)
    ntd_net2_10 = pd.concat(df_recalc_list10)
    ntd_net2_10.drop_duplicates(inplace=True)
    ntd_net2_final = pd.concat([ntd_net2_8, ntd_net2_10])
    # Calculation KHtst and tst_interv for Bal8 and Bal10 based on NET_clp2 and removed FU Bal VIII/X
    df_kh8 = df_net2_bal8.groupby('well')[[ 'TVD_SCS','KHtst','FORMATION_up',
                                            'PHITHtst','VSHHtst','X','Y','field']].apply(lambda x: x.iloc[1]).reset_index()                              
    df_kh10 = df_net2_bal10.groupby('well')[[   'TVD_SCS','KHtst','FORMATION_up',
                                                'PHITHtst','VSHHtst','X','Y','field']].apply(lambda x: x.iloc[1]).reset_index()
    df_tst8 = df_net2_bal8.groupby('well')[['TST', 'TVD_SCS']].apply(lambda x: x.iloc[-1] - x.iloc[1]).reset_index()
    df_tst8['FORMATION_up'] = 'Balakhany VIII'
    df_tst10 = df_net2_bal10.groupby('well')[['TST','TVD_SCS']].apply(lambda x: x.iloc[-1] - x.iloc[1]).reset_index()
    df_tst10['FORMATION_up'] = 'Balakhany X'
    df_kh_fin = pd.concat([df_kh8, df_kh10])
    df_tst_fin = pd.concat([df_tst8, df_tst10])
    df_tst_fin.rename(columns={'TST':'tst_interv', 'TVD_SCS':'tvd_interv'}, inplace=True)
    df_tst_kh = df_tst_fin.set_index(['well','FORMATION_up']).join(df_kh_fin.set_index(['well','FORMATION_up'])).reset_index()
    # Calculation average properties for Bal8 and Bal10 based on NET_clp2 and removed FU Bal VIII/X
    print('Calculation average properties for Bal8 and Bal10 based on NET_clp2')
    avg_prop_bal8 = avg_prop_calculation(ntd_net2_8, df_net2_bal8, 'Balakhany VIII')
    avg_prop_bal10 = avg_prop_calculation(ntd_net2_10, df_net2_bal10, 'Balakhany X')   
    avg_prop_final = pd.concat([avg_prop_bal8, avg_prop_bal10])
    avg_prop_final.sort_values(by='well', inplace=True)  
    # Joing KHtst/tst_interv and avg prop into one dataframe
    avg_prop_tst_kh = avg_prop_final.set_index(['well','FORMATION_up']).join(df_tst_kh.set_index(['well','FORMATION_up'])).reset_index().sort_values(by='well')
    avg_prop_tst_kh = avg_prop_tst_kh.round({   'htst_max': 0, 'htst_sum': 0, 'tst_interv': 0, 'tvd_interv': 0, 'TVD_SCS': 0, 
                                                'KHtst': 0, 'PHITHtst': 0, 'VSHHtst': 0, 'perm_wavg':1, 'phit_wavg':2})
    return avg_prop_tst_kh
def prep_dist_calc_brief(dataset, offset):
    avg_prop_targKH = dataset[['well', 'FORMATION_up', 'KHtst']]
    df_dist_kh_bal8 = dist_prop_calc(dataset, 'Balakhany VIII', offset, 'KHtst')
    df_dist_kh_bal10 = dist_prop_calc(dataset, 'Balakhany X', offset, 'KHtst')
    df_dist_kh_bal_concat = pd.concat([df_dist_kh_bal8, df_dist_kh_bal10])
    df_dist_kh_bal_fin = df_dist_kh_bal_concat.set_index(['well', 'FORMATION_up']).join(avg_prop_targKH.set_index(['well', 'FORMATION_up'])).reset_index()
    df_dist_kh_bal_fin['dist1_op'] = 1/df_dist_kh_bal_fin.dist1
    df_dist_kh_bal_fin['dist2_op'] = 1/df_dist_kh_bal_fin.dist2
    df_dist_kh_bal_fin['dist3_op'] = 1/df_dist_kh_bal_fin.dist3
    return df_dist_kh_bal_fin
def prep_remove_balfu_calc_prop(dataset):
    """
    Calculation XY coordinates of Bal VIII sand/X sand
    Removing Bal VIII and Bal X from FORMATION
    Calculation KHtst and tst_interv for Bal8 and Bal10 based on NET_clp2 and removed FU Bal VIII/X
    Calculation average properties for Bal8 and Bal10 based on NET_clp2.
    Joing KHtst/tst_interv and avg prop into one dataframe
    """
    # Calculation XY coordinates of Bal VIII/X
    df_bal_net2_kh_cut_xy8 = dataset[   (dataset.well.isin(well_clean8_v2)) & 
                                        (dataset.FORMATION_up=='Balakhany VIII') &
                                        (dataset.FORMATION != 'Balakhany VIII')][['well','FORMATION_up','X','Y','field']].groupby(
                                        ['well','FORMATION_up']).apply(lambda x: x.iloc[0]).drop(['well','FORMATION_up'], axis=1).reset_index() 
    df_bal_net2_kh_cut_xy10 = dataset[  (dataset.well.isin(well_clean10_v2)) & 
                                        (dataset.FORMATION_up=='Balakhany X') &
                                        (dataset.FORMATION != 'Balakhany X')][['well','FORMATION_up','X','Y','field']].groupby(
                                        ['well','FORMATION_up']).apply(lambda x: x.iloc[0]).drop(['well','FORMATION_up'], axis=1).reset_index()
    df_bal_net2_kh_cut_xy = pd.concat([df_bal_net2_kh_cut_xy8, df_bal_net2_kh_cut_xy10]).sort_values(by='well')  
    # Removing Bal VIII and Bal X from FORMATION to calc avg prop
    df_net2_bal8_cut = dataset[[ 'well', 'MD', 'TST', 'TVD_SCS','NET_clp2', 'FORMATION_up', 'FORMATION', 
                                        'LPERM', 'PHIT', 'VSH', 'KHtst','PHITHtst', 'VSHHtst', 'X','Y','field']]
    df_net2_bal8_cut = df_net2_bal8_cut[    df_net2_bal8_cut.well.isin(well_clean8_v2) & 
                                            (df_net2_bal8_cut.FORMATION_up=='Balakhany VIII') &
                                            (df_net2_bal8_cut.FORMATION != 'Balakhany VIII')]
    df_net2_bal10_cut = dataset[['well', 'MD', 'TST', 'TVD_SCS','NET_clp2', 'FORMATION_up', 'FORMATION', 
                                        'LPERM', 'PHIT', 'VSH', 'KHtst','PHITHtst','VSHHtst', 'X','Y','field']]
    df_net2_bal10_cut = df_net2_bal10_cut[      df_net2_bal10_cut.well.isin(well_clean10_v2) & 
                                                (df_net2_bal10_cut.FORMATION_up=='Balakhany X') &
                                                (df_net2_bal10_cut.FORMATION != 'Balakhany X')]
    # Calculation NTD for Bal8 and Bal10 based on NET_clp2 and removed FU Bal VIII/X
    print('Calculation NTD for Bal8 and Bal10 based on NET_clp2 and removed FU Bal VIII/X')
    df_recalc_list8_cut = []
    for well in tqdm(df_net2_bal8_cut.well.unique()):
        df = ntd_calculation_brief(df_net2_bal8_cut, well, 'Balakhany VIII', 'NET_clp2')
        df_recalc_list8_cut.append(df)
    ntd_net2_8_cut = pd.concat(df_recalc_list8_cut)
    ntd_net2_8_cut.drop_duplicates(inplace=True)
    df_recalc_list10_cut = []
    for well in tqdm(df_net2_bal10_cut.well.unique()):
        df = ntd_calculation_brief(df_net2_bal10_cut, well, 'Balakhany X', 'NET_clp2')
        df_recalc_list10_cut.append(df)
    ntd_net2_10_cut = pd.concat(df_recalc_list10_cut)
    ntd_net2_10_cut.drop_duplicates(inplace=True)
    ntd_net2_cut_final = pd.concat([ntd_net2_8_cut, ntd_net2_10_cut])
    # Calculation KHtst and tst_interv for Bal8 and Bal10 based on NET_clp2 and removed FU Bal VIII/X
    df_kh8_cut = df_net2_bal8_cut.groupby('well')[[ 'TVD_SCS','KHtst','FORMATION_up',
                                                    'PHITHtst','VSHHtst','X','Y','field']].apply(lambda x: x.iloc[0]).reset_index()                              
    df_kh10_cut = df_net2_bal10_cut.groupby('well')[[   'TVD_SCS','KHtst','FORMATION_up',
                                                        'PHITHtst','VSHHtst','X','Y','field']].apply(lambda x: x.iloc[0]).reset_index()
    df_tst8_cut = df_net2_bal8_cut.groupby('well')[['TST', 'TVD_SCS']].apply(lambda x: x.iloc[-1] - x.iloc[0]).reset_index()
    df_tst8_cut['FORMATION_up'] = 'Balakhany VIII'
    df_tst10_cut = df_net2_bal10_cut.groupby('well')[['TST', 'TVD_SCS']].apply(lambda x: x.iloc[-1] - x.iloc[0]).reset_index()
    df_tst10_cut['FORMATION_up'] = 'Balakhany X'
    df_kh_cut_fin = pd.concat([df_kh8_cut, df_kh10_cut])
    df_tst_cut_fin = pd.concat([df_tst8_cut, df_tst10_cut])
    df_tst_cut_fin.rename(columns={'TST':'tst_interv', 'TVD_SCS':'tvd_interv'}, inplace=True)
    df_tst_kh_cut = df_tst_cut_fin.set_index(['well','FORMATION_up']).join(df_kh_cut_fin.set_index(['well','FORMATION_up'])).reset_index()
    # Calculation average properties for Bal8 and Bal10 based on NET_clp2 and removed FU Bal VIII/X
    print('Calculation average properties for Bal8 and Bal10 based on NET_clp2 and removed FU Bal VIII/X')
    avg_prop_bal8_cut = avg_prop_calculation(ntd_net2_8_cut, df_net2_bal8_cut, 'Balakhany VIII')
    avg_prop_bal10_cut = avg_prop_calculation(ntd_net2_10_cut, df_net2_bal10_cut, 'Balakhany X')   
    avg_prop_final_cut = pd.concat([avg_prop_bal8_cut, avg_prop_bal10_cut])
    avg_prop_final_cut.sort_values(by='well', inplace=True)  
    # Joing KHtst/tst_interv and avg prop into one dataframe
    avg_prop_tst_kh_cut = avg_prop_final_cut.set_index(['well','FORMATION_up']).join(df_tst_kh_cut.set_index(['well','FORMATION_up'])).reset_index().sort_values(by='well')
    avg_prop_tst_kh_cut = avg_prop_tst_kh_cut.round({   'htst_max': 0, 'htst_sum': 0, 'tst_interv': 0, 'tvd_interv': 0, 'TVD_SCS': 0, 
                                                        'KHtst': 0, 'PHITHtst': 3, 'VSHHtst': 3, 'perm_wavg':5, 'phit_wavg':5})
    return avg_prop_tst_kh_cut                  
def prep_dist_calc(dataset, offset, blacklist):
    """
    Calculation properties based for offset wells: KHtst and phit_wavg
    Dataset - dataset with log data and any other variables 
    Offset - I prefer 0 or 300m, but it could be various 
    Blacklist - indices of rows which need to be removed
    """
    df_dist_kh_bal8_cut = dist_prop_calc(dataset, 'Balakhany VIII', offset, 'KHtst')
    df_dist_kh_bal10_cut = dist_prop_calc(dataset, 'Balakhany X', offset, 'KHtst')
    df_dist_kh_bal_cut_fin = pd.concat([df_dist_kh_bal8_cut, df_dist_kh_bal10_cut])
    df_dist_ph_bal8_cut = dist_prop_calc(dataset, 'Balakhany VIII', offset, 'phit_wavg')
    df_dist_ph_bal10_cut = dist_prop_calc(dataset, 'Balakhany X', offset, 'phit_wavg')
    df_dist_ph_bal_cut_fin = pd.concat([df_dist_ph_bal8_cut, df_dist_ph_bal10_cut])
    df_dist_htst_bal8_cut = dist_prop_calc(dataset, 'Balakhany VIII', offset, 'htst_sum')
    df_dist_htst_bal10_cut = dist_prop_calc(dataset, 'Balakhany X', offset, 'htst_sum')
    df_dist_htst_bal10_cut_fin = pd.concat([df_dist_htst_bal8_cut, df_dist_htst_bal10_cut])
    htst_dist = df_dist_htst_bal10_cut_fin[['well','FORMATION_up', 'htst_sum_1', 'htst_sum_2', 'htst_sum_3']]
    
    phit_dist = df_dist_ph_bal_cut_fin[['well','FORMATION_up', 'phit_wavg_1', 'phit_wavg_2', 'phit_wavg_3']]
    phit_dist_htst = phit_dist.set_index(['well','FORMATION_up']).join(htst_dist.set_index(['well','FORMATION_up'])).reset_index()
    df_dist_ph_kh_cut = df_dist_kh_bal_cut_fin.set_index(['well','FORMATION_up']).join(phit_dist_htst.set_index(['well','FORMATION_up']))
    avg_prop_tst_kh_dist_cut = dataset.set_index(['well','FORMATION_up']).join(df_dist_ph_kh_cut).reset_index()

    avg_prop_tst_kh_dist_cut['dist1_op'] = 1/avg_prop_tst_kh_dist_cut.dist1
    avg_prop_tst_kh_dist_cut['dist2_op'] = 1/avg_prop_tst_kh_dist_cut.dist2
    avg_prop_tst_kh_dist_cut['dist3_op'] = 1/avg_prop_tst_kh_dist_cut.dist3

    index_to_drop = blacklist
    avg_prop_tst_kh_cut_test = avg_prop_tst_kh_dist_cut.drop(index_to_drop).reset_index().drop('index', axis=1)
    return avg_prop_tst_kh_cut_test

In [ ]:
# KHtst calculation and join to the main dataframe df_bal_net2
df_kh = proph_calculation(df_bal_net2, 'NET_clp2')
df_bal_net2_kh_init = df_bal_net2.set_index(['well','MD']).join(df_kh.drop(['FORMATION_up', 'TST'], axis=1).set_index(['well','MD'])).reset_index()
# Cleaning KHtst dataset according to mentioned well lists
df_bal_net2_kh8 = df_bal_net2_kh_init[(df_bal_net2_kh_init.well.isin(well_clean8_v2)) & (df_bal_net2_kh_init.FORMATION_up == 'Balakhany VIII')]
df_bal_net2_kh10 = df_bal_net2_kh_init[(df_bal_net2_kh_init.well.isin(well_clean10_v2)) & (df_bal_net2_kh_init.FORMATION_up == 'Balakhany X')]
df_bal_net2_kh = pd.concat([df_bal_net2_kh8, df_bal_net2_kh10])
# Save df_bal_net2_kh to csv-file
save_tocsv(df_bal_net2_kh,'ACG_wells_JOINT_BEST_v10_output.csv', 0)
# Preparation full set of Bal8/10 and save it to csv
avg_prop_tst_kh = prep_fullset_bal_fu_calc_prop(df_bal_net2_kh)
save_tocsv(avg_prop_tst_kh,'avg_prop_tst_kh.csv', 0)
# Dist calculation for full set of Bal8/10
df_dist_kh_bal_fin = prep_dist_calc_brief(avg_prop_tst_kh, 0)

## Removing Bal VIII / X from FORMATION

In [ ]:
# df_bal_net2_kh_save = df_bal_net2_kh[['well', 'MD', 'FORMATION_up', 'NET', 'NET_clp', 'NET_clp2', 'TST', 'TST_smpl', 'X', 'Y','X_traj', 'Y_traj', 'field']]
# df_bal_net2_kh_save.to_csv('C:\\jupyter\\SPP\\inputoutput\\df_bal_net2_kh_save.csv', index=False)

### Experiments wiht NTD

In [ ]:
# # def avg_prop_calculation_expand(dataset_ntd, dataset, formation):
# dataset_ntd = ntd_net2_8_cut
# dataset = df_net2_bal8_cut
# well_data = []
# well_formation = 'Balakhany VIII'
# df_lst = []
# for well in tqdm(dataset_ntd.well.unique()[:]):
#     # print(well)
#     ntd_well_avgprop = dataset_ntd[(dataset_ntd.well ==well)]
#     well_avgprop_sel = dataset[(dataset.well==well)]
#     well_phit = []
#     well_vsh = []
#     well_gperm = []
#     well_h = []
#     well_name = []
#     well_fm = []
#     for layers in range(len(ntd_well_avgprop.well)):
#         ntd_top = ntd_well_avgprop.iloc[layers, 2].round(3)
#         ntd_bot = ntd_well_avgprop.iloc[layers, 3].round(3)
#         ntd_h = ntd_well_avgprop.iloc[layers, 4].round(3)
#         phit_lst = []
#         vsh_lst = []
#         perm_lst = []
#         for depth in range(len(well_avgprop_sel.TST)):
#             well_avgprop_tst = well_avgprop_sel['TST'].iloc[depth].round(3)
#             if well_avgprop_tst >= ntd_top and well_avgprop_tst <= ntd_bot:
#                 phit_lst.append(well_avgprop_sel['PHIT'].iloc[depth])
#                 vsh_lst.append(well_avgprop_sel['VSH'].iloc[depth])
#                 perm_lst.append(well_avgprop_sel['LPERM'].iloc[depth])
#         well_name.append(well)
#         well_fm.append(well_formation)
#         well_phit.append(mean(phit_lst))
#         well_vsh.append(mean(vsh_lst))
#         well_gperm.append(gmean(perm_lst))
#         well_h.append(ntd_h)
#         well_data = zip(well_name,well_fm,well_phit, well_vsh, well_gperm, well_h)
#         well_df = pd.DataFrame(well_data, columns=[  'well','FORMATION_up',        
#                                                 'phit_avg',
#                                                 'vsh_avg', 
#                                                 'perm_avg',
#                                                 'htst'])
#     df_lst.append(well_df)
# test1 = pd.concat(df_lst)

In [ ]:
# KHtst calculation and join to the main dataframe df_bal_net2
df_kh = proph_calculation(df_bal_net2, 'NET_clp2')
df_bal_net2_kh_init = df_bal_net2.set_index(['well','MD']).join(df_kh.drop(['FORMATION_up', 'TST'], axis=1).set_index(['well','MD'])).reset_index()
# Cleaning KHtst dataset according to mentioned well lists
df_bal_net2_kh8 = df_bal_net2_kh_init[(df_bal_net2_kh_init.well.isin(well_clean8_v2)) & (df_bal_net2_kh_init.FORMATION_up == 'Balakhany VIII')]
df_bal_net2_kh10 = df_bal_net2_kh_init[(df_bal_net2_kh_init.well.isin(well_clean10_v2)) & (df_bal_net2_kh_init.FORMATION_up == 'Balakhany X')]
df_bal_net2_kh = pd.concat([df_bal_net2_kh8, df_bal_net2_kh10])
# Removing upper part of Bal8/10 and property calc
avg_prop_tst_kh_cut = prep_remove_balfu_calc_prop(df_bal_net2_kh)
# Calculation of well properties and applying blacklist with indecies of outliers
blacklist = [244, 240, 320, 324]
avg_prop_tst_kh_cut0 = prep_dist_calc(avg_prop_tst_kh_cut, 0, blacklist)
avg_prop_tst_kh_cut0 = avg_prop_tst_kh_cut0.convert_dtypes()
well_offsets_df = avg_prop_tst_kh_cut0.copy()
#Saving well propetries dataset to csv
save_tocsv(avg_prop_tst_kh_cut0, 'avg_prop_tst_kh_dist0_cut.csv', 0)
# Displaying features list of dataframe
avg_prop_tst_kh_cut0.columns

### Maps

In [ ]:
# Drawing maps of well trajectories
def well_traj_dataprep(dataset):
    map_data = dataset[['well','FORMATION_up','FORMATION','X','Y','X_traj','Y_traj', 'Status']]
    map_data_top = map_data.groupby(['well','FORMATION_up'])[['X_traj','Y_traj']].apply(lambda x: x.iloc[0:-100:100]).reset_index()
    map_data_bot = map_data.groupby(['well','FORMATION_up'])[['X_traj','Y_traj']].apply(lambda x: x.iloc[-1]).reset_index()
    map_data_middle = map_data.groupby(['well','FORMATION_up'])[['X','Y', 'Status']].apply(lambda x: x.iloc[0]).reset_index()
    map_data_display = pd.concat([map_data_top, map_data_bot]).sort_values(by=['well','FORMATION_up']).drop('level_2', axis=1)
    return map_data_display, map_data_middle
map_data_display, map_data_middle = well_traj_dataprep(df_bal_net2_kh)
def display_well_traj(map_data_display, map_data_middle, fmname):
    map_data_display = map_data_display[map_data_display.FORMATION_up == fmname]
    map_data_middle = map_data_middle[map_data_middle.FORMATION_up == fmname]
    traj = go.Scatter(  x=map_data_display.X_traj, y=map_data_display.Y_traj, 
                        mode='markers',
                        marker=dict(color='black', size=3),
                        customdata = map_data_display[['well']],
                        hovertemplate="".join(["well:%{customdata[0]}<extra></extra>"]))
    wells = go.Scatter( x=map_data_middle.X, y=map_data_middle.Y, 
                        mode='markers',
                        marker=dict(symbol='diamond', color='red', size=7,
                        opacity=0.75, line=dict(color='rgb(47, 57, 61)', width=0.5)),
                        customdata = map_data_middle[['well', 'Status']],
                        hovertemplate="".join(["well:%{customdata[0]},%{customdata[1]}<extra></extra>"]))
    fig = go.Figure()
    fig.add_trace(traj)
    fig.add_trace(wells)
    fig.update_layout(  title_text= ('map of traj and well points'+ ' ' + fmname),
                        autosize=True, width=1300, height=600, margin=dict(l=10,r=10,b=10,t=50), showlegend=False)
    return fig.show()
display_well_traj(map_data_display, map_data_middle, 'Balakhany VIII')
display_well_traj(map_data_display, map_data_middle, 'Balakhany X')

### Prediction RFR for PHIT

#### Test 1 normal run 1-to-all

In [ ]:
ntd_net2_phit = avg_prop_tst_kh_cut0.copy(deep=True)
ntd_net2_phit = ntd_net2_phit[[ 'well',
                                'FORMATION_up',
                                'X', 'Y',
                                'TVD_SCS', 
                                # 'htst_sum',
                                # 'tst_interv',
                                'phit_wavg',
                                ]]
ntd_net2_phit_corr, ntd_net2_phit_gm = join_df_prediction(ntd_net2_phit, 'phit_wavg')
pairplot_special(ntd_net2_phit_corr, 7, 7, 1)

In [ ]:
# Run train-split block to get an optimized hyper parameters for 1-to-all function
gs_set = {  'bootstrap': [True, False],
            'max_depth': [10, 30, 100, 150, 200],
            'min_samples_leaf': [1, 5, 10],
            'min_samples_split': [1, 5, 10],
            'n_estimators': [10, 30, 100, 150, 200]}
scorer = make_scorer(r2, greater_is_better=True)
hyperdict = run_rfr_train_test_split(ntd_net2_phit_gm, gs_set, scorer, 'phit_wavg', 0.05, 0, 'phit_ml_result.txt', 'run_rfr_grid_search', 0, 0)

In [ ]:
# Run 1-to-all block to to predict target vaiable
phit_pred_simple_loop = run_rfr_1_to_all(ntd_net2_phit_gm, hyperdict, 'phit_wavg', 0.05, 0, 'phit_ml_result.txt', 'run_rfr_1-to-all', 1, 0.3, 0)
phit_pred_simple_loop =  phit_pred_simple_loop.rename(columns={'predict':'phit_wavg_smpl'})[['well','FORMATION_up','phit_wavg_smpl','qc']].sort_values(
                                            by=['well','FORMATION_up']).reset_index().drop('index', axis=1)

#### Test 2 w1w2w3 run 1-to-all - paused

In [ ]:
# ntd_net2_phit = avg_prop_tst_kh_cut0.copy(deep=True)
# ntd_net2_phit_w1w2w3 = ntd_net2_phit[[  'well',
#                                         'FORMATION_up',
#                                         'X', 'Y',
#                                         # 'TVD_SCS', 
#                                         'htst_sum',
#                                         # 'tst_interv',
#                                         'htst_sum_1','htst_sum_2', 'htst_sum_3', 
#                                         'dist1', 'dist2', 'dist3',
#                                         'phit_wavg_1', 'phit_wavg_2','phit_wavg_3',
#                                         'phit_wavg',
#                                         ]]
# ntd_net2_phit_w1w2w3_corr, ntd_net2_phit_w1w2w3_gm = join_df_prediction(ntd_net2_phit_w1w2w3, 'phit_wavg')
# pairplot_special(ntd_net2_phit_w1w2w3_corr, 9, 9, 0)
# def func_phit_pred_w123(dataset, hyperdict, flag_closest=0):
#     """
#     dataset:        just dataset with offset phit
#     flag_closest:   if 0 -> avg_prediction, if 1 -> find_closest_phit
#     """
#     ds1 = dataset.drop(['phit_wavg_2','dist2','htst_sum_2',	'phit_wavg_3', 'dist3','htst_sum_3'], axis=1)
#     ds2 = dataset.drop(['phit_wavg_1','dist1','htst_sum_1',	'phit_wavg_3', 'dist3','htst_sum_3'], axis=1)
#     ds3 = dataset.drop(['phit_wavg_1','dist1','htst_sum_1',	'phit_wavg_2', 'dist2', 'htst_sum_2'], axis=1)
#     test_ph1 = rfr_loop(ds1, 'FORMATION_up', 'phit_wavg', hyperdict, 0.05, 0)
#     test_ph2 = rfr_loop(ds2, 'FORMATION_up', 'phit_wavg', hyperdict, 0.05, 0)
#     test_ph3 = rfr_loop(ds3, 'FORMATION_up', 'phit_wavg', hyperdict, 0.05, 0)
#     test_ph1['result_df']['dataset'] = 'test'
#     test_ph2['result_df']['dataset'] = 'test'
#     test_ph3['result_df']['dataset'] = 'test'

#     write_res_file('phit_ml_result_w1w2w3.txt', 'rfr_train_test_split_w1', 'phit_wavg', 
#                     test_ph1['train_ds'], test_ph1['metrics'], test_ph1['grid_search'])
#     write_res_file('phit_ml_result_w1w2w3.txt', 'rfr_train_test_split_w2', 'phit_wavg', 
#                     test_ph2['train_ds'], test_ph2['metrics'], test_ph2['grid_search'])
#     write_res_file('phit_ml_result_w1w2w3.txt', 'rfr_train_test_split_w3', 'phit_wavg', 
#                     test_ph3['train_ds'], test_ph3['metrics'], test_ph3['grid_search'])
#     print('test_ph1: ',test_ph1['train_ds'])
#     print('test_ph1: ',test_ph1['metrics'])
#     # print('test_ph2: ',test_ph2['train_ds'])
#     print('test_ph2: ',test_ph2['metrics'])
#     # print('test_ph3: ',test_ph3['train_ds'])
#     print('test_ph3: ',test_ph3['metrics'])

#     xplot_qc2(test_ph1['result_df'], 0.3, 0.05, 0)
#     xplot_qc2(test_ph2['result_df'], 0.3, 0.05, 0)
#     xplot_qc2(test_ph3['result_df'], 0.3, 0.05, 0)

#     test_w1 = test_ph1['result_df'][['well','FORMATION_up','actual','predict','qc']].sort_values(by='well').reset_index().drop('index', axis=1)
#     test_w1['run'] = 'well1'
#     test_w2 = test_ph2['result_df'][['well','FORMATION_up','actual','predict','qc']].sort_values(by='well').reset_index().drop('index', axis=1)
#     test_w2['run'] = 'well2'
#     test_w3 = test_ph3['result_df'][['well','FORMATION_up','actual','predict','qc']].sort_values(by='well').reset_index().drop('index', axis=1)
#     test_w3['run'] = 'well3'
#     test_w1w2w3 = pd.concat([test_w1, test_w2, test_w3])
#     test_w1w2w3_actual = test_w1w2w3[['well', 'FORMATION_up', 'actual']].drop_duplicates()

#     # Making up the 'in'-dataset and equal it to w1w2w3_pred
#     test_w1w2w3_pred = test_w1w2w3[test_w1w2w3.qc=='in'].sort_values(by='well').groupby(
#                                         ['well','FORMATION_up'])['predict'].apply(lambda x: list(x)).reset_index()
#     test_w1w2w3_pred.rename(columns={'predict':'predict_list'}, inplace=True)
#     test_w1w2w3_pred['qc'] = 'in'
#     w1w2w3_pred = test_w1w2w3_pred
#     # Making up the 'out'-dataset
#     test_w1w2w3_unpred = test_w1w2w3[test_w1w2w3.qc=='out'].sort_values(by='well').groupby(
#                                         ['well','FORMATION_up'])['predict'].apply(lambda x: list(x)).reset_index()
#     test_w1w2w3_unpred.rename(columns={'predict':'predict_list'}, inplace=True)
#     test_w1w2w3_unpred['qc'] = 'out'
#     # Cleaning unpred dataset to get pure unpdredicted wells
#     w1w2w3_unpred  = test_w1w2w3_pred.set_index(['well','FORMATION_up']).join(
#                         test_w1w2w3_unpred.set_index(['well','FORMATION_up']), how='outer', rsuffix='_r').reset_index()
#     w1w2w3_unpred = w1w2w3_unpred[w1w2w3_unpred.qc.isna()].drop(['predict_list', 'qc'], axis=1)

#     if flag_closest == 0:
#         # Averaging lists of pred & unpred data and concat the final dataset
#         print('avg_prediction approach')
#         def avg_prediction(dataset):
#             lst = []
#             for i in range(len(dataset)):
#                 lst.append((dataset.iloc[i, 0], dataset.iloc[i, 1], mean(dataset.iloc[i, 2])))
#             result1 = pd.DataFrame(lst, columns=['well', 'FORMATION_up', 'phit_wavg_pred'])
#             return result1
#         avg_pred = avg_prediction(w1w2w3_pred)
#         avg_pred['qc'] = 'in'
#         avg_unpred = avg_prediction(w1w2w3_unpred)
#         avg_unpred['qc'] = 'out'
#         phit_pred_w1w2w3 = pd.concat([avg_pred, avg_unpred]).reset_index().drop('index', axis=1).sort_values(by='well')
#         phit_pred_w1w2w3.rename(columns={'phit_wavg_pred':'phit_avg'}, inplace=True)
#         result = phit_pred_w1w2w3
#         print(phit_pred_w1w2w3.qc.value_counts(normalize=True))
#     if flag_closest == 1:
#         # Finding closest predicted value to actual one and concat the final dataset
#         print('find_closest_phit approach')
#         def find_closest_phit(dataset):
#             split_columns = dataset['predict_list'].apply(pd.Series)
#             df_test = pd.concat([dataset, split_columns], axis=1)
#             df_test_v2 = df_test.set_index(['well','FORMATION_up']).join(ntd_net2_phit_w1w2w3.set_index(['well','FORMATION_up'])).reset_index()
#             df_test_v2 = df_test_v2[['well','FORMATION_up',0,1,2, 'phit_wavg']]
#             df_test_v2['d1'] = abs(df_test_v2[0] - df_test_v2.phit_wavg)
#             df_test_v2['d2'] = abs(df_test_v2[1] - df_test_v2.phit_wavg)
#             df_test_v2['d3'] = abs(df_test_v2[2] - df_test_v2.phit_wavg)
#             df_test_v2 = df_test_v2.fillna(9999)
#             list_close_phit = []
#             for index, row in df_test_v2.iterrows():
#                 my_list = [row['d1'], row['d2'], row['d3']]
#                 my_index =  my_list.index(min(my_list))
#                 if my_index == 0:
#                     list_close_phit.append((row['well'],row['FORMATION_up'],row[0]))
#                 if my_index == 1:
#                     list_close_phit.append((row['well'],row['FORMATION_up'],row[1]))
#                 if my_index == 2:
#                     list_close_phit.append((row['well'],row['FORMATION_up'],row[2]))
#             result2 = pd.DataFrame(list_close_phit, columns=['well','FORMATION_up','phit_closest'])
#             return result2
#         test_pred = find_closest_phit(w1w2w3_pred)
#         test_pred['qc'] = 'in'
#         test_unpred = find_closest_phit(w1w2w3_unpred.rename(columns={'predict_list_r':'predict_list'}))
#         test_unpred['qc'] = 'out'
#         test_closest = pd.concat([test_pred, test_unpred])
#         result = test_closest
#         print(test_closest.qc.value_counts(normalize=True))
#     return result
# phit_pred_w1w2w3 = func_phit_pred_w123(ntd_net2_phit_w1w2w3_gm, hyperdict, 0)

In [ ]:
# # Making up well list for Farid once again
# farid_list = ['A19', 'J15', 'J15Z', 'J16']
# # with X, Y, TVD, porosity and GR 
# test_var1 = avg_prop_tst_kh_cut0[['well', 'FORMATION_up','well1', 'well2','well3']]
# test_var1 = test_var1[test_var1.well.isin(['A19', 'J15', 'J15Z', 'J16'])]
# well_list = []
# for iter, row in test_var1.iterrows():
#     well_list.append(row['well1'])
#     well_list.append(row['well2'])
#     well_list.append(row['well3'])
# well_list2 = set(well_list)
# farid_wells = df_bal_net2_kh[df_bal_net2_kh.well.isin(well_list2)]
# print(well_list2)
# farid_wells[['well', 'FORMATION_up', 'X_traj', 'Y_traj', 'TVD_SCS', 'PHIT', 'GR_N' ]].to_csv('farid_wells2.csv', index=False)

#### Test 3 w1w2w3 into 1 dataframe 1-to-all

In [ ]:
test_1df_3offset_wells = avg_prop_tst_kh_cut0.copy()
var_list = ['well', 'FORMATION_up', 
            'htst_sum_1','htst_sum_2','htst_sum_3',
            'phit_wavg_1', 'phit_wavg_2', 'phit_wavg_3', 
            'dist1', 'dist2', 'dist3',
            'x1', 'x2', 'x3', 'y1', 'y2', 'y3',
            'X', 'Y',
            'phit_wavg'
            ]
test_1df_3offset_wells = test_1df_3offset_wells[var_list]
ds1 = test_1df_3offset_wells.drop(['phit_wavg_2','htst_sum_2','dist2', 'x2','y2',	'phit_wavg_3','htst_sum_3','dist3', 'x3','y3'], axis=1)
ds2 = test_1df_3offset_wells.drop(['phit_wavg_1','htst_sum_1','dist1', 'x1','y1',	'phit_wavg_3','htst_sum_3','dist3', 'x3','y3'], axis=1)
ds3 = test_1df_3offset_wells.drop(['phit_wavg_1','htst_sum_1','dist1', 'x1','y1',	'phit_wavg_2','htst_sum_2','dist2', 'x2','y2'], axis=1)
def gm_rnm(df):
    df['FORMATION_up_copy'] = df['FORMATION_up']
    df = pd.get_dummies(df, columns=['FORMATION_up_copy'], drop_first=True)
    df.rename(columns={ 'FORMATION_up_copy_Balakhany X':'FORMATION_up_gm',
                        'phit_wavg_1':'phit_wavg_offset',
                        'phit_wavg_2':'phit_wavg_offset',
                        'phit_wavg_3':'phit_wavg_offset',
                        'htst_sum_1':'htst_sum_offset',
                        'htst_sum_2':'htst_sum_offset',
                        'htst_sum_3':'htst_sum_offset',
                        'dist1':'dist_offset',
                        'dist2':'dist_offset',
                        'dist3':'dist_offset',
                        'x1':'x_offset',
                        'x2':'x_offset',
                        'x3':'x_offset',
                        'y1':'y_offset',
                        'y2':'y_offset',
                        'y3':'y_offset',}, inplace=True)
    return df 
test_1df_3offset_wells_fin = pd.concat([gm_rnm(ds1), gm_rnm(ds2), gm_rnm(ds3)]).reset_index().drop('index', axis=1)
drop_list = [   
                'htst_sum_offset',
                'x_offset', 'y_offset'
            ]
test_1df_3offset_wells_fin = test_1df_3offset_wells_fin.drop(drop_list, axis=1)

In [ ]:
# Run train-split block to get an optimized hyper parameters for 1-to-all function
gs_set = {  'bootstrap': [True, False],
            'max_depth': [10, 30, 100, 150, 200],
            'min_samples_leaf': [1, 5, 10],
            'min_samples_split': [1, 5, 10],
            'n_estimators': [10, 30, 100, 150, 200]}
# scorer_mse = make_scorer(mse, greater_is_better=False)
# hyperdict_1df_3offset_mse = run_rfr_train_test_split(test_1df_3offset_wells_fin, gs_set, scorer_mse, 
#                                                 'phit_wavg', 0.05, 0, 'phit_ml_result_w1w2w3.txt', 'run_rfr_1df_3offset_grd_srch_mse', 0, 0)
# scorer_mae = make_scorer(mae, greater_is_better=False)
# hyperdict_1df_3offset_mae = run_rfr_train_test_split(test_1df_3offset_wells_fin, gs_set, scorer_mae, 
#                                                 'phit_wavg', 0.05, 0, 'phit_ml_result_w1w2w3.txt', 'run_rfr_1df_3offset_grd_srch_mse', 0, 0)
scorer_r2 = make_scorer(r2, greater_is_better=True)
hyperdict_1df_3offset_r2 = run_rfr_train_test_split(test_1df_3offset_wells_fin, gs_set, scorer_r2, 
                                                'phit_wavg', 0.05, 0, 'phit_ml_result_w1w2w3.txt', 'run_rfr_1df_3offset_grd_srch_mse', 0, 0)

In [ ]:
# Run 1-to-all block to to predict target vaiable
pred_phit_3offset_1df_loop = run_rfr_1_to_all(test_1df_3offset_wells_fin, hyperdict_1df_3offset_r2, 
                                            'phit_wavg', 0.05, 0, 'phit_ml_result_w1w2w3.txt', '1df_3offset_loop', 1, 0.3, 1)
phit_pred_w1w2w3_v2_init = pred_phit_3offset_1df_loop[['well','FORMATION_up','predict','qc']]
phit_pred_w1w2w3_v2_init.rename(columns={'predict':'phit_w123_1df'}, inplace=True)
phit_pred_w1w2w3_v2 = phit_pred_w1w2w3_v2_init.groupby(['well','FORMATION_up']).agg({'phit_w123_1df':'mean', 'qc': lambda x: x.mode()[0]}).reset_index()

### Prediction RFR for KHtst

#### Prediction with phit_pred_simple - paused

In [ ]:
# ntd_net2_kh = avg_prop_tst_kh_cut0.copy(deep=True)
# ntd_net2_kh = ntd_net2_kh[[ 'well',
#                             'FORMATION_up',
#                             # 'TVD_SCS',
#                             'X', 'Y',
#                             'htst_sum',
#                             # 'tst_interv',
#                             'KHtst',
#                             ]]
# ntd_net2_kh_phit_smpl_corr, ntd_net2_kh_phit_smpl_gm = join_add_df_prediction(ntd_net2_kh, phit_pred_simple.drop('qc', axis=1), 'KHtst')
# print(ntd_net2_kh_phit_smpl_gm.columns)
# pairplot_special(ntd_net2_kh_phit_smpl_corr, 8, 8, 0)

In [ ]:
# gs_set = {  'bootstrap': [True, False],
#             'max_depth': [10, 30, 100, 150, 200],
#             'min_samples_leaf': [1, 5, 10],
#             'min_samples_split': [1, 5, 10],
#             'n_estimators': [10, 30, 100, 150, 200]}
# scorer = make_scorer(r2, greater_is_better=True)
# hyperdict_kh_smpl = run_rfr_train_test_split(   ntd_net2_kh_phit_smpl_gm, gs_set, scorer, 
#                                                 'KHtst', 0.25, 300, 'pred_kh_phsmpl.txt', 'rfr_train_test_split_phsmpl', 0, 0)

In [ ]:
# pred_kh_smpl = run_rfr_1_to_all(ntd_net2_kh_phit_smpl_gm, hyperdict_kh_smpl, 
#                                 'KHtst', 0.25, 300, 'pred_kh_phsmpl.txt', 'rfr_1-to-all', 1, 27000, 1)

#### Prediction with phit_pred_w1w2w3

In [ ]:
ntd_net2_kh = avg_prop_tst_kh_cut0.copy(deep=True)
ntd_net2_kh = ntd_net2_kh[[ 'well',
                            'FORMATION_up',
                            'TVD_SCS',
                            'X', 'Y',
                            'htst_sum',
                            # 'htst_p50', 'htst_count',
                            'tst_interv',
                            # 'phit_wavg',
                            # 'KHtst_1', 'dist1',
                            # 'vsh_wavg',
                            'KHtst',
                            ]]
ntd_net2_kh_phit_w123_corr, ntd_net2_kh_phit_w123_gm = join_add_df_prediction(ntd_net2_kh, phit_pred_w1w2w3_v2.drop('qc', axis=1), 'KHtst')
print(ntd_net2_kh_phit_w123_gm.columns)
pairplot_special(ntd_net2_kh_phit_w123_corr, 8, 8, 0)

In [ ]:
gs_set = {  'bootstrap': [True],
            'max_depth': [20, 30, 50, 100, 200],
            'min_samples_leaf': [1, 2, 3, 5],
            'min_samples_split': [1, 2, 3, 5],
            'n_estimators': [75, 100, 150, 175]}
scorer = make_scorer(r2, greater_is_better=True)
hyperdict_kh_w123 = run_rfr_train_test_split(   ntd_net2_kh_phit_w123_gm, gs_set, scorer, 
                                                'KHtst', 0.25, 300, 'pred_kh_phw123.txt', 'rfr_train_test_split_grd_srch', 0, 0)

In [ ]:
pred_kh_w123 = run_rfr_1_to_all(    ntd_net2_kh_phit_w123_gm, hyperdict_kh_w123, 
                                    'KHtst', 0.25, 300, 'pred_kh_phw123.txt', 'rfr_1-to-all', 1, 27000, 1)

In [ ]:
def resid_analize_plot(data, max_val, y_lim, text):
    data = data.round({'actual':0, 'predict':0, 'diff':0})
    colors = {'in': 'green', 'out': 'red'}
    qc_colors = [colors[qc] for qc in data.qc]
    scatter = go.Scatter(   x=data.actual, y=data['diff'],
                            mode='markers',
                            marker=dict(color=qc_colors, size=7, opacity=0.75, line=dict(color='rgb(47, 57, 61)', width=0.5)),
                            customdata = data[['well','actual','predict', 'diff', 'FORMATION_up']],
                            hovertemplate="".join(
                            ["w:%{customdata[0]},a:%{customdata[1]}, p:%{customdata[2]}, d:%{customdata[3]}, f:%{customdata[4]}<extra></extra>"])
                            )
    fig = go.Figure()
    line = go.Scatter(x=[0, max_val], y=[0, 0], mode='lines+markers', line=dict(color='black'))
    fig.add_trace(scatter)
    fig.add_trace(line)
    fig.update_xaxes(title_text='actual')
    fig.update_yaxes(title_text='diff')
    fig.update_layout(  title_text= ('Residuals analize' + ' ' + text), width=500, height=500,
                        xaxis=dict(range=[0, max_val]), yaxis=dict(range=[-1*(y_lim), y_lim]),
                        margin=dict(l=10,r=10,b=10,t=75), showlegend=False)
    return fig.show()
diff_analize = pred_kh_w123
diff_analize8 = diff_analize[diff_analize.FORMATION_up == 'Balakhany VIII']
diff_analize10 = diff_analize[diff_analize.FORMATION_up == 'Balakhany X']
max_val = 30000
print('diff = actual - predict')
resid_analize_plot(diff_analize8, 30000, 15000, 'Balakhany VIII')
resid_analize_plot(diff_analize10, 15000, 10000, 'Balakhany X')

In [ ]:
# Residuals analize with map. Maps are printed to C:\jupyter\SPP\plots
def map_qc(text, metadata, data, fmname, scale, nameplot):
    path = 'C:\\jupyter\\SPP\\plots\\'
    data = data[data.FORMATION_up == fmname]
    data_in = data[data.qc=='in']
    data_out = data[data.qc=='out']
    field_avg_coord = metadata.groupby('field')[['X_wellhead','Y_wellhead']].mean().reset_index()
    platform  = go.Scatter(         x=field_avg_coord.X_wellhead, y=field_avg_coord.Y_wellhead, customdata = field_avg_coord[['field']],
                                    text=field_avg_coord['field'], textposition="middle right",
                                    marker=dict(color='rgb(0, 0,0)', size=12, opacity=0.75),
                                    mode='markers+text', 
                                    marker_symbol='square', hovertemplate="".join(["%{customdata[0]}<extra></extra>"])
                                    )
    scatter_data_in = go.Scatter(   x=data_in.X, y=data_in.Y,
                                    mode='markers',
                                    marker=dict(symbol='circle', color='green', size=data_in['actual']*scale,
                                    opacity=0.75, line=dict(color='rgb(47, 57, 61)', width=0.5)
                                    ),
                                    customdata = data_in[['well', 'actual', 'predict', 'diff']],
                                    hovertemplate="".join(["w:%{customdata[0]}, a:%{customdata[1]}, p:%{customdata[2]}, d:%{customdata[3]}<extra></extra>"])
                                    )
    scatter_data_out = go.Scatter(  x=data_out.X, y=data_out.Y, 
                                    mode='markers',
                                    marker=dict(symbol='diamond', color='red', size=data_out['actual']*scale,
                                    opacity=0.75, line=dict(color='rgb(47, 57, 61)', width=0.5)),
                                    customdata = data_out[['well', 'actual', 'predict', 'diff']],
                                    hovertemplate="".join(["w:%{customdata[0]}, a:%{customdata[1]}, p:%{customdata[2]}, d:%{customdata[3]}<extra></extra>"])
                                    )
    fig = go.Figure()
    fig.add_trace(platform)
    fig.add_trace(scatter_data_in)
    fig.add_trace(scatter_data_out)
    fig.update_layout(title_text= (text),autosize=True, width=1000, height=600, margin=dict(l=10,r=10,b=10,t=50), showlegend=False)
    return fig.write_html(path + nameplot)
kh_map = pred_kh_w123.set_index(['well','FORMATION_up']).join(ntd_net2_kh.set_index(['well','FORMATION_up'])).reset_index()
txt = 'rfr_1-to-all phit_pred_w123 size_out[actual]'
map_qc(txt, metadata, kh_map, 'Balakhany VIII', 0.001, 'map_bal8.html')
map_qc(txt, metadata, kh_map, 'Balakhany X', 0.0025,'map_bal10.html')

In [ ]:
w_lst = ['C16','C15','C12']
rhob_neu = df_bal_net2_kh[  (df_bal_net2_kh.well.isin(w_lst)) & 
                            (df_bal_net2_kh.FORMATION_up == 'Balakhany VIII') & 
                            (df_bal_net2_kh.NET_clp2 == 1)]
sns.histplot(rhob_neu, x="RHOB", hue='well', kde=True)

In [ ]:
avg_prop_tst_kh_cut0[   (avg_prop_tst_kh_cut0.well == 'C16') & 
                        (avg_prop_tst_kh_cut0.FORMATION_up=='Balakhany VIII')][['well','FORMATION_up',
                        'KHtst','phit_wavg','well1','well2', 'well3',
                        'KHtst_1', 'KHtst_2', 'KHtst_3', 
                        'phit_wavg_1', 'phit_wavg_2', 'phit_wavg_3']]

### XGBoost regressor

In [ ]:
ntd_net2_kh = avg_prop_tst_kh_cut0.copy(deep=True)
ntd_net2_kh = ntd_net2_kh[[ 'well',
                            'FORMATION_up',
                            'X', 'Y',
                            'htst_sum',
                            # 'KHtst_1',
                            'KHtst',
                            ]]
ntd_net2_kh_phit_w123_corr, ntd_net2_kh_phit_w123_gm = join_add_df_prediction(ntd_net2_kh, phit_pred_w1w2w3_v2.drop('qc',axis=1), 'KHtst')
print(ntd_net2_kh_phit_w123_gm.columns)
pairplot_special(ntd_net2_kh_phit_w123_corr, 8, 8, 0)

In [ ]:
hyper_xgbr = {  'objectvie':'reg:squarederror', 
                'booster':'gbtree',
                'max_depth': 25,
                'n_estimators': 300,
                'max_leaves': 100,
                'learning_rate':0.1}

In [ ]:
def run_xgbr_1_to_all(dataset, hyperdict, target, rng, margin, logtxt_name, comment, xplot_flag, max_val, ftr_imp_flag):
    loop_res = xgbr_loop(dataset, 'FORMATION_up', target, hyperdict, rng, margin)
    write_res_file(logtxt_name, comment, target, loop_res['train_ds'], loop_res['metrics'], loop_res['grid_search'])
    loop_res_pred = loop_res['result_df']
    print('train_ds: ',loop_res['train_ftrs'])
    print('metrics: ',loop_res['metrics'])
    if xplot_flag == 1:
        xplot_qc2_loop(loop_res['result_df'], max_val, rng, margin)
    else:
        pass
    if ftr_imp_flag == 1:
        feature_imp_split(loop_res['feature_imp'], 6, 4)
    else:
        pass
    return loop_res_pred
test_xgbr = run_xgbr_1_to_all(  ntd_net2_kh_phit_w123_gm, hyper_xgbr, 
                                'KHtst', 0.25, 300, 'kh_ml_xgbr_result.txt', 'xgbr_loop', 1, 27000, 0)

In [ ]:
diff_analize = test_xgbr
diff_analize8 = diff_analize[diff_analize.FORMATION_up == 'Balakhany VIII']
diff_analize10 = diff_analize[diff_analize.FORMATION_up == 'Balakhany X']
max_val = 30000
print('diff = actual - predict')
resid_analize_plot(diff_analize8, 30000, 15000, 'Balakhany VIII')
resid_analize_plot(diff_analize10, 15000, 10000, 'Balakhany X')

### Experiments with phit_maps

#### Phit map uploading and processing

In [ ]:
def net_bal810_upload():
    net_maps_bal8_sand_init = pd.read_csv(r'C:\jupyter\SPP\input\maps\net\net_maps_Balakhany_VIII_Sand.csv')
    net_maps_bal8_20_init = pd.read_csv(r'C:\jupyter\SPP\input\maps\net\net_maps_Balakhany_VIII_20.csv')
    net_maps_bal8_10_init = pd.read_csv(r'C:\jupyter\SPP\input\maps\net\net_maps_Balakhany_VIII_10.csv')
    net_maps_bal8_05_init = pd.read_csv(r'C:\jupyter\SPP\input\maps\net\net_maps_Balakhany_VIII_5.csv')
    net_maps_bal8_sand = net_maps_bal8_sand_init[['x','y','NET','Formation']]
    net_maps_bal8_20 = net_maps_bal8_20_init[['x','y','NET','Formation']]
    net_maps_bal8_10 = net_maps_bal8_10_init[['x','y','NET','Formation']]
    net_maps_bal8_05 = net_maps_bal8_05_init[['x','y','NET','Formation']]

    net_maps_bal10_20_init = pd.read_csv(r'C:\jupyter\SPP\input\maps\net\net_maps_Balakhany_X_20.csv')
    net_maps_bal10_40_init = pd.read_csv(r'C:\jupyter\SPP\input\maps\net\net_maps_Balakhany_X_40.csv')
    net_maps_bal10_sand_init = pd.read_csv(r'C:\jupyter\SPP\input\maps\net\net_maps_Balakhany_X_Sand.csv')
    net_maps_bal10_20 = net_maps_bal10_20_init[['x','y','NET','Formation']]
    net_maps_bal10_40 = net_maps_bal10_40_init[['x','y','NET','Formation']]
    net_maps_bal10_sand = net_maps_bal10_sand_init[['x','y','NET','Formation']]

    net_maps_bal8 = pd.concat([net_maps_bal8_sand, net_maps_bal8_20, net_maps_bal8_10, net_maps_bal8_05])
    net_maps_bal10 = pd.concat([net_maps_bal10_20, net_maps_bal10_40, net_maps_bal10_sand])
    net_maps_bal8.rename(columns={'NET':'NTG'}, inplace=True)
    net_maps_bal10.rename(columns={'NET':'NTG'}, inplace=True)
    net_maps_bal10['NTG'] =abs(net_maps_bal10.NTG)
    return net_maps_bal8, net_maps_bal10
def phit_bal810_upload():
    phit_maps_bal8_sand_init = pd.read_csv(r'C:\jupyter\SPP\input\maps\phit\phit_maps_Balakhany_VIII_Sand.csv')
    phit_maps_bal8_20_init = pd.read_csv(r'C:\jupyter\SPP\input\maps\phit\phit_maps_Balakhany_VIII_20.csv')
    phit_maps_bal8_10_init = pd.read_csv(r'C:\jupyter\SPP\input\maps\phit\phit_maps_Balakhany_VIII_10.csv')
    phit_maps_bal8_05_init = pd.read_csv(r'C:\jupyter\SPP\input\maps\phit\phit_maps_Balakhany_VIII_5.csv')
    phit_maps_bal8_sand = phit_maps_bal8_sand_init[['x','y','PHIT','Formation']]
    phit_maps_bal8_20 = phit_maps_bal8_20_init[['x','y','PHIT','Formation']]
    phit_maps_bal8_10 = phit_maps_bal8_10_init[['x','y','PHIT','Formation']]
    phit_maps_bal8_05 = phit_maps_bal8_05_init[['x','y','PHIT','Formation']]

    phit_maps_bal10_20_init = pd.read_csv(r'C:\jupyter\SPP\input\maps\phit\phit_maps_Balakhany_X_20.csv')
    phit_maps_bal10_40_init = pd.read_csv(r'C:\jupyter\SPP\input\maps\phit\phit_maps_Balakhany_X_40.csv')
    phit_maps_bal10_sand_init = pd.read_csv(r'C:\jupyter\SPP\input\maps\phit\phit_maps_Balakhany_X_Sand.csv')

    phit_maps_bal10_20 = phit_maps_bal10_20_init[['x','y','PHIT','Formation']]
    phit_maps_bal10_40 = phit_maps_bal10_40_init[['x','y','PHIT','Formation']]
    phit_maps_bal10_sand = phit_maps_bal10_sand_init[['x','y','PHIT','Formation']]

    phit_maps_bal8 = pd.concat([phit_maps_bal8_sand, phit_maps_bal8_20, phit_maps_bal8_10, phit_maps_bal8_05])
    phit_maps_bal10 = pd.concat([phit_maps_bal10_20, phit_maps_bal10_40, phit_maps_bal10_sand])                               
    phit_maps_bal8.rename(columns={'PHIT':'phit_map'}, inplace=True)
    phit_maps_bal10.rename(columns={'PHIT':'phit_map'}, inplace=True)
    phit_maps_bal10['phit_map'] =abs(phit_maps_bal10.phit_map)
    return phit_maps_bal8, phit_maps_bal10
def map_prop_accum(logging_df, map_df, well_var_x, well_var_y, map_var, zone_size):
    """
    logging_df - dataframe with well data
    map_df - datafram with map data
    well_var_x, well_var_y - well coordinates from wells dataframe
    map_var - name of map's variable
    margin - distance +/- to accumulate map's data
    """
    df_lst = []
    for i in range(len(logging_df.well.unique())):
        well_name = logging_df.iloc[i]['well']
        well_x = logging_df.iloc[i][well_var_x]
        well_y =logging_df.iloc[i][well_var_y]
        var_map_avg = map_df[   (map_df.x < well_x + zone_size) &
                                (map_df.x > well_x - zone_size) &
                                (map_df.y < well_y + zone_size) &
                                (map_df.y > well_y - zone_size)].reset_index()
        var_map_avg=var_map_avg[[map_var,'Formation']]
        var_map_avg['well'] = well_name
        df_lst.append(var_map_avg)
    result = pd.concat(df_lst)
    return result
def map_phit_ntg_process(phit_map_towell, net_map_towell, phit_log, wavg_ntg_flag=1):
    if wavg_ntg_flag == 1:
        phit_net_memcell = phit_map_towell.set_index(['well','Formation']).join(net_map_towell.set_index(['well','Formation'])).reset_index()
        phit_maps_bal_towell_v2 = phit_net_memcell.groupby(['well','Formation']).mean().reset_index()
        phit_maps_bal_towell_v2['pnit_ntg'] = phit_maps_bal_towell_v2.phit_map*phit_maps_bal_towell_v2.NTG
        sum_ntg = phit_maps_bal_towell_v2.groupby('well')['NTG'].sum().reset_index().rename(columns = {'NTG':'sum_ntg'})
        phit_maps_bal_towell_v3 = phit_maps_bal_towell_v2.set_index('well').join(sum_ntg.set_index('well')).reset_index()
        sum_phit = phit_maps_bal_towell_v3.groupby('well')['pnit_ntg'].sum().reset_index().rename(columns = {'pnit_ntg':'sum_phit'})
        phit_maps_bal_towell_v4 = phit_maps_bal_towell_v3.set_index('well').join(sum_phit.set_index('well')).reset_index()
        phit_maps_bal_towell_v4['phit_wavg_map'] = phit_maps_bal_towell_v4.sum_phit / phit_maps_bal_towell_v4.sum_ntg
        phit_maps_bal_towell_v4 = phit_maps_bal_towell_v4[['well','Formation','phit_wavg_map']].groupby('well')[['well', 'phit_wavg_map']].apply(
                                                                                                lambda x: x.iloc[0]).drop('well',axis=1).reset_index()
        phit_log_v2 = phit_log.set_index('well').join(phit_maps_bal_towell_v4.set_index('well')).reset_index()
        result = phit_log_v2.round({'phit_wavg':3, 'phit_wavg_map':3})
    if wavg_ntg_flag == 0:
        phit_maps_bal_towell_v2 = phit_map_towell.groupby(['well','Formation']).mean().reset_index()
        phit_log_v2 = phit_log.set_index('well').join(phit_maps_bal_towell_v2.set_index('well')).reset_index()
        phit_log_v3 = phit_log_v2.groupby(['well','FORMATION_up'])['phit_map'].mean().reset_index()
        phit_log_v3 = phit_log_v3.rename(columns={'phit_map':'phit_avg_map'})
        phit_log_v4 = phit_log_v2.drop( ['Formation','phit_map'], axis=1).groupby(
                                        ['well','FORMATION_up']).apply(lambda x: x.iloc[0]).drop(['well','FORMATION_up'], axis=1)
        result = phit_log_v3.set_index(['well','FORMATION_up']).join(phit_log_v4).reset_index()
    return result
def phit_map_metrics(phit_map, target_value, rng, margin):
    phit_map['l_range'] = phit_map.phit_wavg*(1-rng) - margin 
    phit_map['h_range'] = phit_map.phit_wavg*(1+rng) + margin
    phit_map['qc'] = 'out'
    phit_map.loc[(phit_map[target_value] >= phit_map.l_range) & (phit_map[target_value] <= phit_map.h_range), 'qc'] = 'in'
    phit_map['diff'] = (phit_map.phit_wavg - phit_map[target_value]).round(3)
    metrics_dict = {    'wells_total':          phit_map.shape[0], 
                        'wells_unpred':         phit_map['qc'].value_counts()['out'],
                        'wells_unpred_v/v':     (phit_map['qc'].value_counts()['out']/phit_map.shape[0]).round(2),
                        'wells_pred':           phit_map['qc'].value_counts()['in'],
                        'wells_pred_v/v':       (phit_map['qc'].value_counts()['in']/phit_map.shape[0]).round(2)
                    }
    return metrics_dict
def xplot_qc_map(data, target_value, rng, max_val, margin):
    data = data.round({'phit_wavg':3, target_value:3})
    up_range = rng + 1
    dwn_range = 1 - rng
    colors = {'in': 'green', 'out': 'red'}
    qc_colors = [colors[qc] for qc in data.qc]
    scatter = go.Scatter( x=data.phit_wavg, y=data[target_value],
                            mode='markers',
                            marker=dict(color=qc_colors, size=7, opacity=0.75, line=dict(color='rgb(47, 57, 61)', width=0.5)),
                            customdata = data[['well','phit_wavg', target_value, 'diff', 'FORMATION_up']],
                            hovertemplate="".join(
                            ["well:%{customdata[0]}, phit_wavg: %{customdata[1]}, target_val:%{customdata[2]}, diff:%{customdata[3]}, fm:%{customdata[4]}<extra></extra>"])
                            )
    fig = go.Figure()
    line_up = go.Scatter(x=[0, max_val], y=[0 + margin, max_val*up_range + margin], mode='lines+markers', line=dict(color='blue'))
    line_dw = go.Scatter(x=[0, max_val], y=[0 - margin, max_val*dwn_range - margin], mode='lines+markers', marker=dict(color='blue'))
    fig.add_trace(scatter)
    fig.add_trace(line_up)
    fig.add_trace(line_dw)
    fig.update_xaxes(title_text='phit_wavg')
    fig.update_yaxes(title_text=target_value)
    fig.update_layout(  title_text= ('phit_map'), width=450, height=450, 
                        margin=dict(l=10,r=10,b=10,t=50), showlegend=False)
    return fig.show()

In [ ]:
# # Draw example of PHIT_map and example of well trajectory
# phit_maps_bal8_sand_init = pd.read_csv(r'C:\jupyter\SPP\input\maps\phit\phit_maps_Balakhany_VIII_Sand.csv')
# a01w = phit_log_compare8[phit_log_compare8.well=='A01W']
# phit_maps_bal8_sand = phit_maps_bal8_sand_init[['x','y','PHIT','Formation']]
# wtraj = df_bal_net2_kh[(df_bal_net2_kh.well == 'A01W') & (df_bal_net2_kh.FORMATION_up=='Balakhany VIII')][['X','Y']]
# mid = wtraj.mean()
# wx, wy = mid.X, mid.Y
# add = 50
# phit_maps_bal8_sand_sel = phit_maps_bal8_sand_init[ (phit_maps_bal8_sand.x < wx + add) &
#                                                     (phit_maps_bal8_sand.x > wx - add) &
#                                                     (phit_maps_bal8_sand.y < wy + add) &
#                                                     (phit_maps_bal8_sand.y > wy - add)].reset_index()
# data = phit_maps_bal8_sand_sel
# top = wtraj.iloc[0]
# bot = wtraj.iloc[-1]
# color_values = data.PHIT
# plt.scatter(data.x, data.y, c=color_values,  cmap='viridis')
# plt.scatter(wtraj.X, wtraj.Y)
# plt.scatter(top.X, top.Y, c='black', s=100)
# plt.scatter(bot.X, bot.Y, c='red', s=100)
# plt.scatter(mid.X, mid.Y, c='purple', s=100)

# sm = plt.cm.ScalarMappable(cmap='viridis')
# sm.set_array(color_values)
# sm.set_clim(np.min(color_values), np.max(color_values))
# cbar = plt.colorbar(sm)
# cbar.set_label('PHIT')

# plt.grid()
# plt.show()

In [ ]:
# Uploading data from CSV
net_maps_bal8, net_maps_bal10 = net_bal810_upload()
phit_maps_bal8, phit_maps_bal10 = phit_bal810_upload()
# Getting middle x/y-coordinates from wells for each formation
xy_middle = df_bal_net2_kh[['well','FORMATION_up', 'X','Y']]
xy_middle8 = xy_middle[xy_middle.FORMATION_up == 'Balakhany VIII'].groupby('well').apply(lambda x: x.iloc[0]).drop('well', axis=1).reset_index()
xy_middle10 = xy_middle[xy_middle.FORMATION_up == 'Balakhany X'].groupby('well').apply(lambda x: x.iloc[0]).drop('well', axis=1).reset_index()
phit_wavg8 = avg_prop_tst_kh_cut0[avg_prop_tst_kh_cut0.FORMATION_up == 'Balakhany VIII'][['well','phit_wavg']]
phit_wavg10 = avg_prop_tst_kh_cut0[avg_prop_tst_kh_cut0.FORMATION_up == 'Balakhany X'][['well','phit_wavg']]
phit_wavg8_xy = pd.merge(phit_wavg8, xy_middle8, on='well')
phit_wavg10_xy = pd.merge(phit_wavg10, xy_middle10, on='well')
# Calculation net- & phit average values based on margin 50m (+/- 25m)
zone_size = 50
net_maps_bal8_towell = map_prop_accum(phit_wavg8_xy, net_maps_bal8, 'X','Y', 'NTG', zone_size)
phit_maps_bal8_towell = map_prop_accum(phit_wavg8_xy, phit_maps_bal8, 'X','Y', 'phit_map', zone_size)
net_maps_bal10_towell = map_prop_accum(phit_wavg10_xy, net_maps_bal10, 'X','Y', 'NTG', zone_size)
phit_maps_bal10_towell = map_prop_accum(phit_wavg10_xy, phit_maps_bal10, 'X','Y', 'phit_map', zone_size)
# Calculation weithed on NTG phit average values
phit_map_bal8 = map_phit_ntg_process(phit_maps_bal8_towell, net_maps_bal8_towell, phit_wavg8_xy, 1)
phit_map_bal10 = map_phit_ntg_process(phit_maps_bal10_towell, net_maps_bal10_towell, phit_wavg10_xy, 1)
phit_map = pd.concat([phit_map_bal8, phit_map_bal10]).drop(['X','Y'], axis=1)
phit_map_metrcis = phit_map_metrics(phit_map,'phit_wavg_map', 0.05, 0)
phit_map_final = phit_map[['well', 'FORMATION_up', 'phit_wavg_map', 'qc']]

In [ ]:
# net_maps_bal8_towell['FORMATION_up'] = 'Balakhany VIII'
# net_maps_bal8_towell.groupby(['well','FORMATION_up'])['NTG'].mean().reset_index()

In [ ]:
xplot_qc_map(phit_map,'phit_wavg_map', 0.05, 0.3, 0)
phit_map_metrcis

#### Prediction with phit_map KHtst by RFR

In [ ]:
avg_prop_tst_kh_cut0.columns

In [ ]:
ntd_net2_kh = avg_prop_tst_kh_cut0.copy(deep=True)
ntd_net2_kh = ntd_net2_kh[[ 'well',
                            'FORMATION_up',
                            'X', 'Y',
                            'TVD_SCS',
                            'htst_sum',
                            'tst_interv',
                            'KHtst',
                            ]]
ntd_net2_kh_phitmap_corr, ntd_net2_kh_phitmap = join_add_df_prediction(ntd_net2_kh, phit_map_final.drop('qc', axis=1), 'KHtst')
print(ntd_net2_kh_phitmap.columns)
pairplot_special(ntd_net2_kh_phitmap_corr, 9, 9, 0)

In [ ]:
gs_set = {  'bootstrap': [True],
            'max_depth': [30, 50, 100, 200],
            'min_samples_leaf': [1, 3, 5],
            'min_samples_split': [1, 3, 5],
            'n_estimators': [50, 75, 100, 150]}
scorer = make_scorer(r2, greater_is_better=True)
hyperdict_kh_map = run_rfr_train_test_split(   ntd_net2_kh_phitmap, gs_set, scorer, 
                                                'KHtst', 0.25, 300, 'pred_kh_phmap.txt', 'rfr_grid_search_phmap', 0, 0)

In [ ]:
pred_kh_map = run_rfr_1_to_all( ntd_net2_kh_phitmap, hyperdict_kh_map, 
                                'KHtst', 0.25, 300, 'pred_kh_phmap.txt', 'rfr_1-to-all', 1, 27000, 1)

## Analize of MPL

In [ ]:
test_1df_3offset_wells_fin.head(3)

In [ ]:
def keras_model_creation(n_cols):
    model = keras.Sequential()
    model.add(keras.layers.Dense(32,  activation='tanh', input_shape=(n_cols,)))
    model.add(keras.layers.Dropout(0.1))
    model.add(keras.layers.Dense(16,  activation='tanh'))
    model.add(keras.layers.Dropout(0.1))
    model.add(keras.layers.Dense(1, activation='tanh'))
    optimizer = keras.optimizers.Adam(learning_rate=0.001)
    model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])
    return model
def mlp_train_test_split(train_dataset, target, eph, btchs, rng, margin):
    train_dataset_list = []
    grids_setting_list = []
    metrics_dict = []
    # X_train/x_test data splitting
    scaler = MinMaxScaler()
    y = np.array(train_dataset[['well','FORMATION_up',target]])
    x = np.array(train_dataset.drop(target, axis=1))
    train_dataset_list.append(train_dataset.drop(['FORMATION_up', target], axis=1).columns[1:].values.tolist())
    x_train_init, x_test_init, y_train_init, y_test_init = train_test_split(x, y, test_size=0.1, random_state=42)
    y_train_wells = y_train_init[:,0:2]
    y_test_wells = y_test_init[:,0:2]
    x_train = x_train_init[:,2:]
    x_train_norm = scaler.fit_transform(x_train)
    x_train_tf = tf.convert_to_tensor(x_train_norm, dtype=tf.float32)
    x_test = x_test_init[:,2:]
    x_test_norm = scaler.fit_transform(x_test)
    x_test_tf = tf.convert_to_tensor(x_test_norm, dtype=tf.float32)
    y_train = y_train_init[:,2]
    y_train_rs = y_train.reshape(-1, 1)
    y_train_norm = scaler.fit_transform(y_train_rs)
    y_train_tf = tf.convert_to_tensor(y_train_norm, dtype=tf.float32)
    y_test = y_test_init[:,2]
    y_test_rs = y_test.reshape(-1, 1)
    y_test_norm = scaler.fit_transform(y_test_rs)
    y_test_tf = tf.convert_to_tensor(y_test_norm, dtype=tf.float32)
    # build model
    n_cols = x_train_tf.shape[1]
    model = keras_model_creation(n_cols)
    # train model and perform validation test
    history = model.fit(x_train_tf, y_train_tf, epochs=eph, batch_size=btchs, validation_split=0.1, shuffle='True', verbose=0)
    y_pred_train = model.predict(x_train_tf)
    y_pred_test = model.predict(x_test_tf)
    # Reporting
    y_pred_train_real = scaler.inverse_transform(y_pred_train)
    y_pred_test_real = scaler.inverse_transform(y_pred_test)
    y_train_real =  scaler.inverse_transform(y_train_norm)
    y_test_real =  scaler.inverse_transform(y_test_norm)
    res_mlp_test = pd.DataFrame(zip(y_test_wells[:, 0], 
                                    y_test_wells[:, 1],  
                                    y_test_real.flatten(),
                                    y_pred_test_real.flatten(),), columns=['well', 'FORMATION_up','actual','predict'])
    res_mlp_test['dataset'] = 'test'
    res_mlp_train = pd.DataFrame(zip(   y_train_wells[:, 0], 
                                        y_train_wells[:, 1],  
                                        y_train_real.flatten(),
                                        y_pred_train_real.flatten(),), columns=['well', 'FORMATION_up','actual','predict'])
    res_mlp_train['dataset'] = 'train'
    res_mlp = pd.concat([res_mlp_test, res_mlp_train])
    up_range = rng + 1
    dwn_range = 1 - rng
    res_mlp['l_limit'] = res_mlp.actual*dwn_range - margin
    res_mlp['h_limit'] = res_mlp.actual*up_range + margin
    res_mlp['qc'] = 'out'
    res_mlp.loc[(res_mlp.predict >= res_mlp.l_limit) & (res_mlp.predict <= res_mlp.h_limit), 'qc'] = 'in'
    res_mlp['diff'] = (res_mlp.actual - res_mlp.predict).round(0)
    res_mlp['predict'] =res_mlp.predict.astype('float')
    metrics_dict = {    'train_in':     res_mlp[res_mlp.dataset=='train'].value_counts('qc', normalize=True),
                        'test_in':      res_mlp[res_mlp.dataset=='test'].value_counts('qc', normalize=True)}
    plt.figure(figsize=(6, 4))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
    return {'train_ds':     train_dataset_list[0],
            'history':      history, 
            'metrics':      metrics_dict, 
            'result_df':    res_mlp}
phit_res_mlp_func = mlp_train_test_split(test_1df_3offset_wells_fin, 'phit_wavg', 500, 10, 0.05, 0)
print('train_in ', 'out:', phit_res_mlp_func['metrics']['train_in']['out'].round(2), 'in:', phit_res_mlp_func['metrics']['train_in']['in'].round(2))
print('test_in ', 'out:', phit_res_mlp_func['metrics']['test_in']['out'].round(2), 'in:', phit_res_mlp_func['metrics']['test_in']['in'].round(2))
xplot_qc2(phit_res_mlp_func['result_df'], 0.3, 0.05, 0, 3)

In [ ]:
kh_phit_w123_mlp = ntd_net2_kh_phit_w123_gm.copy()
kh_phit_w123_mlp.head(3)

In [ ]:
def keras_model_creation(n_cols):
    model = keras.Sequential()
    model.add(keras.layers.Dense(32,  activation='tanh', input_shape=(n_cols,)))
    model.add(keras.layers.Dropout(0.1))
    model.add(keras.layers.Dense(16,  activation='tanh'))
    model.add(keras.layers.Dropout(0.1))
    model.add(keras.layers.Dense(1, activation='tanh'))
    optimizer = keras.optimizers.Adam(learning_rate=0.001)
    model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])
    return model
def mlp_train_test_split(train_dataset, target, eph, btchs, rng, margin):
    train_dataset_list = []
    grids_setting_list = []
    metrics_dict = []
    # X_train/x_test data splitting
    scaler = MinMaxScaler()
    y = np.array(train_dataset[['well','FORMATION_up',target]])
    x = np.array(train_dataset.drop(target, axis=1))
    train_dataset_list.append(train_dataset.drop(['FORMATION_up', target], axis=1).columns[1:].values.tolist())
    x_train_init, x_test_init, y_train_init, y_test_init = train_test_split(x, y, test_size=0.1, random_state=42)
    y_train_wells = y_train_init[:,0:2]
    y_test_wells = y_test_init[:,0:2]
    x_train = x_train_init[:,2:]
    x_train_norm = scaler.fit_transform(x_train)
    x_train_tf = tf.convert_to_tensor(x_train_norm, dtype=tf.float32)
    x_test = x_test_init[:,2:]
    x_test_norm = scaler.fit_transform(x_test)
    x_test_tf = tf.convert_to_tensor(x_test_norm, dtype=tf.float32)
    y_train = y_train_init[:,2]
    y_train_rs = y_train.reshape(-1, 1)
    y_train_norm = scaler.fit_transform(y_train_rs)
    y_train_tf = tf.convert_to_tensor(y_train_norm, dtype=tf.float32)
    y_test = y_test_init[:,2]
    y_test_rs = y_test.reshape(-1, 1)
    y_test_norm = scaler.fit_transform(y_test_rs)
    y_test_tf = tf.convert_to_tensor(y_test_norm, dtype=tf.float32)
    # build model
    n_cols = x_train_tf.shape[1]
    model = keras_model_creation(n_cols)
    # train model and perform validation test
    history = model.fit(x_train_tf, y_train_tf, epochs=eph, batch_size=btchs, validation_split=0.1, shuffle='True', verbose=0)
    y_pred_train = model.predict(x_train_tf)
    y_pred_test = model.predict(x_test_tf)
    # Reporting
    y_pred_train_real = scaler.inverse_transform(y_pred_train)
    y_pred_test_real = scaler.inverse_transform(y_pred_test)
    y_train_real =  scaler.inverse_transform(y_train_norm)
    y_test_real =  scaler.inverse_transform(y_test_norm)
    res_mlp_test = pd.DataFrame(zip(y_test_wells[:, 0], 
                                    y_test_wells[:, 1],  
                                    y_test_real.flatten(),
                                    y_pred_test_real.flatten(),), columns=['well', 'FORMATION_up','actual','predict'])
    res_mlp_test['dataset'] = 'test'
    res_mlp_train = pd.DataFrame(zip(   y_train_wells[:, 0], 
                                        y_train_wells[:, 1],  
                                        y_train_real.flatten(),
                                        y_pred_train_real.flatten(),), columns=['well', 'FORMATION_up','actual','predict'])
    res_mlp_train['dataset'] = 'train'
    res_mlp = pd.concat([res_mlp_test, res_mlp_train])
    up_range = rng + 1
    dwn_range = 1 - rng
    res_mlp['l_limit'] = res_mlp.actual*dwn_range - margin
    res_mlp['h_limit'] = res_mlp.actual*up_range + margin
    res_mlp['qc'] = 'out'
    res_mlp.loc[(res_mlp.predict >= res_mlp.l_limit) & (res_mlp.predict <= res_mlp.h_limit), 'qc'] = 'in'
    res_mlp['diff'] = (res_mlp.actual - res_mlp.predict).round(0)
    res_mlp['predict'] =res_mlp.predict.astype('float')
    metrics_dict = {    'train_in':     res_mlp[res_mlp.dataset=='train'].value_counts('qc', normalize=True),
                        'test_in':      res_mlp[res_mlp.dataset=='test'].value_counts('qc', normalize=True)}
    plt.figure(figsize=(6, 4))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
    return {'train_ds':     train_dataset_list[0],
            'history':      history, 
            'metrics':      metrics_dict, 
            'result_df':    res_mlp}
res_mlp_func = mlp_train_test_split(kh_phit_w123_mlp, 'KHtst', 1000, 10, 0.25, 0)
print('train_in ', 'out:', res_mlp_func['metrics']['train_in']['out'].round(2), 'in:', res_mlp_func['metrics']['train_in']['in'].round(2))
print('test_in ', 'out:', res_mlp_func['metrics']['test_in']['out'].round(2), 'in:', res_mlp_func['metrics']['test_in']['in'].round(2))
xplot_qc2(res_mlp_func['result_df'], 27000, 0.25, 0, 0)

## CNN preparation

### Data maps & aux functions

In [ ]:
# Drawing maps of well trajectories
def well_traj_dataprep(dataset):
    map_data = dataset[['well','FORMATION_up','FORMATION','X','Y','X_traj','Y_traj', 'Status']]
    map_data_top = map_data.groupby(['well','FORMATION_up'])[['X_traj','Y_traj']].apply(lambda x: x.iloc[0:-100:100]).reset_index()
    map_data_bot = map_data.groupby(['well','FORMATION_up'])[['X_traj','Y_traj']].apply(lambda x: x.iloc[-1]).reset_index()
    map_data_middle = map_data.groupby(['well','FORMATION_up'])[['X','Y', 'Status']].apply(lambda x: x.iloc[0]).reset_index()
    map_data_display = pd.concat([map_data_top, map_data_bot]).sort_values(by=['well','FORMATION_up']).drop('level_2', axis=1)
    return map_data_display, map_data_middle
map_data_display, map_data_middle = well_traj_dataprep(df_bal_net2_kh)
def display_well_traj(map_data_display, map_data_middle, fmname):
    map_data_display = map_data_display[map_data_display.FORMATION_up == fmname]
    map_data_middle = map_data_middle[map_data_middle.FORMATION_up == fmname]
    traj = go.Scatter(  x=map_data_display.X_traj, y=map_data_display.Y_traj, 
                        mode='markers',
                        marker=dict(color='black', size=3),
                        customdata = map_data_display[['well']],
                        hovertemplate="".join(["well:%{customdata[0]}<extra></extra>"]))
    wells = go.Scatter( x=map_data_middle.X, y=map_data_middle.Y, 
                        mode='markers',
                        marker=dict(symbol='diamond', color='red', size=7,
                        opacity=0.75, line=dict(color='rgb(47, 57, 61)', width=0.5)),
                        customdata = map_data_middle[['well', 'Status']],
                        hovertemplate="".join(["well:%{customdata[0]},%{customdata[1]}<extra></extra>"]))
    fig = go.Figure()
    fig.add_trace(traj)
    fig.add_trace(wells)
    fig.update_layout(  title_text= ('map of traj and well points'+ ' ' + fmname),
                        autosize=True, width=1300, height=600, margin=dict(l=10,r=10,b=10,t=50), showlegend=False)
    return fig.show()
display_well_traj(map_data_display, map_data_middle, 'Balakhany VIII')
display_well_traj(map_data_display, map_data_middle, 'Balakhany X')

In [643]:
def tst_smpl_final(dataset, cut):
    def tst_smpl_processing(dataset):
        test_tst_smpl = dataset.copy()
        test_tst_smpl = test_tst_smpl[~test_tst_smpl.TST_smpl.isna()]

        test_tst_smpl_v2 = test_tst_smpl.groupby(['well','FORMATION_up'])[['MD','TST_smpl']].apply(lambda x: x.iloc[0]).reset_index()
        test_tst_smpl_v2['TST_smpl_remove'] = 1
        test_tst_smpl_v3 = test_tst_smpl.set_index(['well', 'FORMATION_up','MD']).join(
                            test_tst_smpl_v2.set_index(['well', 'FORMATION_up','MD']), rsuffix='_r').reset_index()
        test_tst_smpl_v3 = test_tst_smpl_v3[test_tst_smpl_v3.TST_smpl_remove != 1]
        result = test_tst_smpl_v3.drop(['TST_smpl_r','TST_smpl_remove'], axis=1)
        return result
    def tst_smpl_cut(dataset_cut, cut):
        cc=0
        ind_lst = []
        well_sel = dataset_cut.reset_index().drop('index',axis=1)
        for ind, row in well_sel.iterrows():
            cc += row['TST_smpl']
            if cc > cut:
                ind_lst.append(ind)
                cc=0
        df_well_fm_lst = []
        well_sel['tst_rank'] = 0
        well_sel.iloc[:ind_lst[0], well_sel.columns.get_loc('tst_rank')] = 0
        df_well_fm_lst.append(well_sel.iloc[:ind_lst[0]])
        for i in range(len(ind_lst[:-1])):
            well_sel.iloc[ind_lst[i]:ind_lst[i+1], well_sel.columns.get_loc('tst_rank')] = i+1
            df_well_fm_lst.append(well_sel.iloc[ind_lst[i]:ind_lst[i+1]])
        df_well = pd.concat(df_well_fm_lst)
        df_well_v2 = df_well.join(df_well.groupby(['well','tst_rank'])['TST_smpl'].cumsum(), rsuffix='_r')
        df_well_v2 = df_well_v2.rename(columns={'TST_smpl_r':'TST_smpl_cumsum'})
        result = df_well_v2[[   'well', 'MD', 'FORMATION_up','PHIT','KHtst','X_traj','Y_traj','TVD_SCS',
                                'TST_smpl', 'TST_smpl_cumsum', 'tst_rank']]
        return result
    print('Processing dataset with tst_smpl_processing')
    tst_smpl_proc = tst_smpl_processing(dataset)
    df_lst = []
    for fm in ['Balakhany VIII', 'Balakhany X']:
        print('Processing wells with tst_smpl_cut for ' + fm)
        for wellname in tqdm(tst_smpl_proc.well.unique()):
            try:
                well_tst_cut = tst_smpl_proc[(tst_smpl_proc.well==wellname) & (tst_smpl_proc.FORMATION_up == fm)]
                res_well_tst_cut = tst_smpl_cut(well_tst_cut, cut)
                df_lst.append(res_well_tst_cut)
            except:
                pass
    df_final = pd.concat(df_lst)
    df_final['MD'] = df_final['MD'].round(1)
    return df_final

def tst_resampling_v2(dataset_resample, wellname, fmname, resampled_points):
    df1 = dataset_resample[ (dataset_resample.well == wellname) & 
                            (dataset_resample.FORMATION_up==fmname)].groupby(
                            ['well','tst_rank'])['TST_smpl'].count().reset_index()
    df1 = df1.rename(columns={'TST_smpl':'tst_smpl_qnt'})
    df2 = dataset_resample[ (dataset_resample.well == wellname) & 
                            (dataset_resample.FORMATION_up==fmname)].set_index(['well','tst_rank']).join(
                            df1.set_index(['well','tst_rank'])).reset_index()
    df2['window'] = (df2.tst_smpl_qnt/resampled_points).round(0).astype('int')
    df_lst = []
    for i in df2.tst_rank.unique()[:]:
        dataset = df2[df2.tst_rank==i]
        window = dataset.window.iloc[0]
        md_data = []
        for ind in range(0, len(dataset['MD']), window):
            lst = dataset['MD'].iloc[ind:ind + window]
            central_value = [len(lst) // 2]
            md_data.append(lst.iloc[central_value].round(1).values[0])
        ph_data = [dataset['PHIT'][ind:ind+window].mean().round(3) for ind in range(0, len(dataset['PHIT']), window)]
        ind_data = [ind for ind in range(0, len(dataset['MD']), window)]
        tst_rank = [i for ind in range(0, len(dataset['MD']), window)]
        df_lst.append(pd.DataFrame(zip( md_data[:resampled_points], 
                                        tst_rank[:resampled_points], 
                                        ph_data[:resampled_points]), columns=['MD','tst_rank_aux','PHIT_avg']))
    df3 = pd.concat(df_lst)
    df3['well'] = wellname
    df2['MD'] = df2.MD.round(1)
    result_resample = df2.set_index(['well','MD']).join(df3.set_index(['well','MD'])).reset_index()
    return result_resample

def tst_resampling_v2_per_fm(well_list, data_df, fmname, samples_per_block):
    print('Run calculation tst_resampling_v2 for ' + fmname)
    well_bal = well_list.well.unique()
    df_lst = []
    for well in tqdm(well_bal):
        df_lst.append(tst_resampling_v2(data_df, well, fmname, samples_per_block))
    result = pd.concat(df_lst)
    return result

def check_tst_resampling(dataset_resampl, resampled_points, show_print=1):
    check_tst_smpl_qty = dataset_resampl.groupby(['well','FORMATION_up','tst_rank'])['PHIT_avg'].count().reset_index()
    check_tst_smpl_qty = check_tst_smpl_qty.rename(columns={'PHIT_avg':'PHIT_avg_qty_smpls'})
    check_tst_smpl_qty_noneq = check_tst_smpl_qty[check_tst_smpl_qty.PHIT_avg_qty_smpls != resampled_points]
    check_tst_smpl_qty_eq = check_tst_smpl_qty[check_tst_smpl_qty.PHIT_avg_qty_smpls == resampled_points]
    if show_print == 1:
        print('qty_eq = ',len(check_tst_smpl_qty_eq.well.unique()))
    else:
        pass
    qty_noneq = check_tst_smpl_qty_noneq.groupby(['well','FORMATION_up'])['PHIT_avg_qty_smpls'].count().reset_index()
    qty_noneq = qty_noneq[qty_noneq.PHIT_avg_qty_smpls > 0]
    if show_print == 1:
        print('qty_noneq', len(qty_noneq))
    else:
        pass
    qty_noneq = qty_noneq.sort_values(by='PHIT_avg_qty_smpls', ascending=False)
    result = {  'qty_noneq': qty_noneq,
                'total_ds': check_tst_smpl_qty}
    return result

def display_tst_resampling(dataset, wellname, fmname):
    plt.figure(figsize=(3, 10))
    dataset = dataset[(dataset.well == wellname) & (dataset.FORMATION_up==fmname)]
    plt.plot(dataset['PHIT'], dataset['MD'], label='PHIT', color='green', linestyle='--', zorder=1)
    plt.scatter(dataset['PHIT_avg'], dataset['MD'], label='PHIT_avg', color='red', s=14, zorder=2, alpha=0.75);
    plt.gca().invert_yaxis()
    plt.gca().invert_xaxis()
    plt.title('MD ' + wellname + ' ' + fmname)
    plt.grid()
    plt.legend()
    return plt.show()

def tst_resampling_sft(dataset_sft, wellname, fmname):
        df_well_resmpl = dataset_sft[(dataset_sft.well == wellname) & (dataset_sft.FORMATION_up == fmname)]
        md_data = []
        tst_rank_sft = []
        for i in df_well_resmpl.tst_rank.unique()[:]:
                dataset = df_well_resmpl[df_well_resmpl.tst_rank==i]
                central_value = [len(dataset) // 2]
                md_data.append(dataset['MD'].iloc[central_value].round(1).values[0])
                tst_rank_sft.append(dataset['tst_rank'].iloc[central_value].values[0])
        r_avg_s = pd.DataFrame(zip(md_data, tst_rank_sft), columns=['MD','tst_rank_sft'])
        df_final_well_final = df_well_resmpl.set_index('MD').join(r_avg_s.set_index('MD')).reset_index()
        df_final_well_final['tst_rank_sft'] = df_final_well_final['tst_rank_sft'].fillna(method='ffill')
        return df_final_well_final

def tst_resampling_sft_per_fm(dataset_sft, fmname):
        print('Run shift calculation for ' + fmname)
        df_lst_sft = []
        for well in tqdm(dataset_sft.well.unique()):
                df = tst_resampling_sft(dataset_sft, well, fmname)
                df_lst_sft.append(df)
        result = pd.concat(df_lst_sft)
        return result

def display_tst_shifting(dataset, wellname, fmname, rnk):
    plt.figure(figsize=(3, 7))
    dataset1 = dataset[(dataset.well == wellname) & (dataset.FORMATION_up==fmname) & (dataset.tst_rank==rnk)]
    dataset2 = dataset[(dataset.well == wellname) & (dataset.FORMATION_up==fmname) & (dataset.tst_rank==rnk + 1)]
    dataset_sft = dataset[(dataset.well == wellname) & (dataset.FORMATION_up==fmname) & (dataset.tst_rank_sft == rnk)]
    plt.plot(dataset1['PHIT'], dataset1['MD'], label='PHIT', color='green', linestyle='--', zorder=1)
    plt.plot(dataset2['PHIT'], dataset2['MD'], label='PHIT', color='lightgreen', linestyle='--', zorder=1)
    plt.plot(dataset_sft['PHIT']+0.05, dataset_sft['MD'], label='PHIT', color='blue', linestyle='--', zorder=1)
    plt.scatter(dataset1['PHIT_avg'], dataset1['MD'], label='PHIT_avg', color='darkred', s=18, zorder=2, alpha=0.75)
    plt.scatter(dataset2['PHIT_avg'], dataset2['MD'], label='PHIT_avg', color='red', s=18, zorder=2, alpha=0.75)
    plt.scatter(dataset1[dataset1['PHIT_avg'].notna()]['PHIT'], 
                dataset1[dataset1['PHIT_avg'].notna()]['MD'], label='PHIT', color='blue', s=14, zorder=2, alpha=0.5)
    plt.scatter(dataset2[dataset2['PHIT_avg'].notna()]['PHIT'], 
                dataset2[dataset2['PHIT_avg'].notna()]['MD'], label='PHIT', color='blue', s=14, zorder=2, alpha=0.5)
    plt.gca().invert_yaxis()
    plt.gca().invert_xaxis()
    plt.title('MD ' + wellname + ' ' + fmname)
    plt.grid()
    plt.legend()
    return plt.show()

def tst_resampling_numrow(dataset):
    df_lst = []
    print('Assigning row numbers into each tst_rank for', dataset.FORMATION_up.iloc[0])
    desired_list = ['well', 'FORMATION_up', 'TST_smpl_cumsum', 'tst_rank', 'tst_rank_sft', 'PHIT', 'PHIT_avg', 'X_traj', 'Y_traj', 'TVD_SCS']
    for wellname in tqdm(dataset.well.unique()):
        sample = dataset[(dataset.well == wellname)]
        for rankname in sample.tst_rank.unique():  
            sample_1 = sample[(sample.tst_rank == rankname) & (sample.PHIT_avg.notna())]
            sample_2 = sample_1[desired_list].reset_index()
            sample_2['index'] = 1
            sample_2['num_row'] = sample_2['index'].cumsum()
            sample_3 = sample_2.drop('index', axis=1)
            df_lst.append(sample_3)
    result = pd.concat(df_lst)
    return result

def offset_wells_list(dataset, wellname, fmname):
    """
    overview offset wells based on input dataset with well dist
    """
    offset_well_lst = dataset[['well','FORMATION_up','well1', 'well2', 'well3']]
    offset_well_lst =offset_well_lst[offset_well_lst.FORMATION_up == fmname]
    result = offset_well_lst[offset_well_lst.well==wellname]
    return result

def target_features_merging(dataset_dist, dataset_data, fmname, offset, dist):
    grid_offsets1 = dataset_dist[['well','FORMATION_up', offset]]
    grid_offsets1 =grid_offsets1[grid_offsets1.FORMATION_up == fmname]
    grid_offsets1 = grid_offsets1[['well', offset]]

    grid_offsets1_v2 = grid_offsets1.set_index('well').join(dataset_data.set_index('well')).reset_index()
    grid_offsets1_v2 = grid_offsets1_v2.add_suffix('_trgt',axis=1)
    offet_trgt = offset + '_trgt'
    grid_offsets1_v2 = grid_offsets1_v2.rename(columns = { offet_trgt:'well', 'tst_rank_trgt':'tst_rank', 'num_row_trgt':'num_row'})

    df_bal8_sft_num_ftr1 = dataset_data.add_suffix('_ftr', axis=1)
    df_bal8_sft_num_ftr1 = df_bal8_sft_num_ftr1.rename(columns = {'well_ftr':'well', 'tst_rank_ftr':'tst_rank', 'num_row_ftr':'num_row'})

    grid_offsets1_v3 =  grid_offsets1_v2.set_index(['well','tst_rank','num_row']).join(
                        df_bal8_sft_num_ftr1.set_index(['well','tst_rank','num_row'])).reset_index()
    grid_offsets1_v3 = grid_offsets1_v3.rename(columns = {'well':'well_ftr', 'tst_rank':'tst_rank_ftr', 'num_row':'num_row_ftr'})
    grid_offsets1_v3['offset'] = dist
    grid_offsets1_v3_fin = grid_offsets1_v3[[  'well_trgt','FORMATION_up_trgt','TST_smpl_cumsum_trgt', 'PHIT_avg_trgt','tst_rank_sft_trgt','TVD_SCS_trgt', 'X_traj_trgt','Y_traj_trgt', 
                                                'well_ftr', 'tst_rank_ftr', 'num_row_ftr','TST_smpl_cumsum_ftr', 'PHIT_avg_ftr','tst_rank_sft_ftr',
                                                'TVD_SCS_ftr', 'X_traj_ftr','Y_traj_ftr','offset']]
    result = grid_offsets1_v3_fin.sort_values(by=['well_trgt','tst_rank_ftr','num_row_ftr'])
    return result

def check_samples_quantity_per_well_fm(dataset, samples_per_block):
    """
    Checking how many samples per block for well & formation.
    Dataset have to contain tst_rank, well and formation names
    """
    groupby_res = dataset.groupby(['well_trgt','FORMATION_up_trgt'])['tst_index_ftr'].count().reset_index()
    problems_well_list = groupby_res[groupby_res.tst_index_ftr % samples_per_block != 0]
    return problems_well_list

def cartesian_to_cylindrical(center, point):
    """
    tut center eto tocko s offseta , a point eto tocka targeta. Mi brali tocku offseta kak nacalo koordinat dla cilindriceskoy sistemi
    """
    x0, y0, z0 = center
    x, y, z = point
    rho = math.sqrt((x - x0)**2 + (y - y0)**2)
    theta = math.atan2(y - y0, x - x0)
    z_cylindrical = z - z0
    return rho, theta, z_cylindrical

### Data interpolation, blocking and augmentation

In [ ]:
def interpolate_by_depth_fm(dataset_logs, formation_name, step):
    def interpolate_by_depth(one_well, formation_name, step):
        one_well = one_well.sort_values(by='TST')
        well_name = one_well["well"].iloc[0]
        data_range = np.floor((one_well["TST"].max() - one_well["TST"].min())/step)
        starting_tst = one_well["TST"].iloc[0]
        new_TST_values = [starting_tst + i*0.1 for i in range(1,int(data_range))]
        interp_X = interp1d(one_well['TST'], one_well['X_traj'], kind='linear', fill_value="extrapolate")
        interp_Y = interp1d(one_well['TST'], one_well['Y_traj'], kind='linear', fill_value="extrapolate")
        interp_PHIT = interp1d(one_well['TST'], one_well['PHIT'], kind='linear', fill_value="extrapolate")
        interp_TVD = interp1d(one_well['TST'], one_well['TVD_SCS'], kind='linear', fill_value="extrapolate")
        interp_NET_clp2 = interp1d(one_well['TST'], one_well['NET_clp2'], kind='linear', fill_value="extrapolate")
        interp_LPERM = interp1d(one_well['TST'], one_well['LPERM'], kind='linear', fill_value="extrapolate")
        interp_KHtst = interp1d(one_well['TST'], one_well['KHtst'], kind='linear', fill_value="extrapolate")
        # Create a new DataFrame with the interpolated values for new TVD_SCS
        new_data = {
            'well': [well_name for _ in range(len(new_TST_values))],
            'FORMATION_up': [formation_name for _ in range(len(new_TST_values))],
            'tst_index': [_ for _ in range(len(new_TST_values))],
            'TST': new_TST_values,
            'X_traj': interp_X(new_TST_values),
            'Y_traj': interp_Y(new_TST_values),
            'PHIT': interp_PHIT(new_TST_values),
            'TVD_SCS': interp_TVD(new_TST_values),
            'NET_clp2': interp_NET_clp2(new_TST_values),
            'LPERM': interp_LPERM(new_TST_values),
            'KHtst': interp_KHtst(new_TST_values),
        }
        new_df = pd.DataFrame(new_data)
        return new_df
    df_lst = []
    print(f'Start interpolation of {formation_name}')
    for wellnames in tqdm(dataset_logs.well.unique()):
        well_sel = dataset_logs[dataset_logs.well == wellnames]
        well_interp = interpolate_by_depth(well_sel, formation_name, step)
        df_lst.append(well_interp)
    result = pd.concat(df_lst)
    return result
well_bal8 = df_bal_net2_kh[(df_bal_net2_kh.FORMATION_up == 'Balakhany VIII')]
well_bal10 = df_bal_net2_kh[(df_bal_net2_kh.FORMATION_up == 'Balakhany X')]
well_bal8_interp = interpolate_by_depth_fm(well_bal8, 'Balakhany VIII', 0.1)
well_bal10_interp = interpolate_by_depth_fm(well_bal10, 'Balakhany X', 0.1)

In [ ]:
well_bal8_interp_rn = well_bal8_interp.rename(columns={'PHIT':'PHIT_orig'})
well_bal10_interp_rn = well_bal10_interp.rename(columns={'PHIT':'PHIT_orig'})

def phit_rolling_averaging(input_dataset, samples_per_window):
    df_lst = []
    avg_report = []
    fmname = input_dataset['FORMATION_up'].iloc[0] 
    print(f'Start rolling averaging of {fmname}')
    for wellname in tqdm(input_dataset.well.unique()):
        dataset = input_dataset[input_dataset.well == wellname]
        window_size = int(len(dataset) / samples_per_window)
        dataset['PHIT'] = dataset['PHIT_orig'].rolling(window=window_size, center=True).mean()
        dataset =  dataset.dropna(subset=['PHIT'])
        df_lst.append(dataset)
        avg_report.append((wellname, len(dataset), window_size, samples_per_window))
    result = pd.concat(df_lst)
    avg_report_df = pd.DataFrame(avg_report, columns=['well','lenght_ds','window_size','samples_per_window'])
    return result, avg_report_df
samples_per_window = 100
well_bal8_interp_phavg, avg_report_df8 = phit_rolling_averaging(well_bal8_interp_rn, samples_per_window)
well_bal10_interp_phavg, avg_report_df10 = phit_rolling_averaging(well_bal10_interp_rn, samples_per_window)
well_bal8_interp_phavg['PHIT_clp'] = well_bal8_interp_phavg['PHIT']
well_bal10_interp_phavg['PHIT_clp'] = well_bal10_interp_phavg['PHIT']
well_bal8_interp_phavg['LPERM_clp'] = well_bal8_interp_phavg['LPERM']
well_bal10_interp_phavg['LPERM_clp'] = well_bal10_interp_phavg['LPERM']
well_bal8_interp_phavg.loc[well_bal8_interp_phavg.NET_clp2 == 0, 'PHIT_clp'] = 0.12
well_bal10_interp_phavg.loc[well_bal10_interp_phavg.NET_clp2 == 0, 'PHIT_clp'] = 0.12
well_bal8_interp_phavg.loc[well_bal8_interp_phavg.NET_clp2 == 0, 'LPERM_clp'] = 0.1
well_bal10_interp_phavg.loc[well_bal10_interp_phavg.NET_clp2 == 0, 'LPERM_clp'] = 0.1

In [ ]:
def cutting_block_lenght(dataset, block_lenght):
    df_lst = []
    fmname = dataset['FORMATION_up'].iloc[0]
    print(f'Start processing of dataset for {fmname} with block lenght {block_lenght}')
    for wellname in tqdm(dataset.well.unique()):
        data = dataset[dataset.well == wellname]
        tst_index_repaired = [i for i in range(0, len(data))]
        data['tst_index'] = tst_index_repaired
        new_index = [i for i in range(0, len(data), block_lenght)]
        data_cut = data[(data.tst_index < new_index[-1])]
        df_lst.append(data_cut)
    result = pd.concat(df_lst)
    return result
block_lenght = 50
well_bal8_interp_phavg_cut = cutting_block_lenght(well_bal8_interp_phavg, block_lenght)
well_bal10_interp_phavg_cut = cutting_block_lenght(well_bal10_interp_phavg, block_lenght)

In [2505]:
def target_features_merging_v2(dataset_dist, dataset_data, fmname, offset, dist):
    target_offset_well_name = dataset_dist[['well','FORMATION_up', offset]]
    target_offset_well_name =target_offset_well_name[target_offset_well_name.FORMATION_up == fmname]
    target_offset_well_name = target_offset_well_name[['well', offset]]

    well_target = target_offset_well_name.set_index('well').join(dataset_data.set_index('well')).reset_index()
    well_target_v2 = well_target.add_suffix('_trgt',axis=1)
    offset_well = offset + '_trgt'
    well_target_v2 = well_target_v2.rename(columns = { offset_well:'well','FORMATION_up_trgt':'FORMATION_up','tst_index_trgt':'tst_index'})

    feature_well = dataset_data.add_suffix('_ftr', axis=1)
    feature_well = feature_well.rename(columns = {'well_ftr':'well', 'FORMATION_up_ftr':'FORMATION_up','tst_index_ftr':'tst_index'})

    final_df =  well_target_v2.set_index(['well','FORMATION_up','tst_index']).join(feature_well.set_index(['well','FORMATION_up','tst_index'])).reset_index()
    # final_df = final_df.rename(columns = {'well':'well_ftr', 'FORMATION_up':'FORMATION_up_ftr', 'tst_index':'tst_index_ftr'})
    final_df['offset'] = dist

    final_df = final_df[[  'well_trgt','PHIT_trgt', 'PHIT_clp_trgt','TST_trgt', 'TVD_SCS_trgt', 'X_traj_trgt','Y_traj_trgt', 
                            'well', 'FORMATION_up','tst_index', 'PHIT_ftr', 'PHIT_clp_ftr', 'TST_ftr', 'TVD_SCS_ftr', 'X_traj_ftr','Y_traj_ftr','offset']]
    result = final_df.sort_values(by=['well_trgt','well','FORMATION_up','tst_index'])
    return result
well_interp_bal8_phit_cut_w1 = target_features_merging_v2(avg_prop_tst_kh_cut0, well_bal8_interp_phavg_cut, 'Balakhany VIII', 'well1', 1)
well_interp_bal8_phit_cut_w2 = target_features_merging_v2(avg_prop_tst_kh_cut0, well_bal8_interp_phavg_cut, 'Balakhany VIII', 'well2', 2)
well_interp_bal8_phit_cut_w3 = target_features_merging_v2(avg_prop_tst_kh_cut0, well_bal8_interp_phavg_cut, 'Balakhany VIII', 'well3', 3)

well_interp_bal10_phit_cut_w1 = target_features_merging_v2(avg_prop_tst_kh_cut0, well_bal10_interp_phavg_cut, 'Balakhany X', 'well1', 1)
well_interp_bal10_phit_cut_w2 = target_features_merging_v2(avg_prop_tst_kh_cut0, well_bal10_interp_phavg_cut, 'Balakhany X', 'well2', 2)
well_interp_bal10_phit_cut_w3 = target_features_merging_v2(avg_prop_tst_kh_cut0, well_bal10_interp_phavg_cut, 'Balakhany X', 'well3', 3)

final_phit_bal810 = pd.concat([  well_interp_bal8_phit_cut_w1, well_interp_bal8_phit_cut_w2, well_interp_bal8_phit_cut_w3,
                                    well_interp_bal10_phit_cut_w1, well_interp_bal10_phit_cut_w2, well_interp_bal10_phit_cut_w3])
final_phit_bal810['rho_ftr'] = np.sqrt((final_phit_bal810.X_traj_trgt-final_phit_bal810.X_traj_ftr)**2 + (final_phit_bal810.Y_traj_trgt-final_phit_bal810.Y_traj_ftr)**2)
final_phit_bal810['theta_ftr'] = np.arctan2(final_phit_bal810.Y_traj_trgt-final_phit_bal810.Y_traj_ftr, final_phit_bal810.X_traj_trgt-final_phit_bal810.X_traj_ftr)
final_phit_bal810['theta_deg_ftr'] = np.degrees(final_phit_bal810['theta_ftr'])
final_phit_bal810['z_ftr'] = final_phit_bal810.TVD_SCS_trgt - final_phit_bal810.TVD_SCS_ftr

In [ ]:
def augmentation_target_offsets_wells(dataset, block_shift, block_lenght):
    df_lst = []
    fmname = dataset['FORMATION_up'].iloc[0]
    print(f'Calculation augmentation data for {fmname}') 
    for i in tqdm(dataset.offset.unique()):
        well_pair_list = dataset[(dataset.offset == i)][['well_trgt','well']].groupby(
                        'well_trgt')['well'].apply(lambda x: x.iloc[0]).reset_index()
        for _, row in well_pair_list.iterrows():
            well_pair_selected = dataset[(dataset.well_trgt == row['well_trgt']) & (dataset.well == row['well'])]
            well_pair_selected['tst_index_sft'] = well_pair_selected['tst_index'] - block_shift
            tst_index_sft_positiv = well_pair_selected[well_pair_selected.tst_index_sft >= 0]
            new_index = [i for i in range(0, len(tst_index_sft_positiv), block_lenght)]
            well_pair_tst_index_sft = well_pair_selected[(well_pair_selected['tst_index_sft'] >= 0) & (well_pair_selected['tst_index_sft'] < new_index[-1])]
            df_lst.append(well_pair_tst_index_sft)
    result = pd.concat(df_lst)
    return result
final_phit_bal8 = final_phit_bal810[final_phit_bal810.FORMATION_up == 'Balakhany VIII']
final_phit_bal10 = final_phit_bal810[final_phit_bal810.FORMATION_up == 'Balakhany X']
block_shift = block_lenght / 2
aug_bal8 = augmentation_target_offsets_wells(final_phit_bal8, block_shift, block_lenght)
aug_bal10 = augmentation_target_offsets_wells(final_phit_bal10, block_shift, block_lenght)

In [ ]:
def phit_rollavg_plot(dataset_raw, dataset_rollavg, dataset_interp, welllist, path):
    fig, ax = plt.subplots(1,4, figsize=(8, 10))
    def plot_data(dataset, wellname, column, color, lw, comment, suffix):
        dataset = dataset[(dataset.well == wellname)]
        depth = dataset['TST']
        phit = dataset['PHIT']
        ax[column].plot(phit, depth, label='PHIT' + suffix, color=color, lw=lw, zorder=2, alpha=1)
        ax[column].invert_yaxis()
        ax[column].set_xlim(.3, 0)
        ax[column].set_title(wellname + comment)
        ax[column].legend(fontsize=7)
        ax[column].grid()
    plot_data(dataset_raw,      welllist[3], 0, 'lightgreen', 4, '', '_raw')
    plot_data(dataset_interp,   welllist[3], 0, 'blue', 1, '', '_int')
    plot_data(dataset_rollavg,  welllist[3], 0, 'red', 1, '', '_avg')

    plot_data(dataset_raw,      welllist[2], 1, 'lightgreen', 4, '', '_raw')
    plot_data(dataset_interp,   welllist[2], 1, 'blue', 1, '', '_int')
    plot_data(dataset_rollavg,  welllist[2], 1, 'red', 1, '', '_avg')

    plot_data(dataset_raw,      welllist[1], 2, 'lightgreen', 4, '', '_raw')
    plot_data(dataset_interp,   welllist[1], 2, 'blue', 1, '', '_int')
    plot_data(dataset_rollavg,  welllist[1], 2, 'red', 1, '', '_avg')
    
    plot_data(dataset_raw,      welllist[0], 3, 'lightgreen', 4, '', '_raw')
    plot_data(dataset_interp,   welllist[0], 3, 'blue', 1, '', '_int')
    plot_data(dataset_rollavg,  welllist[0], 3, 'red', 1, ' target', '_avg')
    fig.savefig( path + welllist[0] + '.pdf', format='pdf')

dataset_raw = df_bal_net2_kh[df_bal_net2_kh.FORMATION_up == 'Balakhany VIII']
dataset_interp = well_bal8_interp
dataset_rollavg = well_bal8_interp_phavg
offset_wells8 = avg_prop_tst_kh_cut0[avg_prop_tst_kh_cut0.FORMATION_up == 'Balakhany VIII']
path8 = 'C:\\jupyter\\SPP\\test_polygon\\bal8\\'
for ind, row in offset_wells8.iterrows():
   welllist = [row['well'],row['well1'],row['well2'],row['well3']]
   phit_rollavg_plot(dataset_raw, dataset_rollavg, dataset_interp, welllist, path8)

dataset_raw = df_bal_net2_kh[df_bal_net2_kh.FORMATION_up == 'Balakhany X']
dataset_interp = well_bal10_interp
dataset_rollavg = well_bal10_interp_phavg
offset_wells10 = avg_prop_tst_kh_cut0[avg_prop_tst_kh_cut0.FORMATION_up == 'Balakhany X']
path10 = 'C:\\jupyter\\SPP\\test_polygon\\bal10\\'
for ind, row in offset_wells10.iterrows():
   welllist = [row['well'],row['well1'],row['well2'],row['well3']]
   phit_rollavg_plot(dataset_raw, dataset_rollavg, dataset_interp, welllist, path10)

In [649]:
# block_lenght_m = 10
# tst_rank_final = tst_smpl_final(df_bal_net2_kh, block_lenght_m)

# samples_per_block = 10
# tst_rank_bal8 = tst_rank_final[tst_rank_final.FORMATION_up == 'Balakhany VIII']
# tst_rank_bal10 = tst_rank_final[tst_rank_final.FORMATION_up == 'Balakhany X']

# df_bal8 = tst_resampling_v2_per_fm(tst_rank_bal8, tst_rank_final, 'Balakhany VIII', samples_per_block)
# df_bal10 = tst_resampling_v2_per_fm(tst_rank_bal10, tst_rank_final, 'Balakhany X', samples_per_block)

# df_bal8_sft = tst_resampling_sft_per_fm(df_bal8, 'Balakhany VIII')
# df_bal10_sft = tst_resampling_sft_per_fm(df_bal10, 'Balakhany X')

# df_bal8_sft_num = tst_resampling_numrow(df_bal8_sft)
# df_bal10_sft_num = tst_resampling_numrow(df_bal10_sft)

# grid_offsets8_v3_fin1 = target_features_merging(avg_prop_tst_kh_cut0, df_bal8_sft_num, 'Balakhany VIII', 'well1', 1)
# grid_offsets8_v3_fin2 = target_features_merging(avg_prop_tst_kh_cut0, df_bal8_sft_num,'Balakhany VIII', 'well2', 2)
# grid_offsets8_v3_fin3 = target_features_merging(avg_prop_tst_kh_cut0, df_bal8_sft_num, 'Balakhany VIII', 'well3', 3)
# grid_offsets10_v3_fin1 = target_features_merging(avg_prop_tst_kh_cut0, df_bal10_sft_num, 'Balakhany X', 'well1', 1)
# grid_offsets10_v3_fin2 = target_features_merging(avg_prop_tst_kh_cut0, df_bal10_sft_num,'Balakhany X', 'well2', 2)
# grid_offsets10_v3_fin3 = target_features_merging(avg_prop_tst_kh_cut0, df_bal10_sft_num, 'Balakhany X', 'well3', 3)
# grid_offsets = pd.concat([  grid_offsets8_v3_fin1, grid_offsets8_v3_fin2, grid_offsets8_v3_fin3,
#                             grid_offsets10_v3_fin1, grid_offsets10_v3_fin2, grid_offsets10_v3_fin3])
# grid_offsets['rho_ftr'] = np.sqrt((grid_offsets.X_traj_trgt-grid_offsets.X_traj_ftr)**2 + (grid_offsets.Y_traj_trgt-grid_offsets.Y_traj_ftr)**2)
# grid_offsets['theta_ftr'] = np.arctan2(grid_offsets.Y_traj_trgt-grid_offsets.Y_traj_ftr, grid_offsets.X_traj_trgt-grid_offsets.X_traj_ftr)
# grid_offsets['theta_deg_ftr'] = np.degrees(grid_offsets['theta_ftr'])
# grid_offsets['z_ftr'] = grid_offsets.TVD_SCS_trgt - grid_offsets.TVD_SCS_ftr
# grid_offsets_orig = grid_offsets.copy()
# check_samples_quantity_per_well_fm(grid_offsets_orig, samples_per_block)

# def looking_for_problems_rank(dataset, wellname, fmname, flag=1):
#     """
#     If falg = 1 function return dataframe with samples per tst_rank_ftr
#     If flag = 0 function return number of ranks for removing

#     """
#     count_mode = dataset[       (dataset.well_trgt == wellname) & 
#                                 (dataset.FORMATION_up_trgt == fmname)]['tst_rank_sft_trgt'].value_counts().mode()
#     count_result = dataset[ (dataset.well_trgt == wellname) & 
#                             (dataset.FORMATION_up_trgt == fmname)][['well_trgt','FORMATION_up_trgt','tst_rank_ftr']].value_counts().reset_index().sort_values(by='tst_rank_ftr')
#     rank_to_remove = count_result[count_result['count'] != count_mode[0]]['tst_rank_ftr'].values
#     index_remove = dataset[     ((dataset.well_trgt == wellname) & 
#                                 (dataset.FORMATION_up_trgt == fmname) & 
#                                 (dataset.tst_rank_ftr.isin(rank_to_remove)))].index
#     if len(index_remove) == 0:
#         print(f'Check whole welldata {wellname} - {fmname}')
#     elif len(index_remove) > 150:
#         print(f'Too many rows (> 100) for removing {wellname} - {fmname} - {len(index_remove)}')
#     else:
#         pass
#     if flag == 1:
#         result = count_result
#     if flag == 0:
#         result = rank_to_remove
#     return result
# # lst_rank_1 = looking_for_problems_rank(grid_offsets, 'CHIRAG6', 'Balakhany X', 0)
# # lst_rank_2 = looking_for_problems_rank(grid_offsets, 'D01', 'Balakhany VIII', 0)
# # lst_rank_3 = looking_for_problems_rank(grid_offsets, 'D01', 'Balakhany X', 0)
# # # lst_rank_4 = looking_for_problems_rank(grid_offsets, 'E01Z', 'Balakhany VIII', 0)
# # lst_rank_5 = looking_for_problems_rank(grid_offsets, 'E01Z', 'Balakhany X', 0)
# # # lst_rank_6 = looking_for_problems_rank(grid_offsets, 'E05Z', 'Balakhany X', 0)
# # # lst_rank_7 = looking_for_problems_rank(grid_offsets, 'GCA6', 'Balakhany X', 0)
# lst_rank_1 = looking_for_problems_rank(grid_offsets, 'E05Z', 'Balakhany VIII', 0)
# lst_rank_2 = looking_for_problems_rank(grid_offsets, 'E05Z', 'Balakhany X', 0)

# grid_offsets_orig = grid_offsets_orig[  ~((grid_offsets_orig.well_trgt == 'E05Z') & 
#                                         (grid_offsets_orig.FORMATION_up_trgt == 'Balakhany VIII'))]
# grid_offsets_orig = grid_offsets_orig[  ~((grid_offsets_orig.well_trgt == 'E05Z') & 
#                                         (grid_offsets_orig.FORMATION_up_trgt == 'Balakhany X') & 
#                                         (grid_offsets_orig.tst_rank_ftr.isin(lst_rank_2)))]
# # grid_offsets_orig = grid_offsets_orig[  ~((grid_offsets_orig.well_trgt == 'CHIRAG6') & 
# #                                         (grid_offsets_orig.FORMATION_up_trgt == 'Balakhany X') & 
# #                                         (grid_offsets_orig.tst_rank_ftr.isin(lst_rank_1)))]
# # grid_offsets_orig = grid_offsets_orig[  ~((grid_offsets_orig.well_trgt == 'D01') & 
# #                                         (grid_offsets_orig.FORMATION_up_trgt == 'Balakhany VIII') &
# #                                         (grid_offsets_orig.tst_rank_ftr.isin(lst_rank_2)))]
# # grid_offsets_orig = grid_offsets_orig[  ~((grid_offsets_orig.well_trgt == 'D01') & 
# #                                         (grid_offsets_orig.FORMATION_up_trgt == 'Balakhany X') &
# #                                         (grid_offsets_orig.tst_rank_ftr.isin(lst_rank_3)))]
# # grid_offsets_orig = grid_offsets_orig[  ~((grid_offsets_orig.well_trgt == 'E01Z') & 
# #                                         (grid_offsets_orig.FORMATION_up_trgt == 'Balakhany VIII'))]
# # grid_offsets_orig = grid_offsets_orig[  ~((grid_offsets_orig.well_trgt == 'E01Z') & 
# #                                         (grid_offsets_orig.FORMATION_up_trgt == 'Balakhany X') &
# #                                         (grid_offsets_orig.tst_rank_ftr.isin(lst_rank_5)))]
# # grid_offsets_orig = grid_offsets_orig[  ~((grid_offsets_orig.well_trgt == 'E05Z') & 
# #                                         (grid_offsets_orig.FORMATION_up_trgt == 'Balakhany X'))]
# # grid_offsets_orig = grid_offsets_orig[  ~((grid_offsets_orig.well_trgt == 'GCA6') & 
# #                                         (grid_offsets_orig.FORMATION_up_trgt == 'Balakhany X'))]
# check_samples_quantity_per_well_fm(grid_offsets_orig, samples_per_block)

In [ ]:
def display_features_target_martix(dataset_input, wellname, fmname, var):
    try:
        fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(5, 3))
        dataset = dataset_input[(dataset_input.well_trgt == wellname) & (dataset_input.FORMATION_up == fmname)]
        phit_ftr = var + '_ftr'
        phit_trgt = var + '_trgt'
        
        ftr_well1 = dataset[(dataset.offset == 1)]['well'].iloc[0]
        ftr_well2 = dataset[(dataset.offset == 2)]['well'].iloc[0]
        ftr_well3 = dataset[(dataset.offset == 3)]['well'].iloc[0]
        
        feature_x1 = dataset[(dataset.offset == 1)][phit_ftr]
        ind1 = [i for i in range(len(feature_x1))]
        feature_x2 = dataset[(dataset.offset == 2)][phit_ftr]
        ind2 = [i for i in range(len(feature_x2))]
        feature_x3 = dataset[(dataset.offset == 3)][phit_ftr]
        ind3 = [i for i in range(len(feature_x3))]

        target_y = dataset[(dataset.offset == 1)][phit_trgt]
        target_ind = dataset[(dataset.offset == 1)][phit_trgt]
        ind = [i for i in range(len(target_ind))]

        ax1.plot(target_y, ind, label='PHIT trgt', color='red', lw=2, zorder=2, alpha=0.5)
        ax2.plot(feature_x1, ind1, label='PHIT ftr3', color='green', lw=2, zorder=2, alpha=0.5)
        ax3.plot(feature_x2, ind2, label='PHIT ftr2', color='green', lw=2, zorder=2, alpha=0.5)
        ax4.plot(feature_x3, ind3, label='PHIT ftr1', color='green', lw=2, zorder=2, alpha=0.5)

        ax1.invert_yaxis(), ax2.invert_yaxis(), ax3.invert_yaxis(), ax4.invert_yaxis()
        ax1.set_xlim(0.1, 0.3), ax2.set_xlim(0.1, 0.3), ax3.set_xlim(0.1, 0.3), ax4.set_xlim(0.1, 0.3)       
        ax1.set_title(wellname, fontsize=8),  ax2.set_title(ftr_well1, fontsize=8), ax3.set_title(ftr_well2, fontsize=8), ax4.set_title(ftr_well3, fontsize=8)
        ax1.grid(), ax2.grid(), ax3.grid(), ax4.grid()
        ax1.tick_params(axis='both', labelsize=8), ax2.tick_params(axis='both', labelsize=8), ax3.tick_params(axis='both', labelsize=8), ax4.tick_params(axis='both', labelsize=8)
        fig.tight_layout()
        fig.show()
    except Exception as e:
        print(f'Its look like the desired formation is absent, error is "{e}"')
# display_features_target_martix(final_phit_bal8, 'A01W','Balakhany VIII', 'PHIT_clp')
        
for well_matrix in final_phit_bal8.well_trgt.unique():
    display_features_target_martix(final_phit_bal8, well_matrix,'Balakhany VIII', 'PHIT_clp')

In [ ]:
def display_features_target(dataset_input, wellname, fmname, var):
    try:
        fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(6, 7))
        dataset = dataset_input[(dataset_input.well_trgt == wellname) & (dataset_input.FORMATION_up == fmname)]
        phit_ftr = var + '_ftr'
        phit_trgt = var + '_trgt'
        ftr_well1 = dataset[(dataset.offset == 1)]['well'].iloc[0]
        ftr_well2 = dataset[(dataset.offset == 2)]['well'].iloc[0]
        ftr_well3 = dataset[(dataset.offset == 3)]['well'].iloc[0]
        rho_well1 = dataset[(dataset.offset == 1)]['rho_ftr'].mean()
        rho_well2 = dataset[(dataset.offset == 2)]['rho_ftr'].mean()
        rho_well3 = dataset[(dataset.offset == 3)]['rho_ftr'].mean()

        feature_x1 = dataset[(dataset.offset == 1)][phit_ftr]
        median1 = feature_x1.median()
        feature_x1 = dataset[(dataset.offset == 1)][phit_ftr].fillna(0)
        feature_x2 = dataset[(dataset.offset == 2)][phit_ftr]
        median2 = feature_x2.median()
        feature_x2 = dataset[(dataset.offset == 2)][phit_ftr].fillna(0)
        feature_x3 = dataset[(dataset.offset == 3)][phit_ftr]
        median3 = feature_x3.median()
        feature_x3 = dataset[(dataset.offset == 3)][phit_ftr].fillna(0)

        target_y = dataset[(dataset.offset == 1)][phit_trgt]
        target_ind = dataset[(dataset.offset == 1)][phit_trgt]
        ind = [i for i in range(len(target_ind))]
        cutoff = [0.13 for i in range(len(target_ind))]

        ax1.plot(feature_x3, ind, label='PHIT ftr1', color='green', lw=2, zorder=2, alpha=0.5)
        ax1.plot(cutoff, ind, lw=2, color='black', linestyle = '--', label='phit_cutoff', alpha=0.66)
        ax2.plot(feature_x2, ind, label='PHIT ftr2', color='green', lw=2, zorder=2, alpha=0.5)
        ax2.plot(cutoff, ind, lw=2, color='black', linestyle = '--', label='phit_cutoff', alpha=0.66)
        ax3.plot(feature_x1, ind, label='PHIT ftr3', color='green', lw=2, zorder=2, alpha=0.5)
        ax3.plot(cutoff, ind, lw=2, color='black', linestyle = '--', label='phit_cutoff', alpha=0.66)
        ax4.plot(target_y, ind, label='PHIT trgt', color='red', lw=2, zorder=2, alpha=0.5)
        ax4.plot(cutoff, ind, lw=2, color='black', linestyle = '--', label='phit_cutoff', alpha=0.66)

        ax1.invert_yaxis(), ax2.invert_yaxis(), ax3.invert_yaxis(), ax4.invert_yaxis()
        ax1.set_xlim(.3, 0), ax2.set_xlim(.3, 0), ax3.set_xlim(.3, 0), ax4.set_xlim(.3, 0)
        ax1.set_title('F_3 ' + str(int(rho_well3)) + ' ' + ftr_well3, fontsize=10)
        ax2.set_title('F_2 ' + str(int(rho_well2)) + ' ' + ftr_well2, fontsize=10)
        ax3.set_title('F_1 ' + str(int(rho_well1)) + ' ' + ftr_well1, fontsize=10)
        ax4.set_title('Target ' + wellname, fontsize=10)
        fig.suptitle('Target  ' + wellname + ' ' + fmname)
        ax1.grid(), ax2.grid(), ax3.grid(), ax4.grid()
        ax1.legend(fontsize=8), ax2.legend(fontsize=8), ax3.legend(fontsize=8), ax4.legend(fontsize=8)
        fig.show()
    except Exception as e:
        print(f'Its look like the desired formation is absent, error is "{e}"')
def display_frts_target_polars(dataset, wellname, fmname):
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(6, 7))
    ftr_well1 = dataset[(dataset.well_trgt == wellname) & (dataset.offset == 1)]['well_ftr'].iloc[0]
    # ftr_well2 = dataset[(dataset.well_trgt == wellname) & (dataset.offset == 2)]['well_ftr'].iloc[0]
    # ftr_well3 = dataset[(dataset.well_trgt == wellname) & (dataset.offset == 3)]['well_ftr'].iloc[0]

    feature_x1 = dataset[(dataset.well_trgt == wellname) & (dataset.offset == 1)]['rho_ftr']
    median1 = feature_x1.median()
    feature_x1 = dataset[(dataset.well_trgt == wellname) & (dataset.offset == 1)]['rho_ftr'].fillna(median1)
    feature_x2 = dataset[(dataset.well_trgt == wellname) & (dataset.offset == 1)]['theta_deg_ftr']
    median2 = feature_x2.median()
    feature_x2 = dataset[(dataset.well_trgt == wellname) & (dataset.offset == 1)]['theta_deg_ftr'].fillna(median2)
    feature_x3 = dataset[(dataset.well_trgt == wellname) & (dataset.offset == 1)]['z_ftr']
    median3 = feature_x3.median()
    feature_x3 = dataset[(dataset.well_trgt == wellname) & (dataset.offset == 1)]['z_ftr'].fillna(median3)

    target_y = dataset[(dataset.well_trgt == wellname) & (dataset.offset == 1)]['PHIT_avg_trgt']
    target_ind = dataset[(dataset.well_trgt == wellname) & (dataset.offset == 1)]['PHIT_avg_trgt']
    ind = [i for i in range(len(target_ind))]

    ax1.scatter(feature_x1, ind, label='rho_ftr', color='brown', s=14, zorder=2, alpha=0.5)
    ax2.scatter(feature_x2, ind, label='theta_ftr', color='purple', s=14, zorder=2, alpha=0.5)
    ax3.scatter(feature_x3, ind, label='z_ftr3', color='orange', s=14, zorder=2, alpha=0.5)
    ax4.scatter(target_y, ind, label='PHIT trgt', color='red', s=14, zorder=2, alpha=0.5)

    ax1.invert_yaxis(), ax2.invert_yaxis(), ax3.invert_yaxis(), ax4.invert_yaxis()
    # ax1.set_xlim(0, 0.3), ax2.set_xlim(0, 0.3), ax3.set_xlim(0, 0.3), 
    ax4.set_xlim(0, 0.3)
    ax1.set_title('rho_ftr ' + ftr_well1, fontsize=10)
    ax2.set_title('theta_ftr ' + ftr_well1, fontsize=10)
    ax3.set_title('z_ftr ' + ftr_well1, fontsize=10)
    ax4.set_title('Target ' + wellname, fontsize=10)
    fig.suptitle('Target  ' + wellname + ' ' + fmname)
    ax1.grid(), ax2.grid(), ax3.grid(), ax4.grid()
    ax1.legend(fontsize=8), ax2.legend(fontsize=8), ax3.legend(fontsize=8), ax4.legend(fontsize=8)
    fig.show();
display_features_target(final_phit_bal8, 'B16Y','Balakhany VIII', 'PHIT_clp')
# display_frts_target_polars(grid_offsets, wellname,'Balakhany X')

### 1D AENC run 3f1t+aug

In [ ]:
scaler = MinMaxScaler()
block_size = 50

final_phit_bal810_run = final_phit_bal810.copy()
final_phit_bal810_run = final_phit_bal810_run[['well_trgt', 'PHIT_trgt', 'PHIT_clp_trgt', 'TST_trgt','TVD_SCS_trgt', 
                                               'well', 'FORMATION_up', 'tst_index', 'PHIT_ftr', 'PHIT_clp_ftr', 'TST_ftr', 'TVD_SCS_ftr', 'offset']]
final_phit_bal810_run = final_phit_bal810_run.sort_values(by=['well_trgt','FORMATION_up','tst_index'])
aug_bal8 = aug_bal8.sort_values(by=['well_trgt','FORMATION_up','tst_index_sft'])
final_phit_bal8_run = final_phit_bal810_run[(final_phit_bal810_run.FORMATION_up == 'Balakhany VIII')]

def train_test_wells_selection(dataset, test_well_qty, rerun):
    if rerun == 0:
        print('Well list is not randomized')
    if rerun == 1:
        well_list = list(dataset.well.unique())
        test_well_list = []
        for _ in range(test_well_qty):
            well = well_list[np.random.randint(1, len(well_list))]
            test_well_list.append(well)
        train_well_list = [x for x in well_list if x not in test_well_list]
    return train_well_list, test_well_list
train_well_list, test_well_list = train_test_wells_selection(final_phit_bal8_run, 3, 1)
print(test_well_list)

def train_wells_preparation(dataset_main, dataset_aug, train_well_list, block_size, var):
    train_dataset = dataset_main[dataset_main.well_trgt.isin(train_well_list)]
    aug_dataset = dataset_aug[dataset_aug.well_trgt.isin(train_well_list)]
    var_ftr = var + '_ftr'
    var_trgt = var + '_trgt'
    train_dataset['PHIT_ftr%'] = train_dataset[var_ftr]*100
    train_dataset['PHIT_trgt%'] = train_dataset[var_trgt]*100
    aug_dataset['PHIT_ftr%'] = aug_dataset[var_ftr]*100
    aug_dataset['PHIT_trgt%'] = aug_dataset[var_trgt]*100

    df_x1_ph = train_dataset[(train_dataset.offset==1)]['PHIT_ftr%'].fillna(0).values
    aug_x1 = aug_dataset[(aug_dataset.offset==1)]['PHIT_ftr%'].fillna(0).values

    df_x2_ph = train_dataset[(train_dataset.offset==2)]['PHIT_ftr%'].fillna(0).values
    aug_x2 = aug_dataset[(aug_dataset.offset==2)]['PHIT_ftr%'].fillna(0).values

    df_x3_ph = train_dataset[(train_dataset.offset==3)]['PHIT_ftr%'].fillna(0).values
    aug_x3 = aug_dataset[(aug_dataset.offset==3)]['PHIT_ftr%'].fillna(0).values

    df_ph = np.stack((df_x1_ph, df_x2_ph, df_x3_ph), axis=1)
    aug = np.stack((aug_x1, aug_x2, aug_x3), axis=1)
    df_ph_final = np.concatenate((df_ph, aug))
    # df_ph_norm = scaler.fit_transform(df_ph_final)
    X_train = df_ph_final.reshape(-1, block_size, 1, 3)
    
    y_well = train_dataset[     (train_dataset.offset==1)]['well_trgt'].reset_index().drop('index', axis=1).values
    y_fm = train_dataset[       (train_dataset.offset==1)]['FORMATION_up'].reset_index().drop('index', axis=1).values
    y_data = train_dataset[     (train_dataset.offset==1)]['PHIT_trgt%'].reset_index().drop('index', axis=1).values
    y_well_aug = aug_dataset[     (aug_dataset.offset==1)]['well_trgt'].reset_index().drop('index', axis=1).values
    y_fm_aug = aug_dataset[       (aug_dataset.offset==1)]['FORMATION_up'].reset_index().drop('index', axis=1).values
    y_data_aug = aug_dataset[     (aug_dataset.offset==1)]['PHIT_trgt%'].reset_index().drop('index', axis=1).values    

    # y_data = scaler.fit_transform(y_data)
    # y_data_aug = scaler.fit_transform(y_data_aug)
    y_final = np.stack((y_well, y_fm, y_data), axis=1)
    y_final_aug = np.stack((y_well_aug, y_fm_aug, y_data_aug), axis=1)
    y_final_v2 = np.concatenate((y_final, y_final_aug))
    y_reshape = y_final_v2.reshape(-1, block_size, 1, 3)
    y_train = y_reshape[:,:,:,2].reshape(-1, block_size, 1)
    y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
    return X_train, y_train
X_train, y_train = train_wells_preparation(final_phit_bal8_run, aug_bal8, train_well_list, block_size,'PHIT')
print(X_train.shape, y_train.shape)

def test_wells_preparation(dataset_main, test_well_list, block_size, var):
    test_dataset = dataset_main[dataset_main.well_trgt.isin(test_well_list)]
    var_ftr = var + '_ftr'
    var_trgt = var + '_trgt'
    test_dataset['PHIT_ftr%'] = test_dataset[var_ftr]*100
    test_dataset['PHIT_trgt%'] = test_dataset[var_trgt]*100

    df_x1_ph = test_dataset[(test_dataset.offset==1)]['PHIT_ftr%'].fillna(0).values
    df_x2_ph = test_dataset[(test_dataset.offset==2)]['PHIT_ftr%'].fillna(0).values
    df_x3_ph = test_dataset[(test_dataset.offset==3)]['PHIT_ftr%'].fillna(0).values

    df_ph = np.stack((df_x1_ph, df_x2_ph, df_x3_ph), axis=1)
    # df_ph_norm = scaler.fit_transform(df_ph)
    X_test = df_ph.reshape(-1, block_size, 1, 3)

    y_well = test_dataset[     (test_dataset.offset==1)]['well_trgt'].reset_index().drop('index', axis=1).values
    y_fm = test_dataset[       (test_dataset.offset==1)]['FORMATION_up'].reset_index().drop('index', axis=1).values
    y_data = test_dataset[     (test_dataset.offset==1)]['PHIT_trgt%'].reset_index().drop('index', axis=1).values                  
    # y_data = scaler.fit_transform(y_data)
    y_final = np.stack((y_well, y_fm, y_data), axis=1)
    y_reshape = y_final.reshape(-1, block_size, 1, 3)
    y_test = y_reshape[:,:,:,2].reshape(-1, block_size, 1)
    y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)
    return X_test, y_test, y_final
X_test, y_test, y_test_fullds = test_wells_preparation(final_phit_bal8_run, test_well_list, block_size,'PHIT')
print(X_test.shape, y_test.shape, y_test_fullds.shape)

In [ ]:
# # Running RandomForestRegressor as the baseline for CNN
# X_rfr = X_train.reshape(-1, 3)
# y_rfr = y_train.numpy().reshape(-1, 1)
# X_train_rfr, X_test_rfr, y_train_rfr, y_test_rfr = train_test_split(X_rfr, y_rfr, test_size=0.3, random_state=42, shuffle=False)
# rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
# rf_regressor.fit(X_train_rfr, y_train_rfr)
# predictions = rf_regressor.predict(X_test_rfr)
# mae = mean_absolute_error(y_test_rfr, predictions)
# print("mean_absolute_error RandomForestRegressor:", mae)

In [ ]:
## CNN models cementery
# model_cnn1 = Sequential([
#     Conv1D(128, 3, activation='relu',  data_format="channels_last", padding='same', input_shape =(10,1,3)),
#     MaxPooling1D(2, padding='same'),
#     Dropout(0.25),
#     Conv1D(64, 3, activation='relu', padding='same'),
#     MaxPooling1D(2, padding='same'),
#     Dropout(0.25),
#     Conv1D(32, 3, activation='relu', padding='same'),
#     Flatten(),
#     Dense(32, activation='relu'),
#     Dense(10)  
# ])
# model_cnn1 = Adam(learning_rate=0.001)

# model_cnn = Sequential([
#     Conv2D(128, (5,1), activation='relu',  data_format="channels_last", padding='valid', input_shape =(block_size,1,3)),
#     MaxPooling2D((3,1), padding='valid'),
#     Dropout(0.125),
#     Conv2D(64, (3,1), activation='relu', padding='valid',),
#     MaxPooling2D((2,1), padding='valid'),
#     Dropout(0.25),
#     Conv2D(32, (2,1), activation='relu', padding='valid',),
#     MaxPooling2D((3,1), padding='valid'),
#     Flatten(),
#     Dense(256, activation='relu'),
#     Dense(block_size)  
# ])
# opt_cnn = Adam(learning_rate=0.01)

In [ ]:
model_aenc = Sequential([
    Conv2D(filters=256, kernel_size=(3,3), strides=1, activation='relu', padding='same', 
           kernel_initializer = RandomUniform(-1,1), input_shape =(block_size, 1, 3)),
    Conv2D(filters=32, kernel_size=(3,3), strides=2, activation = 'relu', padding='same'),
    Conv2DTranspose(filters=32, kernel_size=(1,1), strides=1, activation='relu', padding='same'),
    Conv2DTranspose(filters=256, kernel_size=(3,3), strides=2, activation = 'relu', padding='same'),
    Conv2DTranspose(filters=1, kernel_size=(3,3), strides=1, activation="relu", padding='same'),
    Flatten(),
    Dense(block_size)
    ])
opt_aenc = Adam(learning_rate=0.001)
# opt_aenc = tfa.optimizers.Lookahead(opt_aenc)
model_aenc.summary()

def run_model(X_train, y_train, X_test, y_test, model, optmzr, lss, eph, btch_sz, erl_stp, vrbs):
    model.compile(optimizer=optmzr, loss=lss)
    early_stop = EarlyStopping(monitor='loss', patience=10, verbose=1, restore_best_weights=True)
    if erl_stp == 1:
        history = model.fit(X_train, y_train, epochs=eph, batch_size=btch_sz, verbose=vrbs, validation_data=(X_test, y_test),callbacks=[early_stop])
    if erl_stp == 0:
        history = model.fit(X_train, y_train, epochs=eph, batch_size=btch_sz, verbose=vrbs, validation_data=(X_test, y_test))  
    y_pred = model.predict(X_test)
    plt.figure(figsize=(4, 3))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    return y_pred
y_pred = run_model(X_train, y_train, X_test, y_test, model_aenc, opt_aenc, 'mse', 200, 50, 1, 0)

In [ ]:
def model_data_proc(X_test, y_test_init, y_test,  y_pred):
    X_test1_data = np.array(X_test[:,:,:,0]).flatten()
    X_test2_data = np.array(X_test[:,:,:,1]).flatten()
    X_test3_data = np.array(X_test[:,:,:,2]).flatten()
    y_test_data = np.array(y_test[:,:,0]).flatten()
    y_test_wellname = y_test_init[:,0].flatten()
    y_test_formation = y_test_init[:,1].flatten()
    y_pred_data = y_pred.flatten()
    y_test_full = np.stack((y_test_wellname, y_test_formation, y_test_data, y_pred_data, 
                            X_test1_data, X_test2_data, X_test3_data), axis=1)
    result = pd.DataFrame(y_test_full, columns=['well_trgt','FORMATION_up_trgt','PHIT_avg_trgt%', 'PHIT_avg_trgt%_pred', 
                                                        'PHIT_avg_ftr%_1','PHIT_avg_ftr%_2','PHIT_avg_ftr%_3'])
    mae = mean_absolute_error(result['PHIT_avg_trgt%'], result['PHIT_avg_trgt%_pred'])
    print("Mean Absolute Error (MAE):", mae)
    return result
def model_data_display(dataset, dataset_offsets, fmname):
    well_test = dataset.well_trgt.unique()
    try:
        fig, ax = plt.subplots(1, 12, figsize=(20, 8))
        def track_display(dataset, well_test, well_num, track_num):
            feature_x1 = dataset[(dataset.well_trgt == well_test[well_num]) & (dataset.FORMATION_up_trgt == fmname)]['PHIT_avg_ftr%_1']
            feature_x2 = dataset[(dataset.well_trgt == well_test[well_num]) & (dataset.FORMATION_up_trgt == fmname)]['PHIT_avg_ftr%_2']
            feature_x3 = dataset[(dataset.well_trgt == well_test[well_num]) & (dataset.FORMATION_up_trgt == fmname) ]['PHIT_avg_ftr%_3']
            target_y3 = dataset[(dataset.well_trgt == well_test[well_num]) & (dataset.FORMATION_up_trgt == fmname)]['PHIT_avg_trgt%']
            prediction_y3 = dataset[(dataset.well_trgt == well_test[well_num]) & (dataset.FORMATION_up_trgt == fmname)]['PHIT_avg_trgt%_pred']
            target_ind3 = dataset[(dataset.well_trgt == well_test[well_num]) & (dataset.FORMATION_up_trgt == fmname)]['PHIT_avg_trgt%']
            ind3 = [i for i in range(len(target_ind3))]
            cutoff3 = [13 for i in range(len(target_ind3))]
            ax[track_num].plot(feature_x3, ind3, label='PHIT ftr3', color='green', lw=2, zorder=2, alpha=0.5)
            ax[track_num].plot(cutoff3, ind3, lw=2, color='black', linestyle = '--', label='phit_cutoff', alpha=0.33)
            ax[track_num].set_title('well3 ' + offset_wells_list(dataset_offsets, well_test[0],'Balakhany VIII')['well3'].iloc[0], fontsize=9)
            ax[track_num].set_yticks(range(min(ind3), max(ind3), 50))
            ax[track_num].grid(axis='y')

            ax[track_num+1].plot(feature_x2, ind3, label='PHIT ftr2', color='green', lw=2, zorder=2, alpha=0.5)
            ax[track_num+1].plot(cutoff3, ind3, lw=2, color='black', linestyle = '--', label='phit_cutoff', alpha=0.33)
            ax[track_num+1].set_title('well2 ' + offset_wells_list(dataset_offsets, well_test[0],'Balakhany VIII')['well2'].iloc[0], fontsize=9)
            ax[track_num+1].set_yticks(range(min(ind3), max(ind3), 50))
            ax[track_num+1].grid(axis='y')

            ax[track_num+2].plot(feature_x1, ind3, label='PHIT ftr1', color='green', lw=2, zorder=2, alpha=0.5)
            ax[track_num+2].plot(cutoff3, ind3, lw=2, color='black', linestyle = '--', label='phit_cutoff', alpha=0.33)
            ax[track_num+2].set_title('well1 ' + offset_wells_list(dataset_offsets, well_test[0],'Balakhany VIII')['well1'].iloc[0], fontsize=9)
            ax[track_num+2].set_yticks(range(min(ind3), max(ind3), 50))
            ax[track_num+2].grid(axis='y')

            ax[track_num+3].plot(target_y3, ind3, label='PHIT trgt', color='red', lw=2, zorder=2, alpha=0.5)
            ax[track_num+3].plot(prediction_y3, ind3, label='PHIT pred', color='blue', lw=2, zorder=2, alpha=0.5)
            ax[track_num+3].plot(cutoff3, ind3, lw=2, color='black', linestyle = '--', label='phit_cutoff', alpha=0.33)
            ax[track_num+3].set_yticks(range(min(ind3), max(ind3), 50))
            ax[track_num+3].grid(axis='y')

            ax[track_num].invert_yaxis(), ax[track_num+1].invert_yaxis(), ax[track_num+2].invert_yaxis(), ax[track_num+3].invert_yaxis()
            ax[track_num].set_xlim(30, 0), ax[track_num+1].set_xlim(30, 0), ax[track_num+2].set_xlim(30, 0), ax[track_num+3].set_xlim(30, 0)
            ax[track_num+3].set_title('Target ' + well_test[0], fontsize=11)
        track_display(dataset, well_test, 0, 0)
        track_display(dataset, well_test, 1, 4)
        track_display(dataset, well_test, 2, 8)
        fig.tight_layout()
    except Exception as e:
        print(f'Probably 1 or several wells are absent due to the error "{e}"')
    return fig.show()

model_full_df = model_data_proc(X_test, y_test_fullds, y_test, y_pred)
model_data_display(model_full_df, avg_prop_tst_kh_cut0, 'Balakhany VIII')

### 1D AENC run 1t1t

In [ ]:
final_phit_bal810_run = final_phit_bal810.copy()
final_phit_bal810_run = final_phit_bal810_run[['well_trgt', 'PHIT_trgt', 'TST_trgt','TVD_SCS_trgt', 
                                               'well', 'FORMATION_up', 'tst_index', 'PHIT_ftr', 'TST_ftr', 'TVD_SCS_ftr', 'offset']]
final_phit_bal810_run = final_phit_bal810_run.sort_values(by=['well_trgt','FORMATION_up','tst_index'])
aug_bal8 = aug_bal8.sort_values(by=['well_trgt','FORMATION_up','tst_index_sft'])
final_phit_bal8_run = final_phit_bal810_run[(final_phit_bal810_run.FORMATION_up == 'Balakhany VIII')]

def self_train_test_wells_selection(dataset, test_well_qty, rerun):
    if rerun == 0:
        print('Well list is not randomized')
    if rerun == 1:
        well_list = list(dataset.well.unique())
        test_well_list = []
        for _ in range(test_well_qty):
            well = well_list[np.random.randint(1, len(well_list))]
            test_well_list.append(well)
        train_well_list = [x for x in well_list if x not in test_well_list]
    return train_well_list, test_well_list
train_well_list_slf, test_well_list_slf = self_train_test_wells_selection(final_phit_bal8_run, 3, 1)
print(test_well_list_slf)

def self_train_wells_preparation(dataset_main, train_well_list, block_size):
    train_dataset = dataset_main[dataset_main.well_trgt.isin(train_well_list)]
    train_dataset['PHIT_trgt%'] = train_dataset['PHIT_trgt']*100

    phit_feature =train_dataset[(train_dataset.offset==1)]['PHIT_trgt%'].fillna(0).values
    phit_feature_norm = scaler.fit_transform(phit_feature.reshape(-1, 1))
    X_train = phit_feature_norm.reshape(-1, block_size, 1)
    
    y_well = train_dataset[     (train_dataset.offset==1)]['well_trgt'].values.reshape(-1, 1)
    y_fm = train_dataset[       (train_dataset.offset==1)]['FORMATION_up'].values.reshape(-1, 1)
    y_data = train_dataset[     (train_dataset.offset==1)]['PHIT_trgt%'].values.reshape(-1, 1)

    y_data = scaler.fit_transform(y_data)
    y_final = np.stack((y_well, y_fm, y_data), axis=1)
    y_reshape = y_final.reshape(-1, block_size, 1, 3)
    y_train = y_reshape[:,:,:,2].reshape(-1, block_size, 1)
    y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
    return X_train, y_train, y_final
X_train_slf, y_train_slf, y_final_slf = self_train_wells_preparation(final_phit_bal8_run, train_well_list_slf, block_size)
print(X_train_slf.shape, y_train_slf.shape)

def self_test_wells_preparation(dataset_main, test_well_list, block_size):
    test_dataset = dataset_main[dataset_main.well_trgt.isin(test_well_list)]
    test_dataset['PHIT_trgt%'] = test_dataset['PHIT_trgt']*100

    phit_feature = test_dataset[(test_dataset.offset==1)]['PHIT_trgt%'].fillna(0).values
    phit_feature_norm = scaler.fit_transform(phit_feature.reshape(-1, 1))
    X_test = phit_feature_norm.reshape(-1, block_size, 1, 1)

    y_well = test_dataset[     (test_dataset.offset==1)]['well_trgt'].reset_index().drop('index', axis=1).values
    y_fm = test_dataset[       (test_dataset.offset==1)]['FORMATION_up'].reset_index().drop('index', axis=1).values
    y_data = test_dataset[     (test_dataset.offset==1)]['PHIT_trgt%'].reset_index().drop('index', axis=1).values   

    y_data = scaler.fit_transform(y_data)
    y_final = np.stack((y_well, y_fm, y_data), axis=1)
    y_reshape = y_final.reshape(-1, block_size, 1, 3)
    y_test = y_reshape[:,:,:,2].reshape(-1, block_size, 1)
    y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)
    return X_test, y_test, y_final
X_test_slf, y_test_slf, y_test_fullds_slf= self_test_wells_preparation(final_phit_bal8_run, test_well_list_slf, block_size)
print(X_test_slf.shape, y_test_slf.shape)

In [ ]:
model_aenc = Sequential([
    Conv2D(filters=256, kernel_size=(3,3), strides=1, activation='relu', padding='same', 
           kernel_initializer = RandomUniform(-1,1), input_shape =(block_size, 1, 1)),
    Conv2D(filters=32, kernel_size=(3,3), strides=2, activation = 'relu', padding='same'),
    Conv2DTranspose(filters=32, kernel_size=(1,1), strides=1, activation='relu', padding='same'),
    Conv2DTranspose(filters=256, kernel_size=(3,3), strides=2, activation = 'relu', padding='same'),
    Conv2DTranspose(filters=1, kernel_size=(3,3), strides=1, activation="relu", padding='same'),
    Flatten(),
    Dense(block_size)
    ])
opt_aenc = Adam(learning_rate=0.001)
model_aenc.summary()

def run_model_self(X_train, y_train, model, optmzr, lss, eph, btch_sz, erl_stp, vrbs):
    model.compile(optimizer=optmzr, loss=lss)
    early_stop = EarlyStopping(monitor='loss', patience=10, verbose=1, restore_best_weights=True)
    if erl_stp == 1:
        history = model.fit(X_train, y_train, epochs=eph, batch_size=btch_sz, verbose=vrbs, callbacks=[early_stop])
    if erl_stp == 0:
        history = model.fit(X_train, y_train, epochs=eph, batch_size=btch_sz, verbose=vrbs)  
    y_pred = model.predict(X_test_slf)
    plt.figure(figsize=(4, 3))
    plt.plot(history.history['loss'], label='Training Loss')
    # plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    return y_pred
y_pred_slf = run_model_self(X_train_slf, y_train_slf, model_aenc, opt_aenc, 'mse', 100, 25, 1, 0)

In [ ]:
## Archiv of old code
        # feature_x4 = dataset[(dataset.well_trgt == well_test[1]) & (dataset.FORMATION_up_trgt == fmname)]['PHIT_avg_ftr%_1']
        # feature_x5 = dataset[(dataset.well_trgt == well_test[1]) & (dataset.FORMATION_up_trgt == fmname)]['PHIT_avg_ftr%_2']
        # feature_x6 = dataset[(dataset.well_trgt == well_test[1]) & (dataset.FORMATION_up_trgt == fmname) ]['PHIT_avg_ftr%_3']
        # target_y2 = dataset[(dataset.well_trgt == well_test[1]) & (dataset.FORMATION_up_trgt == fmname)]['PHIT_avg_trgt%']
        # prediction_y2 = dataset[(dataset.well_trgt == well_test[1]) & (dataset.FORMATION_up_trgt == fmname)]['PHIT_avg_trgt%_pred']
        # target_ind2 = dataset[(dataset.well_trgt == well_test[1]) & (dataset.FORMATION_up_trgt == fmname)]['PHIT_avg_trgt%']
        # ind2 = [i for i in range(len(target_ind2))]
        # cutoff2 = [0.4 for i in range(len(target_ind2))]
        # ax[4].plot(feature_x6, ind2, label='PHIT ftr3', color='green', lw=2, zorder=2, alpha=0.5)
        # ax[4].plot(cutoff2, ind2, lw=2, color='black', linestyle = '--', label='phit_cutoff', alpha=0.33)
        # ax[4].set_title('well3 ' + offset_wells_list(dataset_offsets, well_test[1],'Balakhany VIII')['well3'].iloc[0], fontsize=9)
        # ax[4].set_yticks(range(min(ind2), max(ind2)+1, 50))
        # ax[4].grid(axis='y')

        # ax[5].plot(feature_x5, ind2, label='PHIT ftr2', color='green', lw=2, zorder=2, alpha=0.5)
        # ax[5].plot(cutoff2, ind2, lw=2, color='black', linestyle = '--', label='phit_cutoff', alpha=0.33)
        # ax[5].set_title('well2 ' + offset_wells_list(dataset_offsets, well_test[1],'Balakhany VIII')['well2'].iloc[0], fontsize=9)
        # ax[5].set_yticks(range(min(ind2), max(ind2)+1, 50))
        # ax[5].grid(axis='y')

        # ax[6].plot(feature_x4, ind2, label='PHIT ftr1', color='green', lw=2, zorder=2, alpha=0.5)
        # ax[6].plot(cutoff2, ind2, lw=2, color='black', linestyle = '--', label='phit_cutoff', alpha=0.33)
        # ax[6].set_title('well1 ' + offset_wells_list(dataset_offsets, well_test[1],'Balakhany VIII')['well1'].iloc[0], fontsize=9)
        # ax[6].set_yticks(range(min(ind2), max(ind2)+1, 50))
        # ax[6].grid(axis='y')

        # ax[7].plot(target_y2, ind2, label='PHIT trgt', color='red', lw=2, zorder=2, alpha=0.5)
        # ax[7].plot(prediction_y2, ind2, label='PHIT pred', color='blue', lw=2, zorder=2, alpha=0.5)
        # ax[7].plot(cutoff2, ind2, lw=2, color='black', linestyle = '--', label='phit_cutoff', alpha=0.33)
        # ax[7].set_yticks(range(min(ind2), max(ind2)+1, 50))
        # ax[7].grid(axis='y')

        # ax[4].invert_yaxis(), ax[5].invert_yaxis(), ax[6].invert_yaxis(), ax[7].invert_yaxis()
        # ax[4].set_xlim(1, 0), ax[5].set_xlim(1, 0), ax[6].set_xlim(1, 0), ax[7].set_xlim(1, 0)
        # ax[7].set_title('Target ' + well_test[1], fontsize=11)

        # feature_x7 = dataset[(dataset.well_trgt == well_test[2]) & (dataset.FORMATION_up_trgt == fmname)]['PHIT_avg_ftr%_1']
        # feature_x8 = dataset[(dataset.well_trgt == well_test[2]) & (dataset.FORMATION_up_trgt == fmname)]['PHIT_avg_ftr%_2']
        # feature_x9 = dataset[(dataset.well_trgt == well_test[2]) & (dataset.FORMATION_up_trgt == fmname) ]['PHIT_avg_ftr%_3']
        # target_y1 = dataset[(dataset.well_trgt == well_test[2]) & (dataset.FORMATION_up_trgt == fmname)]['PHIT_avg_trgt%']
        # prediction_y1 = dataset[(dataset.well_trgt == well_test[2]) & (dataset.FORMATION_up_trgt == fmname)]['PHIT_avg_trgt%_pred']
        # target_ind1 = dataset[(dataset.well_trgt == well_test[2]) & (dataset.FORMATION_up_trgt == fmname)]['PHIT_avg_trgt%']
        # ind1 = [i for i in range(len(target_ind1))]
        # cutoff1 = [0.4 for i in range(len(target_ind1))]
        # ax[8].plot(feature_x7, ind1, label='PHIT ftr3', color='green', lw=2, zorder=2, alpha=0.5)
        # ax[8].plot(cutoff1, ind1, lw=2, color='black', linestyle = '--', label='phit_cutoff', alpha=0.66)
        # ax[8].set_title('well3 ' + offset_wells_list(dataset_offsets, well_test[2],'Balakhany VIII')['well3'].iloc[0], fontsize=9)
        # ax[8].set_yticks(range(min(ind1), max(ind1)+1, 50))
        # ax[8].grid(axis='y')

        # ax[9].plot(feature_x8, ind1, label='PHIT ftr2', color='green', lw=2, zorder=2, alpha=0.5)
        # ax[9].plot(cutoff1, ind1, lw=2, color='black', linestyle = '--', label='phit_cutoff', alpha=0.66)
        # ax[9].set_title('well2 ' + offset_wells_list(dataset_offsets, well_test[2],'Balakhany VIII')['well2'].iloc[0], fontsize=9)
        # ax[9].set_yticks(range(min(ind1), max(ind1)+1, 50))
        # ax[9].grid(axis='y')

        # ax[10].plot(feature_x9, ind1, label='PHIT ftr1', color='green', lw=2, zorder=2, alpha=0.5)
        # ax[10].plot(cutoff1, ind1, lw=2, color='black', linestyle = '--', label='phit_cutoff', alpha=0.66)
        # ax[10].set_title('well1 ' + offset_wells_list(dataset_offsets, well_test[2],'Balakhany VIII')['well1'].iloc[0], fontsize=9)
        # ax[10].set_yticks(range(min(ind1), max(ind1)+1, 50))
        # ax[10].grid(axis='y')

        # ax[11].plot(target_y1, ind1, label='PHIT trgt', color='red', lw=2, zorder=2, alpha=0.5)
        # ax[11].plot(prediction_y1, ind1, label='PHIT pred', color='blue', lw=2, zorder=2, alpha=0.5)
        # ax[11].plot(cutoff1, ind1, lw=2, color='black', linestyle = '--', label='phit_cutoff', alpha=0.66)
        # ax[11].set_yticks(range(min(ind1), max(ind1)+1, 50))
        # ax[11].grid(axis='y')

        # ax[8].invert_yaxis(), ax[9].invert_yaxis(), ax[10].invert_yaxis(), ax[11].invert_yaxis()
        # ax[8].set_xlim(1, 0), ax[9].set_xlim(1, 0), ax[10].set_xlim(1, 0), ax[11].set_xlim(1, 0)
        # ax[11].set_title('Target ' + well_test[2], fontsize=11);

In [ ]:
def self_data_proc(X_test, y_test_init, y_test,  y_pred):
    X_test_data = np.array(X_test[:,:,:,0]).flatten()
    y_test_data = np.array(y_test[:,:,0]).flatten()
    y_test_wellname = y_test_init[:,0].flatten()
    y_test_formation = y_test_init[:,1].flatten()
    y_pred_data = y_pred.flatten()
    y_test_full = np.stack((y_test_wellname, y_test_formation, y_test_data, y_pred_data, X_test_data), axis=1)
    result = pd.DataFrame(y_test_full, columns=['well_trgt','FORMATION_up_trgt','PHIT_trgt%', 'PHIT_trgt%_pred', 'xtest_PHIT_trgt%'])
    mae = mean_absolute_error(result['xtest_PHIT_trgt%'], result['PHIT_trgt%_pred'])
    print("Mean Absolute Error (MAE):", mae)
    return result
def self_data_display(dataset, fmname):
    well_test = dataset.well_trgt.unique()
    try:
        fig, ax = plt.subplots(1, 3, figsize=(7, 7))
        def track_display(dataset, fmname, well_num):
            target_y = dataset[(dataset.well_trgt == well_test[well_num]) & (dataset.FORMATION_up_trgt == fmname)]['xtest_PHIT_trgt%']
            prediction_y = dataset[(dataset.well_trgt == well_test[well_num]) & (dataset.FORMATION_up_trgt == fmname)]['PHIT_trgt%_pred']
            target_ind = dataset[(dataset.well_trgt == well_test[well_num]) & (dataset.FORMATION_up_trgt == fmname)]['xtest_PHIT_trgt%']
            ind = [i for i in range(len(target_ind))]
            ax[well_num].plot(target_y, ind, label='PHIT trgt', color='red', lw=4, zorder=2, alpha=0.5)
            ax[well_num].plot(prediction_y, ind, label='PHIT pred', color='blue', lw=2, zorder=2, alpha=0.5)
            ax[well_num].set_yticks(range(min(ind), max(ind), 50))
            ax[well_num].grid(axis='y')
            ax[well_num].invert_yaxis()
            ax[well_num].set_xlim(1, 0)
            ax[well_num].set_title('Target ' + well_test[well_num], fontsize=11)
            ax[well_num].tick_params(axis='both', labelsize=8)
        track_display(dataset, fmname, 0)
        track_display(dataset, fmname, 1)
        track_display(dataset, fmname, 2)
        plt.tight_layout()
    except Exception as e:
        print(f'Probably 1 or several wells are absent due to the error "{e}"')
    return fig.show()

self_full_df = self_data_proc(X_test_slf, y_test_fullds_slf, y_test_slf, y_pred_slf)
self_data_display(self_full_df, 'Balakhany VIII')

### 1D AENC 1f1t+collect

In [ ]:
final_phit_bal810_run = final_phit_bal810.copy()
final_phit_bal810_run = final_phit_bal810_run[['well_trgt', 'PHIT_trgt', 'PHIT_clp_trgt', 'TST_trgt','TVD_SCS_trgt', 
                                               'well', 'FORMATION_up', 'tst_index', 'PHIT_ftr', 'PHIT_clp_ftr', 'TST_ftr', 'TVD_SCS_ftr', 'offset']]
final_phit_bal810_run = final_phit_bal810_run.sort_values(by=['well_trgt','FORMATION_up','tst_index'])
aug_bal8 = aug_bal8.sort_values(by=['well_trgt','FORMATION_up','tst_index_sft'])
final_phit_bal8_run = final_phit_bal810_run[(final_phit_bal810_run.FORMATION_up == 'Balakhany VIII')]

def seq_train_test_wells_selection(dataset, test_well_qty, rerun):
    if rerun == 0:
        print('Well list is not randomized')
    if rerun == 1:
        well_list = list(dataset.well.unique())
        test_well_list = []
        for _ in range(test_well_qty):
            well = well_list[np.random.randint(1, len(well_list))]
            test_well_list.append(well)
        train_well_list = [x for x in well_list if x not in test_well_list]
    return train_well_list, test_well_list
train_well_list, test_well_list = seq_train_test_wells_selection(final_phit_bal8_run, 3, 1)
print('wells for test:', test_well_list)

def seq_train_wells_preparation(dataset_main, dataset_aug, train_well_list, block_size, var):
    train_dataset = dataset_main[dataset_main.well_trgt.isin(train_well_list)]
    aug_dataset = dataset_aug[dataset_aug.well_trgt.isin(train_well_list)]
    var_ftr = var + '_ftr'
    var_trgt = var + '_trgt'
    train_dataset['PHIT_ftr%'] = train_dataset[var_ftr]*100
    train_dataset['PHIT_trgt%'] = train_dataset[var_trgt]*100
    aug_dataset['PHIT_ftr%'] = aug_dataset[var_ftr]*100
    aug_dataset['PHIT_trgt%'] = aug_dataset[var_trgt]*100

    def feature_aug_stacking(train_dataset, aug_dataset, offset_num):
        feature = train_dataset[(train_dataset.offset==offset_num)]['PHIT_ftr%'].fillna(0).values
        aug = aug_dataset[(aug_dataset.offset==offset_num)]['PHIT_ftr%'].fillna(0).values
        concat_data = np.concatenate((feature, aug))
        # concat_data_norm = scaler.fit_transform(concat_data.reshape(-1,1))
        # result = concat_data_norm.reshape(-1, block_size, 1)  
        result = concat_data.reshape(-1, block_size, 1) 
        return result
    X_train_1 = feature_aug_stacking(train_dataset, aug_dataset, 1)
    X_train_2 = feature_aug_stacking(train_dataset, aug_dataset, 2)
    X_train_3 = feature_aug_stacking(train_dataset, aug_dataset, 3)
   
    y_well = train_dataset[     (train_dataset.offset==1)]['well_trgt'].values.reshape(-1, 1)
    y_fm = train_dataset[       (train_dataset.offset==1)]['FORMATION_up'].values.reshape(-1, 1)
    y_data = train_dataset[     (train_dataset.offset==1)]['PHIT_trgt%'].values.reshape(-1, 1)
    y_well_aug = aug_dataset[     (aug_dataset.offset==1)]['well_trgt'].values.reshape(-1, 1)
    y_fm_aug = aug_dataset[       (aug_dataset.offset==1)]['FORMATION_up'].values.reshape(-1, 1)
    y_data_aug = aug_dataset[     (aug_dataset.offset==1)]['PHIT_trgt%'].values.reshape(-1, 1)

    # y_data = scaler.fit_transform(y_data)
    # y_data_aug = scaler.fit_transform(y_data_aug)
    y_final = np.stack((y_well, y_fm, y_data), axis=1)
    y_final_aug = np.stack((y_well_aug, y_fm_aug, y_data_aug), axis=1)
    y_final_v2 = np.concatenate((y_final, y_final_aug))
    y_reshape = y_final_v2.reshape(-1, block_size, 1, 3)
    y_train = y_reshape[:,:,:,2].reshape(-1, block_size, 1)
    y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
    return X_train_1, X_train_2, X_train_3, y_train
X_train_1, X_train_2, X_train_3, y_train_seq = seq_train_wells_preparation(final_phit_bal8_run, aug_bal8, train_well_list, block_size, 'PHIT_clp')
print('X_train / y_train datasets: ',X_train_1.shape, y_train_seq.shape)

def seq_test_wells_preparation(dataset_main, test_well_list, block_size, var):
    test_dataset = dataset_main[dataset_main.well_trgt.isin(test_well_list)]
    var_ftr = var + '_ftr'
    var_trgt = var + '_trgt'
    test_dataset['PHIT_ftr%'] = test_dataset[var_ftr]*100
    test_dataset['PHIT_trgt%'] = test_dataset[var_trgt]*100

    def feature_offset(test_dataset, offset_num):
        phit_feature = test_dataset[(test_dataset.offset==offset_num)]['PHIT_ftr%'].fillna(0).values
        # phit_feature_norm = scaler.fit_transform(phit_feature.reshape(-1, 1))
        # X_test = phit_feature_norm.reshape(-1, block_size, 1)
        X_test = phit_feature.reshape(-1, block_size, 1)
        return X_test
    X_test_1 = feature_offset(test_dataset, 1)
    X_test_2 = feature_offset(test_dataset, 2)
    X_test_3 = feature_offset(test_dataset, 3)

    y_well = test_dataset[     (test_dataset.offset==1)]['well_trgt'].values.reshape(-1, 1)
    y_fm = test_dataset[       (test_dataset.offset==1)]['FORMATION_up'].values.reshape(-1, 1)
    y_data = test_dataset[     (test_dataset.offset==1)]['PHIT_trgt%'].values.reshape(-1, 1)

    # y_data = scaler.fit_transform(y_data)
    y_final = np.stack((y_well, y_fm, y_data), axis=1)
    y_reshape = y_final.reshape(-1, block_size, 1, 3)
    y_test = y_reshape[:,:,:,2].reshape(-1, block_size, 1)
    y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)
    return X_test_1, X_test_2, X_test_3, y_test, y_final, y_reshape
X_test_1, X_test_2, X_test_3, y_test_seq, y_test_fullds_seq, y_reshape = seq_test_wells_preparation(final_phit_bal8_run, test_well_list, block_size, 'PHIT_clp')
print('X_test / y_test / y_final datasets: ', X_test_1.shape, y_test_seq.shape, y_test_fullds_seq.shape)

In [ ]:
model_aenc_seq = Sequential([
    Conv2D(filters=256, kernel_size=(3,3), strides=1, activation='relu', padding='same', 
           kernel_initializer = RandomUniform(-1,1), input_shape =(block_size, 1, 1)),
    Conv2D(filters=32, kernel_size=(3,3), strides=2, activation = 'relu', padding='same'),
    Conv2DTranspose(filters=32, kernel_size=(1,1), strides=1, activation='relu', padding='same'),
    Conv2DTranspose(filters=256, kernel_size=(3,3), strides=2, activation = 'relu', padding='same'),
    Conv2DTranspose(filters=1, kernel_size=(3,3), strides=1, activation="relu", padding='same'),
    Flatten(),
    Dense(block_size)
    ])
opt_aenc_seq = Adam(learning_rate=0.01)
model_aenc_seq.summary()

def run_model_seq(X_train, y_train, X_test, y_test, model, optmzr, lss, eph, btch_sz, erl_stp, vrbs):
    model.compile(optimizer=optmzr, loss=lss)
    early_stop = EarlyStopping(monitor='loss', patience=10, verbose=1, restore_best_weights=True)
    if erl_stp == 1:
        history = model.fit(X_train, y_train, epochs=eph, batch_size=btch_sz, verbose=vrbs, validation_data=(X_test, y_test),callbacks=[early_stop])
    if erl_stp == 0:
        history = model.fit(X_train, y_train, epochs=eph, batch_size=btch_sz, verbose=vrbs, validation_data=(X_test, y_test))  
    y_pred = model.predict(X_test)
    plt.figure(figsize=(4, 3))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    return y_pred
y_pred_seq = run_model_seq(X_train_1, y_train_seq, X_test_1, y_test_seq, model_aenc_seq, opt_aenc_seq, 'mse', 250, 25, 1, 0)

In [2662]:
# display_3offset_wells('B32Z', 'Balakhany X')

In [ ]:
def seq_data_proc(X_test, y_test_init, y_test,  y_pred):
    X_test_data = np.array(X_test[:,:,0]).flatten()
    y_test_data = np.array(y_test[:,:,0]).flatten()
    y_test_wellname = y_test_init[:,0].flatten()
    y_test_formation = y_test_init[:,1].flatten()
    y_pred_data = y_pred.flatten()
    y_test_full = np.stack((y_test_wellname, y_test_formation, y_test_data, y_pred_data, X_test_data), axis=1)
    result = pd.DataFrame(y_test_full, columns=['well_trgt','FORMATION_up_trgt','PHIT_trgt%', 'PHIT_trgt%_pred', 'xtest_PHIT_ftr%'])
    mae = mean_absolute_error(result['xtest_PHIT_ftr%'], result['PHIT_trgt%_pred'])
    print("Mean Absolute Error (MAE):", mae)
    return result
def seq_data_display(dataset, dataset_offsets, fmname):
    well_test = dataset.well_trgt.unique()
    try:
        fig, ax = plt.subplots(1, 6, figsize=(12, 10))
        def track_display(dataset, well_test, well_num, track_num):
            feature = dataset[(dataset.well_trgt == well_test[well_num]) & (dataset.FORMATION_up_trgt == fmname)]['xtest_PHIT_ftr%']
            target = dataset[(dataset.well_trgt == well_test[well_num]) & (dataset.FORMATION_up_trgt == fmname)]['PHIT_trgt%']
            prediction = dataset[(dataset.well_trgt == well_test[well_num]) & (dataset.FORMATION_up_trgt == fmname)]['PHIT_trgt%_pred']
            target_ind = dataset[(dataset.well_trgt == well_test[well_num]) & (dataset.FORMATION_up_trgt == fmname)]['PHIT_trgt%_pred']
            ind = [i for i in range(len(target_ind))]
            cutoff = [13 for i in range(len(target_ind))]
            ax[track_num].plot(feature, ind, label='PHIT ftr1', color='green', lw=2, zorder=2, alpha=0.5)
            ax[track_num].plot(cutoff, ind, lw=2, color='black', linestyle = '--', label='phit_cutoff', alpha=0.33)
            ax[track_num].set_title('well1 ' + offset_wells_list(dataset_offsets, well_test[well_num],'Balakhany VIII')['well1'].iloc[0], fontsize=9)
            ax[track_num].set_yticks(range(min(ind), max(ind), 50))
            ax[track_num].invert_yaxis()
            ax[track_num].set_xlim(30, 0)
            ax[track_num].grid(axis='y')
            ax[track_num+1].plot(target, ind, label='PHIT trgt', color='red', lw=2, zorder=2, alpha=0.5)
            ax[track_num+1].plot(prediction, ind, label='PHIT pred', color='blue', lw=2, zorder=2, alpha=0.5)
            ax[track_num+1].plot(cutoff, ind, lw=2, color='black', linestyle = '--', label='phit_cutoff', alpha=0.33)
            ax[track_num+1].set_yticks(range(min(ind), max(ind), 50))
            ax[track_num+1].invert_yaxis()
            ax[track_num+1].set_xlim(30, 0)
            ax[track_num+1].grid(axis='y')
            ax[track_num+1].legend(fontsize=7)
            ax[track_num+1].set_title('Target ' + well_test[well_num], fontsize=11)
        track_display(dataset, well_test, 0, 0)
        track_display(dataset, well_test, 1, 2)
        track_display(dataset, well_test, 2, 4)
    except Exception as e:
        print(f'Probably 1 or several wells are absent due to the error "{e}"')
    return fig.show()

seq_full_df = seq_data_proc(X_test_1, y_test_fullds_seq, y_test_seq, y_pred_seq)
seq_data_display(seq_full_df, avg_prop_tst_kh_cut0, 'Balakhany VIII')

### PHIT_clipped

In [ ]:
def exercise_phit_avg_kh(dataset, wellname):
    well_a01w = dataset[(dataset.well==wellname) & (dataset.FORMATION_up=='Balakhany VIII')]

    well_a01w['PHIT_clipped'] = well_a01w['PHIT']
    well_a01w.loc[well_a01w.NET_clp2 == 0, 'PHIT_clipped'] = 0
    well_a01w['LPERM_avg'] = 0.00000002*(np.exp(well_a01w.PHIT*105.56))
    well_a01w.loc[well_a01w['PHIT'] >= 0.2, 'LPERM_avg'] = (7.7925*((well_a01w.PHIT*100)**2))-(29881.0*well_a01w.PHIT)+2891.8
    well_a01w.loc[well_a01w['PHIT'] < 0.16, 'LPERM_avg'] = 0.0159*(np.exp(well_a01w.PHIT*21.27))
    well_a01w['khtst'] = well_a01w.LPERM_avg*0.1
    well_a01w['KHtst_avg'] = well_a01w.loc[::-1, 'khtst'].cumsum()[::-1]

    y = well_a01w.TST
    phit_orig = well_a01w.PHIT_orig
    phit_avg = well_a01w.PHIT
    phit_cliped = well_a01w.PHIT_clipped
    net = well_a01w.NET_clp2
    perm = well_a01w.LPERM
    perm_avg = well_a01w.LPERM_avg
    kh = well_a01w.KHtst
    kh_avg = well_a01w.KHtst_avg
    print(  'KH orig:', kh.iloc[0].round(0), 
            'KH avg:',kh_avg.iloc[0].round(0), 
            'KHavg/KHorig:',((kh.iloc[0].round(0)-kh_avg.iloc[0].round(0))/kh.iloc[0].round(0)).round(2))

    fig, ax = plt.subplots(1, 4, figsize=(6, 7))
    ax[0].plot(phit_orig, y, color='green')
    ax[0].plot(phit_avg, y, color='red')
    ax[0].set_xlim(0, 0.3)
    ax[0].invert_yaxis()
    ax[0].set_title(wellname)
    ax[1].plot(phit_cliped, y, color='red', zorder=1)
    ax[1].plot(net, y, color='orange', zorder=0)
    ax[1].set_xlim(0, 0.3)
    ax[1].invert_yaxis()
    ax[2].plot(perm, y, color='purple', lw=3)
    ax[2].plot(perm_avg, y, color='yellow')
    ax[2].invert_yaxis()
    ax[2].set_xscale('log')
    ax[3].plot(kh, y, color='black')
    ax[3].plot(kh_avg, y, color='gray')
    ax[3].invert_yaxis()
    fig.show()
exercise_phit_avg_kh(well_bal8_interp_phavg_cut, 'J31')

###  PHIT - checking integrity 

In [ ]:
def well_letter_def(dataset):
    wells_letter = [wellname[0] for wellname in dataset.well.unique()]
    return set(wells_letter)
well_letter_def(well_bal8_interp_phavg_cut)

In [2510]:
# rows = 4
# columns = 9
# dataset = well_bal10_interp_phavg_cut
# wells_letter = [wellname for wellname in dataset.well.unique() if wellname.startswith('E')]
# # fig, ax = plt.subplots(rows,columns, figsize=(18,rows*2.5))
# counter = 0
# y_real_list = []
# for j in range(0, 4):
#     for i in range(0,9):
#         if counter < len(wells_letter):
#             data = dataset[dataset.well==wells_letter[counter]]
#             y_real_list.append(len(data))
#             print(wells_letter[counter], len(data))
#             counter +=1

In [ ]:
def well_plots_matrix(dataset, platform, variable, flag, max_var, comment):
    """
    flag = 'phit' or 'perm'
    """
    rows = 4
    columns = 9
    wells_letter = [wellname for wellname in dataset.well.unique() if wellname.startswith(platform)]
    fig, ax = plt.subplots(rows,columns, figsize=(18,rows*3))
    counter = 0
    y_real_list = []
    for j in range(0, rows):
        for i in range(0, columns):
            if counter < len(wells_letter):
                data = dataset[dataset.well==wells_letter[counter]]
                y_real_list.append(len(data))
                counter +=1
    max_ind = max(y_real_list)
    counter = 0
    for j in range(0, rows):
        for i in range(0, columns):
            if counter < len(wells_letter):
                well_data = dataset[dataset.well==wells_letter[counter]]
                ind = well_data[variable]
                y_real = [k for k in range(len(ind))]
                y_desired = [k for k in range(max_ind)]
                y_diff = len(y_desired) - len(y_real)
                values_to_add = [0.12 for k in range(y_diff)]
                x = well_data[variable]
                x_new = pd.concat([x, pd.Series(values_to_add)])              
                if flag == 'phit':
                    ax[j,i].plot(x_new, y_desired, color='green', lw=2, alpha=0.75)
                    ax[j,i].set_xlim(0.1, max_var)
                if flag == 'perm':
                    ax[j,i].plot(x_new, y_desired, color='purple', lw=2, alpha=0.75)
                    ax[j,i].set_xscale('log')
                    ax[j,i].set_xlim(0.1, max_var)
                ax[j,i].set_title(wells_letter[counter] + comment)
                ax[j,i].invert_yaxis()
                ax[j,i].grid()
                counter +=1
    return fig.tight_layout()

for letter in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J']:
    well_plots_matrix(well_bal8_interp_phavg_cut, letter, 'PHIT_clp', 'phit', 0.35, ' bal8')

In [ ]:
for letter in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J']:
    well_plots_matrix(well_bal10_interp_phavg_cut, letter, 'PHIT_clp', 'phit', 0.35, ' bal10')

### PHIT to picture

In [ ]:
a01w = well_bal8_interp_phavg_cut[well_bal8_interp_phavg_cut.well == 'A01W'][['TST','PHIT']]
a01w['PHIT_int'] = a01w['PHIT'].round(3)
a01w_pvt = a01w.pivot(index='TST', columns='PHIT_int', values='PHIT')
a01w_pvt = a01w_pvt.apply(lambda row: row.fillna(row.median()), axis=1)
a01w_pvt2 = a01w.pivot(index='TST', columns='PHIT_int', values='PHIT')
fig, ax = plt.subplots(1,2,figsize=(4,10))
# ax[0].imshow(a01w_pvt)
ax[0].imshow(a01w_pvt2.fillna(0), alpha=0.5)
ax[0].set_title('A01W');
ax[1].plot(a01w.PHIT, a01w.index, color='red', lw=2)
ax[1].invert_yaxis()

In [2706]:
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D
# import numpy as np

# bal8_3d =  df_bal_net2_kh[['well', 'FORMATION_up', 'TVD_SCS', 'X_traj','Y_traj','field']]
# bal8_3d = bal8_3d[(bal8_3d.FORMATION_up == 'Balakhany VIII')]

# # Create a 3D scatter plot
# fig = plt.figure(figsize=(10,10))
# ax = fig.add_subplot(111, projection='3d')
# # bal8_3d = bal8_3d[bal8_3d.well.isin(['B05', 'B06', 'B07'])]

# ax.scatter(bal8_3d.X_traj, bal8_3d.Y_traj, bal8_3d.TVD_SCS, c='gray', marker='o', s=5)  # Scatter plot with blue color and 'o' markers

# ax.set_xlabel('X Label')
# ax.set_ylabel('Y Label')
# ax.set_zlabel('Z Label')
# ax.invert_zaxis()
# ax.view_init(elev=30, azim=45)
# ax.set_title('3D Scatter Plot')

# plt.show()


In [ ]:
import numpy as np

## A noisy sine wave as query
idx = np.linspace(0,6.28,num=100)
query = np.sin(idx) + np.random.uniform(size=100)/10.0

## A cosine is for template; sin and cos are offset by 25 samples
template = np.cos(idx)

## Find the best match with the canonical recursion formula
from dtw import *
alignment = dtw(query, template, keep_internals=True)

## Display the warping curve, i.e. the alignment curve
alignment.plot(type="threeway")

## Align and plot with the Rabiner-Juang type VI-c unsmoothed recursion
dtw(query, template, keep_internals=True, 
    step_pattern=rabinerJuangStepPattern(6, "c"))\
    .plot(type="twoway",offset=-2)

## See the recursion relation, as formula and diagram
print(rabinerJuangStepPattern(6,"c"))
rabinerJuangStepPattern(6,"c").plot()